Librerias necesarias

In [82]:
import pandas as pd
import os 
import random
import uuid
import string
from faker import Faker
from datetime import datetime, timedelta
from IPython.display import display, Markdown # Libreria unicamente para visualizar datos markdown en jupyter

Es necesario definir donde se almacenaran las bases de datos generadas para mantener un orden

In [83]:
# Carpeta para almacenar las bases de datos generadas
carpeta_database = 'database'

# Crear una carpeta si esta no existe
if not os.path.exists(carpeta_database):
  os.makedirs(carpeta_database)

# Path para guardar el archivo
path_info_clientes = os.path.join(carpeta_database, 'info_clientes.xlsx')
path_inventario = os.path.join(carpeta_database, 'inventario.xlsx')
path_demanda = os.path.join(carpeta_database, 'demanda.xlsx')

# Generacion de base de datos para clientes



In [84]:
def generar_rut():
    # Genera un número aleatorio de 8 dígitos para el cuerpo del RUT 
    cuerpo_rut = str(random.randint(10000000, 20000000))

    # Calcula el dígito verificador utilizando el algoritmo de Módulo 11
    suma = sum(int(cuerpo_rut[i]) * (i % 6 + 2) for i in range(7))
    digito_verificador = 11 - (suma % 11)
    if digito_verificador == 11:
        digito_verificador = 0
    elif digito_verificador == 10:
        digito_verificador = 'K'

    # Combina el cuerpo y el dígito verificador para formar el RUT completo
    rut_completo = f"{cuerpo_rut}-{digito_verificador}"

    return rut_completo

In [85]:
fake = Faker('es_CL')

def generar_cliente():
  rut = generar_rut()
  nombre = fake.name()
  correo = fake.email()
  telefono = fake.phone_number()
  ciudad = fake.city()

  return {"Nombre": nombre, "Rut": rut, "Ciudad": ciudad, "Correo": correo, "Telefono": telefono}

cantidad_clientes = 1000

cliente = [generar_cliente() for _ in range (cantidad_clientes)]

df = pd.DataFrame(cliente)

# Guardar el archivo
df.to_excel(path_info_clientes, index=False)

# Visualizar en markdown
#display(Markdown(df.to_markdown()))

# Generacion de base de datos para inventario

In [86]:
# Listas de tipos de productos y sabores
tipos_productos = ['Torta', 'Queque', 'Brownie', 'Rosca', 'Dona', 'Galleta', 'Magdalena', 'Tarta', 'Cupcake']
sabores = ['Chocolate', 'Frutilla', 'Vainilla', 'Caramelo', 'Nuez', 'Mora', 'Maracuya', 'Limon', 'Canela', 'Coco', 'Frambuesa', 'Pera', 'Plátano', 'Mango', 'Pistacho', 'Café']
tamaños = ['Individual', 'Mini', 'Grande', 'Extra grande']
rellenos = ['Manjar', 'Dulce de leche', 'Crema pastelera', 'Crema de chocolate', 'Crema de frutilla']
cubiertas = ['Mani', 'Almendras', 'Chocolate negro', 'Chocolate blanco', 'Merengue', 'Crema', 'Nuez']
provedores = ['Panadería Fina & Co.', 'Dolce Gusto S.R.L.', 'Repostería Selecta', 'Delicias Artesanas', 'Pastelería Gourmet', 'Pan y Más']

## Genera un ID unico para cada producto segun su tipo

In [87]:

conteo_por_prefijo = {}
def generar_codigo_producto(tipos_producto):
    # Obtén los tres primeros caracteres de la descripción
    prefijo = tipos_producto[:3].upper()

    if prefijo not in conteo_por_prefijo:
        conteo_por_prefijo[prefijo] = 1
    else:
        conteo_por_prefijo[prefijo] += 1

    codigo_str = str(conteo_por_prefijo[prefijo]).zfill(4)

    return f"{prefijo}{codigo_str}"


In [88]:

def generar_descripcion():
  sabor = random.choice(sabores)
  tamaño = random.choice(tamaños)
  cubierta = random.choice(cubiertas)
  relleno = random.choice(rellenos)

  return sabor, tamaño, cubierta, relleno


def generar_producto():
  tipo_producto = random.choice(tipos_productos)
  sabor, tamaño, cubierta, relleno = generar_descripcion()
  descripcion = f"{tipo_producto} {tamaño} sabor {sabor} relleno de {relleno} y cubierto de {cubierta}"
  id_producto = generar_codigo_producto(tipo_producto)
  precio_venta = fake.random_int(min=1000, max=10000)
  coste_compra = precio_venta - (precio_venta * random.uniform(0.4, 0.9))
  provedor = random.choice(provedores)

  return {"Tipo de producto": tipo_producto, "ID Producto": id_producto, "Descripcion": descripcion,"Precio de venta": precio_venta,"Coste de compra": coste_compra, "Proveedor": provedor}

num_productos = 1000

producto = [generar_producto() for _ in range (num_productos)]

df = pd.DataFrame(producto)

#Guardar el archivo
df.to_excel(path_inventario, index=False)

# Visualizar en markdown
#display(Markdown(df.to_markdown()))

|     | Tipo de producto   | ID Producto   | Descripcion                                                                                       |   Precio de venta |   Coste de compra | Proveedor            |
|----:|:-------------------|:--------------|:--------------------------------------------------------------------------------------------------|------------------:|------------------:|:---------------------|
|   0 | Rosca              | ROS0001       | Rosca Mini sabor Nuez relleno de Crema de chocolate y cubierto de Chocolate blanco                |              2634 |          1793.58  | Panadería Fina & Co. |
|   1 | Torta              | TOR0001       | Torta Extra grande sabor Café relleno de Crema de frutilla y cubierto de Chocolate blanco         |              2871 |          2286.45  | Repostería Selecta   |
|   2 | Brownie            | BRO0001       | Brownie Individual sabor Canela relleno de Crema pastelera y cubierto de Chocolate negro          |              8443 |          6064     | Pastelería Gourmet   |
|   3 | Rosca              | ROS0002       | Rosca Mini sabor Frutilla relleno de Dulce de leche y cubierto de Nuez                            |              4264 |          2947.45  | Repostería Selecta   |
|   4 | Dona               | DON0001       | Dona Individual sabor Vainilla relleno de Crema de chocolate y cubierto de Merengue               |              3836 |          2870.13  | Delicias Artesanas   |
|   5 | Brownie            | BRO0002       | Brownie Individual sabor Coco relleno de Manjar y cubierto de Crema                               |              9466 |          6489.35  | Repostería Selecta   |
|   6 | Dona               | DON0002       | Dona Extra grande sabor Vainilla relleno de Dulce de leche y cubierto de Nuez                     |              5104 |          2783.92  | Delicias Artesanas   |
|   7 | Cupcake            | CUP0001       | Cupcake Mini sabor Plátano relleno de Manjar y cubierto de Chocolate negro                        |              7389 |          5625.72  | Repostería Selecta   |
|   8 | Brownie            | BRO0003       | Brownie Individual sabor Mora relleno de Crema de frutilla y cubierto de Chocolate blanco         |              3355 |          2011.12  | Pan y Más            |
|   9 | Queque             | QUE0001       | Queque Extra grande sabor Canela relleno de Crema pastelera y cubierto de Crema                   |              2362 |          1735.71  | Pan y Más            |
|  10 | Rosca              | ROS0003       | Rosca Extra grande sabor Pera relleno de Crema pastelera y cubierto de Mani                       |              1098 |           562.106 | Pan y Más            |
|  11 | Queque             | QUE0002       | Queque Grande sabor Café relleno de Dulce de leche y cubierto de Merengue                         |              9255 |          6350.21  | Panadería Fina & Co. |
|  12 | Brownie            | BRO0004       | Brownie Grande sabor Frambuesa relleno de Crema pastelera y cubierto de Mani                      |              6249 |          3784.95  | Panadería Fina & Co. |
|  13 | Dona               | DON0003       | Dona Grande sabor Canela relleno de Manjar y cubierto de Chocolate blanco                         |              1956 |          1481.24  | Panadería Fina & Co. |
|  14 | Brownie            | BRO0005       | Brownie Mini sabor Maracuya relleno de Crema de frutilla y cubierto de Mani                       |              5526 |          3946.47  | Pastelería Gourmet   |
|  15 | Torta              | TOR0002       | Torta Grande sabor Pistacho relleno de Crema de frutilla y cubierto de Crema                      |              7863 |          5178.45  | Dolce Gusto S.R.L.   |
|  16 | Torta              | TOR0003       | Torta Extra grande sabor Pistacho relleno de Crema de frutilla y cubierto de Chocolate blanco     |              5409 |          3471.73  | Pan y Más            |
|  17 | Brownie            | BRO0006       | Brownie Extra grande sabor Caramelo relleno de Crema de chocolate y cubierto de Mani              |              7716 |          4059.29  | Repostería Selecta   |
|  18 | Brownie            | BRO0007       | Brownie Mini sabor Vainilla relleno de Crema pastelera y cubierto de Merengue                     |              9775 |          5489.09  | Dolce Gusto S.R.L.   |
|  19 | Rosca              | ROS0004       | Rosca Individual sabor Vainilla relleno de Manjar y cubierto de Crema                             |              9010 |          7185.28  | Repostería Selecta   |
|  20 | Galleta            | GAL0001       | Galleta Grande sabor Limon relleno de Crema pastelera y cubierto de Almendras                     |              8022 |          6107.34  | Delicias Artesanas   |
|  21 | Queque             | QUE0003       | Queque Extra grande sabor Coco relleno de Dulce de leche y cubierto de Chocolate negro            |              3660 |          2830.1   | Pastelería Gourmet   |
|  22 | Galleta            | GAL0002       | Galleta Grande sabor Mango relleno de Crema pastelera y cubierto de Almendras                     |              2738 |          1398.79  | Dolce Gusto S.R.L.   |
|  23 | Galleta            | GAL0003       | Galleta Individual sabor Frutilla relleno de Crema de frutilla y cubierto de Nuez                 |              4848 |          3788.83  | Pan y Más            |
|  24 | Brownie            | BRO0008       | Brownie Individual sabor Pera relleno de Crema de frutilla y cubierto de Mani                     |              6998 |          5248.42  | Delicias Artesanas   |
|  25 | Queque             | QUE0004       | Queque Grande sabor Coco relleno de Dulce de leche y cubierto de Mani                             |              2826 |          1905.69  | Delicias Artesanas   |
|  26 | Galleta            | GAL0004       | Galleta Extra grande sabor Frambuesa relleno de Crema de chocolate y cubierto de Chocolate blanco |              4535 |          2668.03  | Pan y Más            |
|  27 | Galleta            | GAL0005       | Galleta Extra grande sabor Café relleno de Manjar y cubierto de Merengue                          |              1159 |           818.924 | Panadería Fina & Co. |
|  28 | Brownie            | BRO0009       | Brownie Mini sabor Plátano relleno de Manjar y cubierto de Nuez                                   |              8541 |          6654.68  | Panadería Fina & Co. |
|  29 | Torta              | TOR0004       | Torta Extra grande sabor Coco relleno de Manjar y cubierto de Nuez                                |              3168 |          2074.75  | Pan y Más            |
|  30 | Cupcake            | CUP0002       | Cupcake Extra grande sabor Frambuesa relleno de Dulce de leche y cubierto de Nuez                 |              2855 |          2254.44  | Delicias Artesanas   |
|  31 | Torta              | TOR0005       | Torta Grande sabor Canela relleno de Crema de chocolate y cubierto de Nuez                        |              5077 |          3129.63  | Panadería Fina & Co. |
|  32 | Brownie            | BRO0010       | Brownie Grande sabor Coco relleno de Dulce de leche y cubierto de Almendras                       |              3059 |          2443.72  | Pan y Más            |
|  33 | Queque             | QUE0005       | Queque Extra grande sabor Mango relleno de Dulce de leche y cubierto de Merengue                  |              7054 |          4913.69  | Dolce Gusto S.R.L.   |
|  34 | Cupcake            | CUP0003       | Cupcake Mini sabor Canela relleno de Dulce de leche y cubierto de Mani                            |              4849 |          3514.67  | Dolce Gusto S.R.L.   |
|  35 | Dona               | DON0004       | Dona Grande sabor Caramelo relleno de Crema pastelera y cubierto de Crema                         |              3600 |          1817.74  | Pan y Más            |
|  36 | Magdalena          | MAG0001       | Magdalena Individual sabor Frambuesa relleno de Dulce de leche y cubierto de Almendras            |              6262 |          3461.02  | Dolce Gusto S.R.L.   |
|  37 | Torta              | TOR0006       | Torta Mini sabor Limon relleno de Crema de frutilla y cubierto de Crema                           |              9403 |          6620.84  | Panadería Fina & Co. |
|  38 | Rosca              | ROS0005       | Rosca Grande sabor Frutilla relleno de Crema de frutilla y cubierto de Chocolate blanco           |              1514 |           840.599 | Pastelería Gourmet   |
|  39 | Tarta              | TAR0001       | Tarta Individual sabor Mora relleno de Crema de chocolate y cubierto de Chocolate blanco          |              9934 |          6065.54  | Panadería Fina & Co. |
|  40 | Brownie            | BRO0011       | Brownie Grande sabor Café relleno de Crema de frutilla y cubierto de Mani                         |              2677 |          1361.53  | Pan y Más            |
|  41 | Cupcake            | CUP0004       | Cupcake Grande sabor Chocolate relleno de Dulce de leche y cubierto de Nuez                       |              5516 |          3617.32  | Pastelería Gourmet   |
|  42 | Cupcake            | CUP0005       | Cupcake Mini sabor Vainilla relleno de Crema pastelera y cubierto de Almendras                    |              8253 |          4497.31  | Pastelería Gourmet   |
|  43 | Rosca              | ROS0006       | Rosca Mini sabor Pistacho relleno de Dulce de leche y cubierto de Nuez                            |              2738 |          2124.97  | Delicias Artesanas   |
|  44 | Magdalena          | MAG0002       | Magdalena Grande sabor Pera relleno de Crema de frutilla y cubierto de Merengue                   |              9628 |          7395.49  | Dolce Gusto S.R.L.   |
|  45 | Dona               | DON0005       | Dona Extra grande sabor Mango relleno de Dulce de leche y cubierto de Crema                       |              7206 |          5024.6   | Panadería Fina & Co. |
|  46 | Dona               | DON0006       | Dona Extra grande sabor Nuez relleno de Manjar y cubierto de Chocolate negro                      |              3382 |          2346.71  | Dolce Gusto S.R.L.   |
|  47 | Galleta            | GAL0006       | Galleta Individual sabor Plátano relleno de Crema de chocolate y cubierto de Mani                 |              5940 |          3584.95  | Delicias Artesanas   |
|  48 | Dona               | DON0007       | Dona Extra grande sabor Caramelo relleno de Crema pastelera y cubierto de Almendras               |              6452 |          3667.39  | Delicias Artesanas   |
|  49 | Magdalena          | MAG0003       | Magdalena Mini sabor Canela relleno de Crema pastelera y cubierto de Almendras                    |              8931 |          5865.18  | Dolce Gusto S.R.L.   |
|  50 | Cupcake            | CUP0006       | Cupcake Grande sabor Café relleno de Manjar y cubierto de Nuez                                    |              7234 |          5449.51  | Pastelería Gourmet   |
|  51 | Galleta            | GAL0007       | Galleta Extra grande sabor Vainilla relleno de Crema pastelera y cubierto de Merengue             |              3992 |          2462.01  | Delicias Artesanas   |
|  52 | Queque             | QUE0006       | Queque Extra grande sabor Maracuya relleno de Manjar y cubierto de Nuez                           |              9233 |          7173.98  | Dolce Gusto S.R.L.   |
|  53 | Rosca              | ROS0007       | Rosca Grande sabor Coco relleno de Manjar y cubierto de Chocolate blanco                          |              4923 |          2536.49  | Pastelería Gourmet   |
|  54 | Rosca              | ROS0008       | Rosca Individual sabor Pera relleno de Crema de frutilla y cubierto de Merengue                   |              2179 |          1345.88  | Pan y Más            |
|  55 | Magdalena          | MAG0004       | Magdalena Grande sabor Pistacho relleno de Dulce de leche y cubierto de Almendras                 |              1310 |           944.478 | Repostería Selecta   |
|  56 | Rosca              | ROS0009       | Rosca Mini sabor Chocolate relleno de Manjar y cubierto de Almendras                              |              2523 |          1326.01  | Dolce Gusto S.R.L.   |
|  57 | Rosca              | ROS0010       | Rosca Grande sabor Café relleno de Crema pastelera y cubierto de Nuez                             |              7784 |          6106.1   | Pan y Más            |
|  58 | Torta              | TOR0007       | Torta Extra grande sabor Vainilla relleno de Crema pastelera y cubierto de Almendras              |              2098 |          1561.77  | Repostería Selecta   |
|  59 | Dona               | DON0008       | Dona Mini sabor Pistacho relleno de Crema de chocolate y cubierto de Mani                         |              6984 |          4099.5   | Panadería Fina & Co. |
|  60 | Queque             | QUE0007       | Queque Grande sabor Chocolate relleno de Dulce de leche y cubierto de Chocolate negro             |              4683 |          3678.54  | Repostería Selecta   |
|  61 | Galleta            | GAL0008       | Galleta Mini sabor Caramelo relleno de Manjar y cubierto de Almendras                             |              9178 |          4944.17  | Pan y Más            |
|  62 | Cupcake            | CUP0007       | Cupcake Grande sabor Maracuya relleno de Crema de chocolate y cubierto de Almendras               |              2459 |          1483.53  | Delicias Artesanas   |
|  63 | Magdalena          | MAG0005       | Magdalena Grande sabor Frutilla relleno de Dulce de leche y cubierto de Chocolate negro           |              4992 |          2928.77  | Pan y Más            |
|  64 | Tarta              | TAR0002       | Tarta Mini sabor Café relleno de Dulce de leche y cubierto de Chocolate blanco                    |              6607 |          4845.65  | Delicias Artesanas   |
|  65 | Torta              | TOR0008       | Torta Individual sabor Pera relleno de Crema de chocolate y cubierto de Crema                     |              5347 |          3441.95  | Panadería Fina & Co. |
|  66 | Queque             | QUE0008       | Queque Mini sabor Plátano relleno de Crema de frutilla y cubierto de Mani                         |              9821 |          5900.87  | Dolce Gusto S.R.L.   |
|  67 | Brownie            | BRO0012       | Brownie Extra grande sabor Frutilla relleno de Manjar y cubierto de Crema                         |              4329 |          2812.1   | Panadería Fina & Co. |
|  68 | Torta              | TOR0009       | Torta Mini sabor Maracuya relleno de Dulce de leche y cubierto de Almendras                       |              7688 |          4367.89  | Repostería Selecta   |
|  69 | Queque             | QUE0009       | Queque Extra grande sabor Frutilla relleno de Manjar y cubierto de Almendras                      |              8928 |          4831.12  | Panadería Fina & Co. |
|  70 | Tarta              | TAR0003       | Tarta Extra grande sabor Pistacho relleno de Crema de chocolate y cubierto de Chocolate blanco    |              4412 |          2263.56  | Panadería Fina & Co. |
|  71 | Dona               | DON0009       | Dona Mini sabor Chocolate relleno de Crema pastelera y cubierto de Almendras                      |              4896 |          3769.76  | Delicias Artesanas   |
|  72 | Dona               | DON0010       | Dona Individual sabor Chocolate relleno de Dulce de leche y cubierto de Mani                      |              8981 |          6129.32  | Panadería Fina & Co. |
|  73 | Dona               | DON0011       | Dona Extra grande sabor Limon relleno de Crema de chocolate y cubierto de Mani                    |              8503 |          4661.19  | Pan y Más            |
|  74 | Brownie            | BRO0013       | Brownie Individual sabor Maracuya relleno de Crema de frutilla y cubierto de Mani                 |              4248 |          2923.65  | Dolce Gusto S.R.L.   |
|  75 | Torta              | TOR0010       | Torta Extra grande sabor Frutilla relleno de Dulce de leche y cubierto de Chocolate blanco        |              5454 |          4029.01  | Panadería Fina & Co. |
|  76 | Dona               | DON0012       | Dona Individual sabor Coco relleno de Dulce de leche y cubierto de Nuez                           |              6494 |          3494.47  | Pastelería Gourmet   |
|  77 | Magdalena          | MAG0006       | Magdalena Mini sabor Café relleno de Manjar y cubierto de Crema                                   |              4931 |          3643.13  | Panadería Fina & Co. |
|  78 | Rosca              | ROS0011       | Rosca Mini sabor Mora relleno de Crema de chocolate y cubierto de Mani                            |              3264 |          1734.46  | Repostería Selecta   |
|  79 | Brownie            | BRO0014       | Brownie Mini sabor Nuez relleno de Crema pastelera y cubierto de Chocolate negro                  |              1700 |           921.346 | Repostería Selecta   |
|  80 | Brownie            | BRO0015       | Brownie Grande sabor Limon relleno de Crema pastelera y cubierto de Mani                          |              3443 |          2492.79  | Pan y Más            |
|  81 | Queque             | QUE0010       | Queque Mini sabor Canela relleno de Dulce de leche y cubierto de Crema                            |              9231 |          6313.42  | Dolce Gusto S.R.L.   |
|  82 | Galleta            | GAL0009       | Galleta Extra grande sabor Limon relleno de Crema de frutilla y cubierto de Crema                 |              5595 |          4420.04  | Pastelería Gourmet   |
|  83 | Tarta              | TAR0004       | Tarta Individual sabor Frambuesa relleno de Crema de chocolate y cubierto de Mani                 |              4408 |          2488.11  | Repostería Selecta   |
|  84 | Rosca              | ROS0012       | Rosca Individual sabor Caramelo relleno de Crema de frutilla y cubierto de Chocolate negro        |              7596 |          4393.62  | Delicias Artesanas   |
|  85 | Galleta            | GAL0010       | Galleta Extra grande sabor Caramelo relleno de Dulce de leche y cubierto de Chocolate negro       |              2484 |          1980.23  | Pan y Más            |
|  86 | Cupcake            | CUP0008       | Cupcake Mini sabor Caramelo relleno de Crema de frutilla y cubierto de Chocolate blanco           |              8747 |          5724.9   | Panadería Fina & Co. |
|  87 | Galleta            | GAL0011       | Galleta Individual sabor Chocolate relleno de Crema de chocolate y cubierto de Merengue           |              5278 |          4163.22  | Pan y Más            |
|  88 | Dona               | DON0013       | Dona Extra grande sabor Nuez relleno de Dulce de leche y cubierto de Mani                         |              3176 |          2465.53  | Pastelería Gourmet   |
|  89 | Tarta              | TAR0005       | Tarta Grande sabor Canela relleno de Dulce de leche y cubierto de Merengue                        |              1985 |          1495.18  | Pastelería Gourmet   |
|  90 | Galleta            | GAL0012       | Galleta Mini sabor Caramelo relleno de Manjar y cubierto de Chocolate negro                       |              7949 |          5009     | Pastelería Gourmet   |
|  91 | Tarta              | TAR0006       | Tarta Mini sabor Chocolate relleno de Manjar y cubierto de Merengue                               |              4713 |          2851.88  | Panadería Fina & Co. |
|  92 | Cupcake            | CUP0009       | Cupcake Grande sabor Frutilla relleno de Crema de frutilla y cubierto de Crema                    |              6969 |          5052.01  | Repostería Selecta   |
|  93 | Brownie            | BRO0016       | Brownie Individual sabor Caramelo relleno de Manjar y cubierto de Almendras                       |              1973 |          1349.24  | Repostería Selecta   |
|  94 | Queque             | QUE0011       | Queque Mini sabor Vainilla relleno de Crema pastelera y cubierto de Mani                          |              7975 |          4678.01  | Panadería Fina & Co. |
|  95 | Queque             | QUE0012       | Queque Grande sabor Mora relleno de Crema pastelera y cubierto de Merengue                        |              3973 |          2840.45  | Pan y Más            |
|  96 | Magdalena          | MAG0007       | Magdalena Extra grande sabor Vainilla relleno de Dulce de leche y cubierto de Nuez                |              2642 |          1832.64  | Repostería Selecta   |
|  97 | Magdalena          | MAG0008       | Magdalena Extra grande sabor Limon relleno de Manjar y cubierto de Nuez                           |              8351 |          6367.72  | Dolce Gusto S.R.L.   |
|  98 | Torta              | TOR0011       | Torta Mini sabor Pistacho relleno de Crema de chocolate y cubierto de Merengue                    |              2461 |          1813.14  | Delicias Artesanas   |
|  99 | Magdalena          | MAG0009       | Magdalena Grande sabor Vainilla relleno de Crema pastelera y cubierto de Mani                     |              6080 |          4443.95  | Dolce Gusto S.R.L.   |
| 100 | Torta              | TOR0012       | Torta Mini sabor Maracuya relleno de Crema de chocolate y cubierto de Chocolate negro             |              9852 |          6895.88  | Pan y Más            |
| 101 | Cupcake            | CUP0010       | Cupcake Grande sabor Limon relleno de Crema pastelera y cubierto de Chocolate negro               |              6188 |          4712.38  | Dolce Gusto S.R.L.   |
| 102 | Tarta              | TAR0007       | Tarta Grande sabor Plátano relleno de Crema de chocolate y cubierto de Nuez                       |              5692 |          3045.29  | Repostería Selecta   |
| 103 | Dona               | DON0014       | Dona Mini sabor Maracuya relleno de Crema de frutilla y cubierto de Crema                         |              2702 |          1462.35  | Repostería Selecta   |
| 104 | Galleta            | GAL0013       | Galleta Extra grande sabor Vainilla relleno de Manjar y cubierto de Chocolate blanco              |              3944 |          2494.63  | Panadería Fina & Co. |
| 105 | Galleta            | GAL0014       | Galleta Extra grande sabor Mango relleno de Manjar y cubierto de Nuez                             |              7285 |          4095.84  | Pan y Más            |
| 106 | Brownie            | BRO0017       | Brownie Extra grande sabor Caramelo relleno de Crema pastelera y cubierto de Chocolate negro      |              2115 |          1353.14  | Pastelería Gourmet   |
| 107 | Cupcake            | CUP0011       | Cupcake Grande sabor Caramelo relleno de Crema pastelera y cubierto de Nuez                       |              7246 |          5550.55  | Panadería Fina & Co. |
| 108 | Magdalena          | MAG0010       | Magdalena Individual sabor Nuez relleno de Crema pastelera y cubierto de Almendras                |              7460 |          4439.32  | Pan y Más            |
| 109 | Brownie            | BRO0018       | Brownie Mini sabor Frambuesa relleno de Dulce de leche y cubierto de Nuez                         |              2426 |          1715.6   | Delicias Artesanas   |
| 110 | Brownie            | BRO0019       | Brownie Grande sabor Mora relleno de Crema de frutilla y cubierto de Mani                         |              3508 |          2099.04  | Panadería Fina & Co. |
| 111 | Brownie            | BRO0020       | Brownie Mini sabor Coco relleno de Dulce de leche y cubierto de Mani                              |              9459 |          6040.84  | Delicias Artesanas   |
| 112 | Brownie            | BRO0021       | Brownie Mini sabor Café relleno de Crema de chocolate y cubierto de Chocolate blanco              |              8224 |          5646.14  | Pastelería Gourmet   |
| 113 | Magdalena          | MAG0011       | Magdalena Individual sabor Caramelo relleno de Dulce de leche y cubierto de Crema                 |              7243 |          4385.64  | Pan y Más            |
| 114 | Magdalena          | MAG0012       | Magdalena Extra grande sabor Café relleno de Dulce de leche y cubierto de Almendras               |              9180 |          5164.64  | Delicias Artesanas   |
| 115 | Rosca              | ROS0013       | Rosca Individual sabor Pera relleno de Crema de frutilla y cubierto de Chocolate blanco           |              1748 |           884.529 | Repostería Selecta   |
| 116 | Cupcake            | CUP0012       | Cupcake Extra grande sabor Mora relleno de Crema de frutilla y cubierto de Mani                   |              8465 |          4908.81  | Delicias Artesanas   |
| 117 | Dona               | DON0015       | Dona Individual sabor Mango relleno de Manjar y cubierto de Mani                                  |              5993 |          3778.57  | Panadería Fina & Co. |
| 118 | Magdalena          | MAG0013       | Magdalena Grande sabor Maracuya relleno de Dulce de leche y cubierto de Merengue                  |              6334 |          5066.04  | Delicias Artesanas   |
| 119 | Torta              | TOR0013       | Torta Mini sabor Mango relleno de Crema de frutilla y cubierto de Crema                           |              2755 |          1667.26  | Delicias Artesanas   |
| 120 | Galleta            | GAL0015       | Galleta Grande sabor Coco relleno de Crema de frutilla y cubierto de Chocolate negro              |              3460 |          1911.88  | Dolce Gusto S.R.L.   |
| 121 | Queque             | QUE0013       | Queque Grande sabor Plátano relleno de Dulce de leche y cubierto de Nuez                          |              2958 |          1966.23  | Pastelería Gourmet   |
| 122 | Torta              | TOR0014       | Torta Mini sabor Pistacho relleno de Crema de chocolate y cubierto de Mani                        |              7641 |          5063.38  | Delicias Artesanas   |
| 123 | Torta              | TOR0015       | Torta Grande sabor Limon relleno de Manjar y cubierto de Chocolate negro                          |              9543 |          7337.19  | Panadería Fina & Co. |
| 124 | Rosca              | ROS0014       | Rosca Grande sabor Vainilla relleno de Crema de chocolate y cubierto de Crema                     |              1497 |           896.217 | Dolce Gusto S.R.L.   |
| 125 | Queque             | QUE0014       | Queque Extra grande sabor Pera relleno de Dulce de leche y cubierto de Merengue                   |              3477 |          1903.49  | Pastelería Gourmet   |
| 126 | Tarta              | TAR0008       | Tarta Grande sabor Pistacho relleno de Crema de chocolate y cubierto de Almendras                 |              7266 |          5234.11  | Pastelería Gourmet   |
| 127 | Brownie            | BRO0022       | Brownie Grande sabor Plátano relleno de Manjar y cubierto de Almendras                            |              3256 |          1872.62  | Pan y Más            |
| 128 | Galleta            | GAL0016       | Galleta Individual sabor Caramelo relleno de Crema de frutilla y cubierto de Merengue             |              1868 |          1421.42  | Pan y Más            |
| 129 | Dona               | DON0016       | Dona Extra grande sabor Mango relleno de Crema de chocolate y cubierto de Almendras               |              1392 |          1099.36  | Repostería Selecta   |
| 130 | Queque             | QUE0015       | Queque Mini sabor Caramelo relleno de Crema de frutilla y cubierto de Merengue                    |              7756 |          6034.61  | Delicias Artesanas   |
| 131 | Magdalena          | MAG0014       | Magdalena Extra grande sabor Coco relleno de Crema de chocolate y cubierto de Chocolate negro     |              5791 |          3112.77  | Panadería Fina & Co. |
| 132 | Dona               | DON0017       | Dona Grande sabor Pistacho relleno de Crema de frutilla y cubierto de Chocolate blanco            |              3414 |          1888.33  | Pastelería Gourmet   |
| 133 | Torta              | TOR0016       | Torta Individual sabor Mora relleno de Crema de frutilla y cubierto de Merengue                   |              9993 |          7378.44  | Dolce Gusto S.R.L.   |
| 134 | Magdalena          | MAG0015       | Magdalena Grande sabor Mango relleno de Crema pastelera y cubierto de Nuez                        |              8171 |          5729.72  | Panadería Fina & Co. |
| 135 | Tarta              | TAR0009       | Tarta Individual sabor Canela relleno de Crema de chocolate y cubierto de Merengue                |              1143 |           704.106 | Repostería Selecta   |
| 136 | Tarta              | TAR0010       | Tarta Grande sabor Maracuya relleno de Dulce de leche y cubierto de Mani                          |              8925 |          6773.73  | Delicias Artesanas   |
| 137 | Queque             | QUE0016       | Queque Individual sabor Coco relleno de Crema pastelera y cubierto de Chocolate blanco            |              3958 |          2439.55  | Dolce Gusto S.R.L.   |
| 138 | Brownie            | BRO0023       | Brownie Individual sabor Pistacho relleno de Crema pastelera y cubierto de Mani                   |              3528 |          2804.17  | Dolce Gusto S.R.L.   |
| 139 | Cupcake            | CUP0013       | Cupcake Individual sabor Vainilla relleno de Crema de chocolate y cubierto de Almendras           |              9953 |          5566.4   | Panadería Fina & Co. |
| 140 | Dona               | DON0018       | Dona Extra grande sabor Coco relleno de Manjar y cubierto de Chocolate negro                      |              6211 |          4097.24  | Delicias Artesanas   |
| 141 | Dona               | DON0019       | Dona Extra grande sabor Frambuesa relleno de Manjar y cubierto de Crema                           |              1770 |          1334.6   | Delicias Artesanas   |
| 142 | Galleta            | GAL0017       | Galleta Mini sabor Mora relleno de Manjar y cubierto de Chocolate negro                           |              7742 |          4269.11  | Repostería Selecta   |
| 143 | Torta              | TOR0017       | Torta Mini sabor Plátano relleno de Crema pastelera y cubierto de Crema                           |              1086 |           815.567 | Dolce Gusto S.R.L.   |
| 144 | Tarta              | TAR0011       | Tarta Individual sabor Mora relleno de Dulce de leche y cubierto de Merengue                      |              2474 |          1338.08  | Panadería Fina & Co. |
| 145 | Rosca              | ROS0015       | Rosca Mini sabor Nuez relleno de Manjar y cubierto de Mani                                        |              5748 |          2926.35  | Delicias Artesanas   |
| 146 | Brownie            | BRO0024       | Brownie Extra grande sabor Limon relleno de Crema de frutilla y cubierto de Chocolate blanco      |              1551 |           838.796 | Dolce Gusto S.R.L.   |
| 147 | Cupcake            | CUP0014       | Cupcake Individual sabor Mora relleno de Crema de chocolate y cubierto de Crema                   |              1011 |           522.544 | Pan y Más            |
| 148 | Galleta            | GAL0018       | Galleta Grande sabor Vainilla relleno de Crema de chocolate y cubierto de Nuez                    |              5161 |          2873.49  | Panadería Fina & Co. |
| 149 | Torta              | TOR0018       | Torta Grande sabor Maracuya relleno de Dulce de leche y cubierto de Mani                          |              6744 |          4813.46  | Dolce Gusto S.R.L.   |
| 150 | Brownie            | BRO0025       | Brownie Individual sabor Frambuesa relleno de Crema pastelera y cubierto de Chocolate blanco      |              9936 |          6266.56  | Delicias Artesanas   |
| 151 | Brownie            | BRO0026       | Brownie Grande sabor Vainilla relleno de Crema pastelera y cubierto de Almendras                  |              1919 |          1244.81  | Panadería Fina & Co. |
| 152 | Brownie            | BRO0027       | Brownie Individual sabor Frutilla relleno de Crema pastelera y cubierto de Merengue               |              3175 |          1627.33  | Panadería Fina & Co. |
| 153 | Dona               | DON0020       | Dona Extra grande sabor Frutilla relleno de Crema de frutilla y cubierto de Almendras             |              8277 |          5714.43  | Pan y Más            |
| 154 | Brownie            | BRO0028       | Brownie Individual sabor Plátano relleno de Crema de chocolate y cubierto de Mani                 |              7588 |          5841.25  | Pastelería Gourmet   |
| 155 | Brownie            | BRO0029       | Brownie Extra grande sabor Caramelo relleno de Crema de chocolate y cubierto de Crema             |              5342 |          3523.91  | Dolce Gusto S.R.L.   |
| 156 | Rosca              | ROS0016       | Rosca Individual sabor Canela relleno de Crema de chocolate y cubierto de Almendras               |              6396 |          3558.08  | Panadería Fina & Co. |
| 157 | Dona               | DON0021       | Dona Mini sabor Café relleno de Crema de frutilla y cubierto de Almendras                         |              6732 |          3406.7   | Pastelería Gourmet   |
| 158 | Brownie            | BRO0030       | Brownie Grande sabor Frutilla relleno de Crema de chocolate y cubierto de Merengue                |              2944 |          1707.3   | Delicias Artesanas   |
| 159 | Galleta            | GAL0019       | Galleta Individual sabor Nuez relleno de Crema de chocolate y cubierto de Merengue                |              3975 |          2931.01  | Panadería Fina & Co. |
| 160 | Brownie            | BRO0031       | Brownie Individual sabor Nuez relleno de Crema de frutilla y cubierto de Mani                     |              2456 |          1442.17  | Repostería Selecta   |
| 161 | Queque             | QUE0017       | Queque Extra grande sabor Frutilla relleno de Manjar y cubierto de Nuez                           |              4006 |          2212.33  | Delicias Artesanas   |
| 162 | Galleta            | GAL0020       | Galleta Mini sabor Maracuya relleno de Crema de frutilla y cubierto de Mani                       |              3313 |          1746.61  | Pan y Más            |
| 163 | Cupcake            | CUP0015       | Cupcake Extra grande sabor Plátano relleno de Crema pastelera y cubierto de Nuez                  |              4893 |          3220.6   | Delicias Artesanas   |
| 164 | Galleta            | GAL0021       | Galleta Extra grande sabor Maracuya relleno de Crema pastelera y cubierto de Nuez                 |              8287 |          5403.51  | Repostería Selecta   |
| 165 | Torta              | TOR0019       | Torta Mini sabor Frutilla relleno de Manjar y cubierto de Merengue                                |              7003 |          4232.17  | Panadería Fina & Co. |
| 166 | Queque             | QUE0018       | Queque Mini sabor Nuez relleno de Crema pastelera y cubierto de Chocolate negro                   |              5957 |          3201.85  | Panadería Fina & Co. |
| 167 | Dona               | DON0022       | Dona Individual sabor Pistacho relleno de Crema de chocolate y cubierto de Nuez                   |              7324 |          4220.28  | Pan y Más            |
| 168 | Queque             | QUE0019       | Queque Grande sabor Nuez relleno de Crema pastelera y cubierto de Merengue                        |              2858 |          2271.21  | Pan y Más            |
| 169 | Brownie            | BRO0032       | Brownie Individual sabor Café relleno de Manjar y cubierto de Merengue                            |              3543 |          1784.7   | Dolce Gusto S.R.L.   |
| 170 | Cupcake            | CUP0016       | Cupcake Grande sabor Caramelo relleno de Dulce de leche y cubierto de Mani                        |              6240 |          4200.95  | Delicias Artesanas   |
| 171 | Magdalena          | MAG0016       | Magdalena Mini sabor Nuez relleno de Manjar y cubierto de Merengue                                |              5605 |          4115.97  | Delicias Artesanas   |
| 172 | Dona               | DON0023       | Dona Individual sabor Nuez relleno de Crema de chocolate y cubierto de Chocolate negro            |              6278 |          4833.03  | Panadería Fina & Co. |
| 173 | Galleta            | GAL0022       | Galleta Mini sabor Mango relleno de Dulce de leche y cubierto de Mani                             |              5928 |          4252.75  | Delicias Artesanas   |
| 174 | Cupcake            | CUP0017       | Cupcake Individual sabor Plátano relleno de Crema de chocolate y cubierto de Merengue             |              5624 |          3929.38  | Pan y Más            |
| 175 | Cupcake            | CUP0018       | Cupcake Extra grande sabor Vainilla relleno de Crema pastelera y cubierto de Merengue             |              7976 |          4618.93  | Repostería Selecta   |
| 176 | Queque             | QUE0020       | Queque Individual sabor Mango relleno de Dulce de leche y cubierto de Nuez                        |              1199 |           662.102 | Dolce Gusto S.R.L.   |
| 177 | Dona               | DON0024       | Dona Mini sabor Mango relleno de Dulce de leche y cubierto de Almendras                           |              1671 |          1090.02  | Delicias Artesanas   |
| 178 | Magdalena          | MAG0017       | Magdalena Grande sabor Limon relleno de Crema de chocolate y cubierto de Chocolate negro          |              1749 |          1243.08  | Panadería Fina & Co. |
| 179 | Magdalena          | MAG0018       | Magdalena Individual sabor Canela relleno de Crema de frutilla y cubierto de Almendras            |              3049 |          2299.6   | Delicias Artesanas   |
| 180 | Magdalena          | MAG0019       | Magdalena Grande sabor Caramelo relleno de Crema de frutilla y cubierto de Nuez                   |              2456 |          1960.75  | Delicias Artesanas   |
| 181 | Torta              | TOR0020       | Torta Extra grande sabor Plátano relleno de Manjar y cubierto de Merengue                         |              6377 |          4296.28  | Repostería Selecta   |
| 182 | Galleta            | GAL0023       | Galleta Mini sabor Pistacho relleno de Crema de chocolate y cubierto de Chocolate blanco          |              6286 |          4309.37  | Pan y Más            |
| 183 | Queque             | QUE0021       | Queque Mini sabor Pistacho relleno de Crema de frutilla y cubierto de Almendras                   |              2296 |          1414.77  | Panadería Fina & Co. |
| 184 | Cupcake            | CUP0019       | Cupcake Individual sabor Mora relleno de Manjar y cubierto de Chocolate blanco                    |              6984 |          5526.65  | Delicias Artesanas   |
| 185 | Dona               | DON0025       | Dona Grande sabor Limon relleno de Crema de frutilla y cubierto de Chocolate blanco               |              4248 |          2805.75  | Pastelería Gourmet   |
| 186 | Rosca              | ROS0017       | Rosca Mini sabor Caramelo relleno de Dulce de leche y cubierto de Almendras                       |              3051 |          1642.23  | Pan y Más            |
| 187 | Torta              | TOR0021       | Torta Extra grande sabor Plátano relleno de Crema de frutilla y cubierto de Crema                 |              9489 |          6106.49  | Pan y Más            |
| 188 | Galleta            | GAL0024       | Galleta Mini sabor Pistacho relleno de Crema pastelera y cubierto de Almendras                    |              5052 |          3777.83  | Pan y Más            |
| 189 | Cupcake            | CUP0020       | Cupcake Individual sabor Chocolate relleno de Crema pastelera y cubierto de Merengue              |              1816 |           979.425 | Repostería Selecta   |
| 190 | Tarta              | TAR0012       | Tarta Grande sabor Pistacho relleno de Crema de chocolate y cubierto de Chocolate negro           |              4050 |          2266.61  | Pan y Más            |
| 191 | Rosca              | ROS0018       | Rosca Mini sabor Plátano relleno de Crema de frutilla y cubierto de Almendras                     |              4312 |          2305.64  | Panadería Fina & Co. |
| 192 | Galleta            | GAL0025       | Galleta Extra grande sabor Pistacho relleno de Dulce de leche y cubierto de Mani                  |              9447 |          6230.76  | Dolce Gusto S.R.L.   |
| 193 | Magdalena          | MAG0020       | Magdalena Mini sabor Chocolate relleno de Crema de chocolate y cubierto de Mani                   |              4251 |          2761.34  | Dolce Gusto S.R.L.   |
| 194 | Torta              | TOR0022       | Torta Grande sabor Canela relleno de Crema de chocolate y cubierto de Chocolate negro             |              3040 |          2364.48  | Pan y Más            |
| 195 | Queque             | QUE0022       | Queque Extra grande sabor Plátano relleno de Manjar y cubierto de Merengue                        |              9821 |          7774.49  | Delicias Artesanas   |
| 196 | Rosca              | ROS0019       | Rosca Extra grande sabor Caramelo relleno de Crema de frutilla y cubierto de Crema                |              8505 |          5550.5   | Pan y Más            |
| 197 | Rosca              | ROS0020       | Rosca Extra grande sabor Coco relleno de Manjar y cubierto de Chocolate negro                     |              9831 |          7766.46  | Panadería Fina & Co. |
| 198 | Queque             | QUE0023       | Queque Mini sabor Caramelo relleno de Crema de chocolate y cubierto de Chocolate negro            |              7430 |          4225.56  | Delicias Artesanas   |
| 199 | Galleta            | GAL0026       | Galleta Extra grande sabor Plátano relleno de Dulce de leche y cubierto de Mani                   |              7518 |          5195.92  | Repostería Selecta   |
| 200 | Dona               | DON0026       | Dona Mini sabor Coco relleno de Crema de chocolate y cubierto de Crema                            |              4252 |          2452.46  | Pastelería Gourmet   |
| 201 | Brownie            | BRO0033       | Brownie Mini sabor Vainilla relleno de Crema de frutilla y cubierto de Chocolate blanco           |              2215 |          1709.61  | Delicias Artesanas   |
| 202 | Dona               | DON0027       | Dona Extra grande sabor Chocolate relleno de Crema pastelera y cubierto de Nuez                   |              3021 |          2392.43  | Repostería Selecta   |
| 203 | Cupcake            | CUP0021       | Cupcake Extra grande sabor Frutilla relleno de Crema pastelera y cubierto de Almendras            |              7348 |          5689.2   | Delicias Artesanas   |
| 204 | Torta              | TOR0023       | Torta Grande sabor Maracuya relleno de Crema de frutilla y cubierto de Chocolate negro            |              2683 |          1706.83  | Pastelería Gourmet   |
| 205 | Dona               | DON0028       | Dona Individual sabor Frutilla relleno de Manjar y cubierto de Almendras                          |              9529 |          5842.48  | Panadería Fina & Co. |
| 206 | Dona               | DON0029       | Dona Grande sabor Nuez relleno de Crema de chocolate y cubierto de Nuez                           |              1910 |          1061.77  | Pan y Más            |
| 207 | Magdalena          | MAG0021       | Magdalena Mini sabor Mora relleno de Dulce de leche y cubierto de Almendras                       |              2271 |          1270.25  | Dolce Gusto S.R.L.   |
| 208 | Galleta            | GAL0027       | Galleta Grande sabor Maracuya relleno de Manjar y cubierto de Almendras                           |              8574 |          5906.63  | Pan y Más            |
| 209 | Torta              | TOR0024       | Torta Grande sabor Coco relleno de Crema de chocolate y cubierto de Merengue                      |              5205 |          2912.79  | Dolce Gusto S.R.L.   |
| 210 | Dona               | DON0030       | Dona Extra grande sabor Café relleno de Dulce de leche y cubierto de Chocolate negro              |              8937 |          5529.19  | Pan y Más            |
| 211 | Queque             | QUE0024       | Queque Extra grande sabor Canela relleno de Dulce de leche y cubierto de Almendras                |              4493 |          2478.41  | Pan y Más            |
| 212 | Tarta              | TAR0013       | Tarta Extra grande sabor Coco relleno de Manjar y cubierto de Crema                               |              5069 |          3647.59  | Pastelería Gourmet   |
| 213 | Queque             | QUE0025       | Queque Extra grande sabor Vainilla relleno de Dulce de leche y cubierto de Almendras              |              6117 |          4348.9   | Repostería Selecta   |
| 214 | Dona               | DON0031       | Dona Extra grande sabor Caramelo relleno de Crema de frutilla y cubierto de Almendras             |              2618 |          1757     | Pastelería Gourmet   |
| 215 | Magdalena          | MAG0022       | Magdalena Individual sabor Chocolate relleno de Manjar y cubierto de Merengue                     |              4764 |          3732.64  | Repostería Selecta   |
| 216 | Queque             | QUE0026       | Queque Mini sabor Maracuya relleno de Crema pastelera y cubierto de Chocolate negro               |              8114 |          4082.29  | Delicias Artesanas   |
| 217 | Dona               | DON0032       | Dona Grande sabor Plátano relleno de Manjar y cubierto de Chocolate blanco                        |              6191 |          4809.8   | Delicias Artesanas   |
| 218 | Galleta            | GAL0028       | Galleta Extra grande sabor Frambuesa relleno de Dulce de leche y cubierto de Chocolate negro      |              2215 |          1713.65  | Repostería Selecta   |
| 219 | Brownie            | BRO0034       | Brownie Individual sabor Mora relleno de Crema de frutilla y cubierto de Nuez                     |              2079 |          1162.81  | Dolce Gusto S.R.L.   |
| 220 | Dona               | DON0033       | Dona Grande sabor Coco relleno de Dulce de leche y cubierto de Crema                              |              7997 |          6037.73  | Panadería Fina & Co. |
| 221 | Brownie            | BRO0035       | Brownie Individual sabor Canela relleno de Crema de chocolate y cubierto de Nuez                  |              3001 |          1669.2   | Dolce Gusto S.R.L.   |
| 222 | Dona               | DON0034       | Dona Extra grande sabor Mango relleno de Crema de chocolate y cubierto de Chocolate blanco        |              3655 |          2640.43  | Pan y Más            |
| 223 | Galleta            | GAL0029       | Galleta Individual sabor Limon relleno de Crema de frutilla y cubierto de Crema                   |              9573 |          5638.29  | Dolce Gusto S.R.L.   |
| 224 | Galleta            | GAL0030       | Galleta Mini sabor Nuez relleno de Dulce de leche y cubierto de Chocolate negro                   |              5167 |          2812.19  | Delicias Artesanas   |
| 225 | Queque             | QUE0027       | Queque Extra grande sabor Coco relleno de Crema de chocolate y cubierto de Chocolate negro        |              8666 |          5100.53  | Delicias Artesanas   |
| 226 | Galleta            | GAL0031       | Galleta Grande sabor Frutilla relleno de Crema de chocolate y cubierto de Crema                   |              8572 |          5434.63  | Dolce Gusto S.R.L.   |
| 227 | Queque             | QUE0028       | Queque Individual sabor Mora relleno de Manjar y cubierto de Chocolate negro                      |              2421 |          1820.3   | Pastelería Gourmet   |
| 228 | Brownie            | BRO0036       | Brownie Grande sabor Pera relleno de Manjar y cubierto de Almendras                               |              5946 |          3168.55  | Dolce Gusto S.R.L.   |
| 229 | Rosca              | ROS0021       | Rosca Grande sabor Vainilla relleno de Dulce de leche y cubierto de Nuez                          |              2293 |          1579.92  | Delicias Artesanas   |
| 230 | Galleta            | GAL0032       | Galleta Mini sabor Canela relleno de Crema de chocolate y cubierto de Chocolate negro             |              1467 |           972.452 | Repostería Selecta   |
| 231 | Torta              | TOR0025       | Torta Extra grande sabor Plátano relleno de Crema de chocolate y cubierto de Merengue             |              3921 |          1976.24  | Delicias Artesanas   |
| 232 | Galleta            | GAL0033       | Galleta Individual sabor Caramelo relleno de Crema pastelera y cubierto de Merengue               |              1440 |          1032.72  | Pan y Más            |
| 233 | Torta              | TOR0026       | Torta Individual sabor Mora relleno de Crema pastelera y cubierto de Merengue                     |              5356 |          3731.31  | Panadería Fina & Co. |
| 234 | Queque             | QUE0029       | Queque Mini sabor Maracuya relleno de Manjar y cubierto de Merengue                               |              6678 |          4462.19  | Delicias Artesanas   |
| 235 | Galleta            | GAL0034       | Galleta Individual sabor Mora relleno de Crema de chocolate y cubierto de Almendras               |              7585 |          4402.9   | Panadería Fina & Co. |
| 236 | Cupcake            | CUP0022       | Cupcake Extra grande sabor Plátano relleno de Manjar y cubierto de Merengue                       |              7664 |          4386.34  | Repostería Selecta   |
| 237 | Torta              | TOR0027       | Torta Individual sabor Canela relleno de Crema de chocolate y cubierto de Nuez                    |              3922 |          2914.11  | Delicias Artesanas   |
| 238 | Cupcake            | CUP0023       | Cupcake Extra grande sabor Caramelo relleno de Crema de chocolate y cubierto de Crema             |              2757 |          1488.86  | Dolce Gusto S.R.L.   |
| 239 | Queque             | QUE0030       | Queque Individual sabor Limon relleno de Dulce de leche y cubierto de Mani                        |              4834 |          2418.85  | Pastelería Gourmet   |
| 240 | Torta              | TOR0028       | Torta Extra grande sabor Mango relleno de Crema de chocolate y cubierto de Mani                   |              3903 |          1951.59  | Delicias Artesanas   |
| 241 | Torta              | TOR0029       | Torta Individual sabor Frutilla relleno de Crema de chocolate y cubierto de Chocolate blanco      |              9053 |          6628.02  | Pan y Más            |
| 242 | Queque             | QUE0031       | Queque Extra grande sabor Canela relleno de Dulce de leche y cubierto de Chocolate negro          |              8144 |          5722.8   | Pan y Más            |
| 243 | Tarta              | TAR0014       | Tarta Individual sabor Café relleno de Dulce de leche y cubierto de Nuez                          |              4039 |          2245.73  | Panadería Fina & Co. |
| 244 | Galleta            | GAL0035       | Galleta Grande sabor Frambuesa relleno de Crema de chocolate y cubierto de Merengue               |              6513 |          4937.95  | Pan y Más            |
| 245 | Dona               | DON0035       | Dona Extra grande sabor Frutilla relleno de Dulce de leche y cubierto de Almendras                |              6966 |          4520.1   | Dolce Gusto S.R.L.   |
| 246 | Dona               | DON0036       | Dona Grande sabor Coco relleno de Crema de chocolate y cubierto de Merengue                       |              8003 |          5279.88  | Delicias Artesanas   |
| 247 | Tarta              | TAR0015       | Tarta Grande sabor Pera relleno de Crema de chocolate y cubierto de Crema                         |              1874 |          1034.42  | Dolce Gusto S.R.L.   |
| 248 | Galleta            | GAL0036       | Galleta Individual sabor Nuez relleno de Dulce de leche y cubierto de Chocolate blanco            |              9233 |          6727.52  | Delicias Artesanas   |
| 249 | Galleta            | GAL0037       | Galleta Mini sabor Maracuya relleno de Crema pastelera y cubierto de Crema                        |              4157 |          2945.36  | Pastelería Gourmet   |
| 250 | Torta              | TOR0030       | Torta Grande sabor Pistacho relleno de Crema de frutilla y cubierto de Crema                      |              6929 |          4026.38  | Repostería Selecta   |
| 251 | Rosca              | ROS0022       | Rosca Grande sabor Caramelo relleno de Dulce de leche y cubierto de Nuez                          |              2343 |          1732.62  | Dolce Gusto S.R.L.   |
| 252 | Magdalena          | MAG0023       | Magdalena Grande sabor Chocolate relleno de Manjar y cubierto de Chocolate blanco                 |              8120 |          5346     | Repostería Selecta   |
| 253 | Brownie            | BRO0037       | Brownie Extra grande sabor Canela relleno de Crema pastelera y cubierto de Mani                   |              6214 |          3209.46  | Pan y Más            |
| 254 | Torta              | TOR0031       | Torta Mini sabor Maracuya relleno de Crema de frutilla y cubierto de Chocolate blanco             |              9733 |          7592.92  | Panadería Fina & Co. |
| 255 | Cupcake            | CUP0024       | Cupcake Mini sabor Canela relleno de Crema de chocolate y cubierto de Merengue                    |              7282 |          4426.1   | Pan y Más            |
| 256 | Magdalena          | MAG0024       | Magdalena Grande sabor Canela relleno de Crema de chocolate y cubierto de Crema                   |              5250 |          3558.45  | Pan y Más            |
| 257 | Queque             | QUE0032       | Queque Individual sabor Frutilla relleno de Dulce de leche y cubierto de Mani                     |              6038 |          4606.28  | Delicias Artesanas   |
| 258 | Tarta              | TAR0016       | Tarta Mini sabor Vainilla relleno de Crema de chocolate y cubierto de Mani                        |              9250 |          7020.02  | Delicias Artesanas   |
| 259 | Cupcake            | CUP0025       | Cupcake Mini sabor Maracuya relleno de Manjar y cubierto de Crema                                 |              4685 |          2402.03  | Pastelería Gourmet   |
| 260 | Rosca              | ROS0023       | Rosca Grande sabor Nuez relleno de Crema pastelera y cubierto de Merengue                         |              6569 |          4361.49  | Repostería Selecta   |
| 261 | Rosca              | ROS0024       | Rosca Extra grande sabor Vainilla relleno de Crema de chocolate y cubierto de Chocolate blanco    |              2907 |          2140.96  | Pastelería Gourmet   |
| 262 | Queque             | QUE0033       | Queque Mini sabor Café relleno de Crema de chocolate y cubierto de Merengue                       |              9297 |          6631.29  | Delicias Artesanas   |
| 263 | Queque             | QUE0034       | Queque Extra grande sabor Nuez relleno de Crema de chocolate y cubierto de Nuez                   |              9341 |          4692.97  | Pan y Más            |
| 264 | Rosca              | ROS0025       | Rosca Extra grande sabor Maracuya relleno de Crema de frutilla y cubierto de Crema                |              7257 |          4754.46  | Pastelería Gourmet   |
| 265 | Magdalena          | MAG0025       | Magdalena Grande sabor Coco relleno de Crema pastelera y cubierto de Chocolate negro              |              5450 |          3740.56  | Pastelería Gourmet   |
| 266 | Magdalena          | MAG0026       | Magdalena Grande sabor Caramelo relleno de Manjar y cubierto de Mani                              |              6341 |          5030.2   | Repostería Selecta   |
| 267 | Tarta              | TAR0017       | Tarta Grande sabor Plátano relleno de Crema pastelera y cubierto de Almendras                     |              7354 |          5237.07  | Delicias Artesanas   |
| 268 | Dona               | DON0037       | Dona Grande sabor Plátano relleno de Crema de frutilla y cubierto de Crema                        |              7245 |          5286.64  | Panadería Fina & Co. |
| 269 | Rosca              | ROS0026       | Rosca Individual sabor Caramelo relleno de Crema pastelera y cubierto de Chocolate negro          |              5597 |          3231.89  | Panadería Fina & Co. |
| 270 | Dona               | DON0038       | Dona Mini sabor Chocolate relleno de Crema de chocolate y cubierto de Almendras                   |              5562 |          4060.73  | Pastelería Gourmet   |
| 271 | Cupcake            | CUP0026       | Cupcake Grande sabor Mango relleno de Crema de chocolate y cubierto de Nuez                       |              7417 |          3878.12  | Pastelería Gourmet   |
| 272 | Galleta            | GAL0038       | Galleta Grande sabor Maracuya relleno de Dulce de leche y cubierto de Mani                        |              4202 |          2534.96  | Panadería Fina & Co. |
| 273 | Torta              | TOR0032       | Torta Extra grande sabor Plátano relleno de Crema de frutilla y cubierto de Chocolate negro       |              3531 |          2341.48  | Pastelería Gourmet   |
| 274 | Magdalena          | MAG0027       | Magdalena Grande sabor Plátano relleno de Crema pastelera y cubierto de Chocolate negro           |              6081 |          3655.62  | Delicias Artesanas   |
| 275 | Tarta              | TAR0018       | Tarta Individual sabor Pera relleno de Dulce de leche y cubierto de Mani                          |              4089 |          2670.92  | Pan y Más            |
| 276 | Tarta              | TAR0019       | Tarta Individual sabor Vainilla relleno de Crema pastelera y cubierto de Mani                     |              1554 |           881.102 | Dolce Gusto S.R.L.   |
| 277 | Dona               | DON0039       | Dona Extra grande sabor Canela relleno de Dulce de leche y cubierto de Crema                      |              7833 |          5239.81  | Pastelería Gourmet   |
| 278 | Queque             | QUE0035       | Queque Grande sabor Maracuya relleno de Manjar y cubierto de Mani                                 |              7300 |          5081.72  | Delicias Artesanas   |
| 279 | Tarta              | TAR0020       | Tarta Mini sabor Limon relleno de Manjar y cubierto de Crema                                      |              6019 |          3632.79  | Pan y Más            |
| 280 | Rosca              | ROS0027       | Rosca Individual sabor Maracuya relleno de Crema de frutilla y cubierto de Almendras              |              7845 |          5685.7   | Dolce Gusto S.R.L.   |
| 281 | Queque             | QUE0036       | Queque Mini sabor Chocolate relleno de Crema de frutilla y cubierto de Chocolate negro            |              2118 |          1152.4   | Pan y Más            |
| 282 | Tarta              | TAR0021       | Tarta Grande sabor Frambuesa relleno de Dulce de leche y cubierto de Mani                         |              7856 |          4507.72  | Delicias Artesanas   |
| 283 | Torta              | TOR0033       | Torta Individual sabor Mora relleno de Crema de chocolate y cubierto de Nuez                      |              4762 |          2691.04  | Pan y Más            |
| 284 | Torta              | TOR0034       | Torta Mini sabor Pistacho relleno de Crema de chocolate y cubierto de Almendras                   |              4284 |          3257.5   | Dolce Gusto S.R.L.   |
| 285 | Tarta              | TAR0022       | Tarta Grande sabor Mango relleno de Crema pastelera y cubierto de Chocolate blanco                |              1181 |           810.958 | Delicias Artesanas   |
| 286 | Brownie            | BRO0038       | Brownie Grande sabor Limon relleno de Crema de frutilla y cubierto de Mani                        |              9091 |          5118.14  | Repostería Selecta   |
| 287 | Queque             | QUE0037       | Queque Grande sabor Mango relleno de Manjar y cubierto de Crema                                   |              2898 |          1678.55  | Pastelería Gourmet   |
| 288 | Cupcake            | CUP0027       | Cupcake Extra grande sabor Mora relleno de Dulce de leche y cubierto de Chocolate blanco          |              3195 |          2478.03  | Delicias Artesanas   |
| 289 | Galleta            | GAL0039       | Galleta Extra grande sabor Frambuesa relleno de Crema de frutilla y cubierto de Crema             |              3409 |          2189.02  | Pastelería Gourmet   |
| 290 | Galleta            | GAL0040       | Galleta Individual sabor Mora relleno de Dulce de leche y cubierto de Nuez                        |              3296 |          1880.94  | Dolce Gusto S.R.L.   |
| 291 | Tarta              | TAR0023       | Tarta Grande sabor Frutilla relleno de Crema de chocolate y cubierto de Merengue                  |              2553 |          1298.03  | Pastelería Gourmet   |
| 292 | Brownie            | BRO0039       | Brownie Extra grande sabor Mango relleno de Crema de frutilla y cubierto de Crema                 |              5797 |          3909.4   | Dolce Gusto S.R.L.   |
| 293 | Torta              | TOR0035       | Torta Grande sabor Frutilla relleno de Manjar y cubierto de Mani                                  |              9181 |          5897.85  | Panadería Fina & Co. |
| 294 | Tarta              | TAR0024       | Tarta Mini sabor Mora relleno de Manjar y cubierto de Crema                                       |              6036 |          3451.09  | Panadería Fina & Co. |
| 295 | Dona               | DON0040       | Dona Extra grande sabor Chocolate relleno de Crema de frutilla y cubierto de Chocolate negro      |              8930 |          5952.38  | Dolce Gusto S.R.L.   |
| 296 | Dona               | DON0041       | Dona Grande sabor Frutilla relleno de Dulce de leche y cubierto de Almendras                      |              2687 |          1785.71  | Panadería Fina & Co. |
| 297 | Cupcake            | CUP0028       | Cupcake Mini sabor Coco relleno de Crema de chocolate y cubierto de Merengue                      |              8226 |          4581.51  | Delicias Artesanas   |
| 298 | Magdalena          | MAG0028       | Magdalena Grande sabor Limon relleno de Dulce de leche y cubierto de Mani                         |              6695 |          5353.54  | Delicias Artesanas   |
| 299 | Rosca              | ROS0028       | Rosca Individual sabor Coco relleno de Crema pastelera y cubierto de Nuez                         |              5593 |          3466.76  | Repostería Selecta   |
| 300 | Queque             | QUE0038       | Queque Grande sabor Nuez relleno de Crema pastelera y cubierto de Chocolate blanco                |              2319 |          1626.02  | Delicias Artesanas   |
| 301 | Cupcake            | CUP0029       | Cupcake Extra grande sabor Maracuya relleno de Dulce de leche y cubierto de Chocolate blanco      |              1861 |          1080.53  | Delicias Artesanas   |
| 302 | Queque             | QUE0039       | Queque Extra grande sabor Pera relleno de Crema de frutilla y cubierto de Crema                   |              3978 |          2241.37  | Repostería Selecta   |
| 303 | Tarta              | TAR0025       | Tarta Mini sabor Mora relleno de Crema de frutilla y cubierto de Almendras                        |              2993 |          2265.02  | Dolce Gusto S.R.L.   |
| 304 | Queque             | QUE0040       | Queque Grande sabor Chocolate relleno de Crema de chocolate y cubierto de Crema                   |              4081 |          2055.74  | Repostería Selecta   |
| 305 | Brownie            | BRO0040       | Brownie Mini sabor Plátano relleno de Crema de chocolate y cubierto de Mani                       |              9031 |          6922.75  | Delicias Artesanas   |
| 306 | Tarta              | TAR0026       | Tarta Grande sabor Caramelo relleno de Manjar y cubierto de Chocolate blanco                      |              1249 |           883.17  | Repostería Selecta   |
| 307 | Rosca              | ROS0029       | Rosca Mini sabor Limon relleno de Crema de chocolate y cubierto de Chocolate blanco               |              9933 |          5494.52  | Repostería Selecta   |
| 308 | Magdalena          | MAG0029       | Magdalena Individual sabor Pera relleno de Manjar y cubierto de Chocolate blanco                  |              3269 |          2577.84  | Pan y Más            |
| 309 | Queque             | QUE0041       | Queque Extra grande sabor Vainilla relleno de Dulce de leche y cubierto de Chocolate blanco       |              8426 |          5092.24  | Panadería Fina & Co. |
| 310 | Cupcake            | CUP0030       | Cupcake Extra grande sabor Pera relleno de Dulce de leche y cubierto de Nuez                      |              2249 |          1753.16  | Dolce Gusto S.R.L.   |
| 311 | Tarta              | TAR0027       | Tarta Mini sabor Nuez relleno de Crema pastelera y cubierto de Almendras                          |              6864 |          4753.02  | Dolce Gusto S.R.L.   |
| 312 | Tarta              | TAR0028       | Tarta Extra grande sabor Maracuya relleno de Manjar y cubierto de Chocolate negro                 |              9328 |          7428.83  | Repostería Selecta   |
| 313 | Tarta              | TAR0029       | Tarta Grande sabor Caramelo relleno de Manjar y cubierto de Mani                                  |              5764 |          3164.76  | Delicias Artesanas   |
| 314 | Tarta              | TAR0030       | Tarta Extra grande sabor Pera relleno de Crema pastelera y cubierto de Chocolate negro            |              5569 |          3106.42  | Dolce Gusto S.R.L.   |
| 315 | Brownie            | BRO0041       | Brownie Mini sabor Chocolate relleno de Crema de chocolate y cubierto de Mani                     |              3162 |          1882.9   | Pastelería Gourmet   |
| 316 | Torta              | TOR0036       | Torta Grande sabor Pera relleno de Crema de chocolate y cubierto de Chocolate negro               |              8685 |          5589.09  | Panadería Fina & Co. |
| 317 | Dona               | DON0042       | Dona Mini sabor Frambuesa relleno de Crema de chocolate y cubierto de Chocolate negro             |              1824 |          1362.92  | Pan y Más            |
| 318 | Galleta            | GAL0041       | Galleta Individual sabor Limon relleno de Crema de chocolate y cubierto de Mani                   |              6878 |          5084.66  | Repostería Selecta   |
| 319 | Brownie            | BRO0042       | Brownie Extra grande sabor Café relleno de Manjar y cubierto de Crema                             |              1242 |           835.275 | Delicias Artesanas   |
| 320 | Galleta            | GAL0042       | Galleta Extra grande sabor Nuez relleno de Crema de chocolate y cubierto de Chocolate blanco      |              6612 |          4452.92  | Pan y Más            |
| 321 | Queque             | QUE0042       | Queque Extra grande sabor Mora relleno de Crema de chocolate y cubierto de Chocolate negro        |              2251 |          1732.46  | Panadería Fina & Co. |
| 322 | Galleta            | GAL0043       | Galleta Individual sabor Frambuesa relleno de Crema pastelera y cubierto de Chocolate negro       |              7844 |          5447.29  | Dolce Gusto S.R.L.   |
| 323 | Queque             | QUE0043       | Queque Extra grande sabor Limon relleno de Crema pastelera y cubierto de Merengue                 |              5253 |          2720.83  | Pan y Más            |
| 324 | Rosca              | ROS0030       | Rosca Grande sabor Maracuya relleno de Crema de frutilla y cubierto de Nuez                       |              6541 |          5147.69  | Pastelería Gourmet   |
| 325 | Dona               | DON0043       | Dona Extra grande sabor Café relleno de Crema de chocolate y cubierto de Almendras                |              9352 |          6399.9   | Repostería Selecta   |
| 326 | Cupcake            | CUP0031       | Cupcake Extra grande sabor Caramelo relleno de Crema de chocolate y cubierto de Chocolate blanco  |              8270 |          5229.31  | Pan y Más            |
| 327 | Rosca              | ROS0031       | Rosca Grande sabor Canela relleno de Crema de frutilla y cubierto de Chocolate negro              |              5998 |          3778.53  | Pan y Más            |
| 328 | Magdalena          | MAG0030       | Magdalena Grande sabor Caramelo relleno de Crema pastelera y cubierto de Almendras                |              4255 |          3026.01  | Panadería Fina & Co. |
| 329 | Queque             | QUE0044       | Queque Mini sabor Mango relleno de Manjar y cubierto de Almendras                                 |              1201 |           877.549 | Pan y Más            |
| 330 | Tarta              | TAR0031       | Tarta Individual sabor Mango relleno de Manjar y cubierto de Chocolate blanco                     |              3216 |          2235.87  | Repostería Selecta   |
| 331 | Dona               | DON0044       | Dona Individual sabor Pistacho relleno de Crema pastelera y cubierto de Nuez                      |              7953 |          4984.66  | Delicias Artesanas   |
| 332 | Galleta            | GAL0044       | Galleta Mini sabor Limon relleno de Crema de frutilla y cubierto de Almendras                     |              2797 |          1448.28  | Panadería Fina & Co. |
| 333 | Magdalena          | MAG0031       | Magdalena Grande sabor Limon relleno de Crema de frutilla y cubierto de Almendras                 |              3905 |          2613.95  | Dolce Gusto S.R.L.   |
| 334 | Magdalena          | MAG0032       | Magdalena Extra grande sabor Pistacho relleno de Crema de frutilla y cubierto de Crema            |              3908 |          2872.86  | Repostería Selecta   |
| 335 | Torta              | TOR0037       | Torta Mini sabor Chocolate relleno de Crema pastelera y cubierto de Chocolate negro               |              2062 |          1112.22  | Pastelería Gourmet   |
| 336 | Tarta              | TAR0032       | Tarta Grande sabor Mora relleno de Crema de frutilla y cubierto de Crema                          |              8381 |          5849.5   | Delicias Artesanas   |
| 337 | Magdalena          | MAG0033       | Magdalena Grande sabor Chocolate relleno de Crema pastelera y cubierto de Chocolate negro         |              4784 |          2625.81  | Dolce Gusto S.R.L.   |
| 338 | Brownie            | BRO0043       | Brownie Extra grande sabor Chocolate relleno de Manjar y cubierto de Nuez                         |              4563 |          3105     | Dolce Gusto S.R.L.   |
| 339 | Dona               | DON0045       | Dona Mini sabor Pera relleno de Manjar y cubierto de Almendras                                    |              7781 |          4724.2   | Repostería Selecta   |
| 340 | Queque             | QUE0045       | Queque Individual sabor Limon relleno de Manjar y cubierto de Mani                                |              6879 |          4707.79  | Dolce Gusto S.R.L.   |
| 341 | Rosca              | ROS0032       | Rosca Individual sabor Nuez relleno de Crema pastelera y cubierto de Mani                         |              5035 |          3240.37  | Delicias Artesanas   |
| 342 | Galleta            | GAL0045       | Galleta Mini sabor Pistacho relleno de Crema de chocolate y cubierto de Mani                      |              1928 |          1266.89  | Delicias Artesanas   |
| 343 | Queque             | QUE0046       | Queque Extra grande sabor Frambuesa relleno de Crema de frutilla y cubierto de Chocolate blanco   |              1878 |          1231.35  | Delicias Artesanas   |
| 344 | Queque             | QUE0047       | Queque Extra grande sabor Nuez relleno de Crema de chocolate y cubierto de Mani                   |              8193 |          5043.97  | Panadería Fina & Co. |
| 345 | Rosca              | ROS0033       | Rosca Extra grande sabor Frambuesa relleno de Crema pastelera y cubierto de Chocolate negro       |              8562 |          6694.81  | Repostería Selecta   |
| 346 | Magdalena          | MAG0034       | Magdalena Extra grande sabor Pistacho relleno de Crema pastelera y cubierto de Nuez               |              8382 |          4627.97  | Pastelería Gourmet   |
| 347 | Dona               | DON0046       | Dona Individual sabor Maracuya relleno de Dulce de leche y cubierto de Chocolate negro            |              2807 |          1921.74  | Delicias Artesanas   |
| 348 | Tarta              | TAR0033       | Tarta Individual sabor Nuez relleno de Manjar y cubierto de Chocolate negro                       |              9966 |          5542.95  | Delicias Artesanas   |
| 349 | Dona               | DON0047       | Dona Extra grande sabor Maracuya relleno de Crema de chocolate y cubierto de Nuez                 |              4085 |          2155.58  | Repostería Selecta   |
| 350 | Rosca              | ROS0034       | Rosca Mini sabor Pera relleno de Dulce de leche y cubierto de Chocolate blanco                    |              9821 |          5385.17  | Pastelería Gourmet   |
| 351 | Cupcake            | CUP0032       | Cupcake Extra grande sabor Coco relleno de Manjar y cubierto de Mani                              |              1063 |           846.046 | Panadería Fina & Co. |
| 352 | Dona               | DON0048       | Dona Mini sabor Limon relleno de Crema de frutilla y cubierto de Chocolate negro                  |              9200 |          5825.7   | Repostería Selecta   |
| 353 | Cupcake            | CUP0033       | Cupcake Extra grande sabor Coco relleno de Crema de frutilla y cubierto de Mani                   |              6923 |          5508.06  | Dolce Gusto S.R.L.   |
| 354 | Torta              | TOR0038       | Torta Grande sabor Mango relleno de Crema de frutilla y cubierto de Almendras                     |              3108 |          2367.16  | Pan y Más            |
| 355 | Cupcake            | CUP0034       | Cupcake Extra grande sabor Vainilla relleno de Crema pastelera y cubierto de Merengue             |              3938 |          3038.66  | Pan y Más            |
| 356 | Cupcake            | CUP0035       | Cupcake Extra grande sabor Nuez relleno de Manjar y cubierto de Almendras                         |              2010 |          1556.6   | Delicias Artesanas   |
| 357 | Queque             | QUE0048       | Queque Mini sabor Canela relleno de Manjar y cubierto de Chocolate blanco                         |              4692 |          2913.6   | Delicias Artesanas   |
| 358 | Rosca              | ROS0035       | Rosca Extra grande sabor Café relleno de Crema de chocolate y cubierto de Nuez                    |              6113 |          3517.82  | Panadería Fina & Co. |
| 359 | Dona               | DON0049       | Dona Mini sabor Coco relleno de Crema de chocolate y cubierto de Crema                            |              9615 |          5948.57  | Panadería Fina & Co. |
| 360 | Tarta              | TAR0034       | Tarta Grande sabor Pera relleno de Crema de chocolate y cubierto de Nuez                          |              3396 |          2426.76  | Delicias Artesanas   |
| 361 | Tarta              | TAR0035       | Tarta Grande sabor Maracuya relleno de Crema de frutilla y cubierto de Chocolate blanco           |              9407 |          7425.61  | Panadería Fina & Co. |
| 362 | Torta              | TOR0039       | Torta Extra grande sabor Maracuya relleno de Manjar y cubierto de Chocolate negro                 |              3337 |          1702.85  | Panadería Fina & Co. |
| 363 | Dona               | DON0050       | Dona Individual sabor Canela relleno de Dulce de leche y cubierto de Mani                         |              5164 |          3213.91  | Panadería Fina & Co. |
| 364 | Magdalena          | MAG0035       | Magdalena Extra grande sabor Frambuesa relleno de Dulce de leche y cubierto de Almendras          |              4905 |          2606.13  | Delicias Artesanas   |
| 365 | Cupcake            | CUP0036       | Cupcake Individual sabor Vainilla relleno de Manjar y cubierto de Crema                           |              1641 |          1198     | Delicias Artesanas   |
| 366 | Galleta            | GAL0046       | Galleta Mini sabor Canela relleno de Crema de frutilla y cubierto de Chocolate blanco             |              1761 |           908.728 | Panadería Fina & Co. |
| 367 | Rosca              | ROS0036       | Rosca Extra grande sabor Pera relleno de Crema de chocolate y cubierto de Chocolate blanco        |              8418 |          6253.97  | Panadería Fina & Co. |
| 368 | Tarta              | TAR0036       | Tarta Extra grande sabor Vainilla relleno de Dulce de leche y cubierto de Almendras               |              8473 |          6605.91  | Pan y Más            |
| 369 | Queque             | QUE0049       | Queque Mini sabor Pistacho relleno de Crema de frutilla y cubierto de Nuez                        |              1176 |           623.172 | Dolce Gusto S.R.L.   |
| 370 | Cupcake            | CUP0037       | Cupcake Individual sabor Nuez relleno de Crema de chocolate y cubierto de Chocolate negro         |              3742 |          2338.69  | Panadería Fina & Co. |
| 371 | Magdalena          | MAG0036       | Magdalena Extra grande sabor Caramelo relleno de Manjar y cubierto de Merengue                    |              8996 |          6645.94  | Repostería Selecta   |
| 372 | Galleta            | GAL0047       | Galleta Individual sabor Pera relleno de Crema de chocolate y cubierto de Almendras               |              5039 |          3696.31  | Pan y Más            |
| 373 | Torta              | TOR0040       | Torta Grande sabor Coco relleno de Manjar y cubierto de Chocolate blanco                          |              2776 |          1568.84  | Pastelería Gourmet   |
| 374 | Torta              | TOR0041       | Torta Extra grande sabor Caramelo relleno de Manjar y cubierto de Mani                            |              9814 |          5868.65  | Dolce Gusto S.R.L.   |
| 375 | Brownie            | BRO0044       | Brownie Grande sabor Caramelo relleno de Dulce de leche y cubierto de Crema                       |              5509 |          3909.59  | Panadería Fina & Co. |
| 376 | Torta              | TOR0042       | Torta Grande sabor Mango relleno de Crema de frutilla y cubierto de Chocolate blanco              |              3874 |          3005.31  | Repostería Selecta   |
| 377 | Queque             | QUE0050       | Queque Grande sabor Café relleno de Manjar y cubierto de Almendras                                |              8948 |          6027.17  | Delicias Artesanas   |
| 378 | Tarta              | TAR0037       | Tarta Individual sabor Pera relleno de Crema de chocolate y cubierto de Crema                     |              4093 |          2265.33  | Repostería Selecta   |
| 379 | Rosca              | ROS0037       | Rosca Individual sabor Limon relleno de Crema de frutilla y cubierto de Merengue                  |              1825 |          1293.83  | Pan y Más            |
| 380 | Tarta              | TAR0038       | Tarta Mini sabor Coco relleno de Crema pastelera y cubierto de Mani                               |              8548 |          6054.16  | Pastelería Gourmet   |
| 381 | Queque             | QUE0051       | Queque Grande sabor Mora relleno de Manjar y cubierto de Nuez                                     |              6773 |          3729.54  | Pastelería Gourmet   |
| 382 | Torta              | TOR0043       | Torta Extra grande sabor Café relleno de Dulce de leche y cubierto de Crema                       |              4771 |          2509.51  | Panadería Fina & Co. |
| 383 | Dona               | DON0051       | Dona Individual sabor Frutilla relleno de Dulce de leche y cubierto de Chocolate negro            |              2424 |          1342.84  | Pan y Más            |
| 384 | Tarta              | TAR0039       | Tarta Mini sabor Caramelo relleno de Crema de chocolate y cubierto de Mani                        |              2534 |          1476.95  | Pan y Más            |
| 385 | Queque             | QUE0052       | Queque Extra grande sabor Plátano relleno de Crema pastelera y cubierto de Merengue               |              1523 |          1070.33  | Dolce Gusto S.R.L.   |
| 386 | Cupcake            | CUP0038       | Cupcake Extra grande sabor Canela relleno de Crema de chocolate y cubierto de Merengue            |              8660 |          6725.8   | Delicias Artesanas   |
| 387 | Magdalena          | MAG0037       | Magdalena Individual sabor Frambuesa relleno de Dulce de leche y cubierto de Nuez                 |              9007 |          6376.16  | Pan y Más            |
| 388 | Tarta              | TAR0040       | Tarta Extra grande sabor Pistacho relleno de Manjar y cubierto de Nuez                            |              5105 |          2725.49  | Panadería Fina & Co. |
| 389 | Brownie            | BRO0045       | Brownie Extra grande sabor Caramelo relleno de Dulce de leche y cubierto de Mani                  |              3335 |          2080.82  | Pan y Más            |
| 390 | Rosca              | ROS0038       | Rosca Individual sabor Nuez relleno de Manjar y cubierto de Chocolate negro                       |              8276 |          6405.56  | Pastelería Gourmet   |
| 391 | Dona               | DON0052       | Dona Grande sabor Caramelo relleno de Crema de chocolate y cubierto de Merengue                   |              8377 |          5028.26  | Dolce Gusto S.R.L.   |
| 392 | Tarta              | TAR0041       | Tarta Grande sabor Limon relleno de Crema de frutilla y cubierto de Nuez                          |              4519 |          3401.51  | Repostería Selecta   |
| 393 | Brownie            | BRO0046       | Brownie Grande sabor Pera relleno de Crema de chocolate y cubierto de Nuez                        |              2376 |          1832.91  | Delicias Artesanas   |
| 394 | Cupcake            | CUP0039       | Cupcake Mini sabor Canela relleno de Crema pastelera y cubierto de Chocolate blanco               |              2432 |          1615.9   | Dolce Gusto S.R.L.   |
| 395 | Tarta              | TAR0042       | Tarta Mini sabor Pistacho relleno de Manjar y cubierto de Mani                                    |              7001 |          4763.46  | Pastelería Gourmet   |
| 396 | Cupcake            | CUP0040       | Cupcake Extra grande sabor Mora relleno de Dulce de leche y cubierto de Chocolate blanco          |              3730 |          2300.65  | Pan y Más            |
| 397 | Rosca              | ROS0039       | Rosca Individual sabor Plátano relleno de Crema pastelera y cubierto de Almendras                 |              7466 |          4520.17  | Repostería Selecta   |
| 398 | Tarta              | TAR0043       | Tarta Individual sabor Canela relleno de Manjar y cubierto de Nuez                                |              4679 |          3561.48  | Pan y Más            |
| 399 | Magdalena          | MAG0038       | Magdalena Grande sabor Frutilla relleno de Crema de chocolate y cubierto de Nuez                  |              6497 |          3773.24  | Repostería Selecta   |
| 400 | Rosca              | ROS0040       | Rosca Grande sabor Pistacho relleno de Crema de chocolate y cubierto de Mani                      |              3323 |          2379.79  | Pastelería Gourmet   |
| 401 | Brownie            | BRO0047       | Brownie Grande sabor Chocolate relleno de Manjar y cubierto de Nuez                               |              9976 |          6084.53  | Delicias Artesanas   |
| 402 | Dona               | DON0053       | Dona Extra grande sabor Coco relleno de Crema de chocolate y cubierto de Crema                    |              5476 |          3796.82  | Pastelería Gourmet   |
| 403 | Cupcake            | CUP0041       | Cupcake Mini sabor Frambuesa relleno de Crema pastelera y cubierto de Almendras                   |              3052 |          2354.14  | Pan y Más            |
| 404 | Dona               | DON0054       | Dona Grande sabor Chocolate relleno de Manjar y cubierto de Chocolate blanco                      |              3834 |          2408.49  | Dolce Gusto S.R.L.   |
| 405 | Tarta              | TAR0044       | Tarta Extra grande sabor Vainilla relleno de Crema de chocolate y cubierto de Merengue            |              2782 |          1773.97  | Panadería Fina & Co. |
| 406 | Cupcake            | CUP0042       | Cupcake Individual sabor Pistacho relleno de Crema de frutilla y cubierto de Merengue             |              8095 |          5811.52  | Panadería Fina & Co. |
| 407 | Torta              | TOR0044       | Torta Extra grande sabor Frambuesa relleno de Crema de frutilla y cubierto de Nuez                |              7760 |          5334.18  | Delicias Artesanas   |
| 408 | Cupcake            | CUP0043       | Cupcake Mini sabor Plátano relleno de Crema de chocolate y cubierto de Crema                      |              9949 |          5353.49  | Pastelería Gourmet   |
| 409 | Queque             | QUE0053       | Queque Grande sabor Frutilla relleno de Crema de frutilla y cubierto de Almendras                 |              3498 |          2777.87  | Dolce Gusto S.R.L.   |
| 410 | Cupcake            | CUP0044       | Cupcake Mini sabor Limon relleno de Crema de chocolate y cubierto de Chocolate negro              |              8547 |          6471.16  | Pastelería Gourmet   |
| 411 | Galleta            | GAL0048       | Galleta Mini sabor Limon relleno de Crema pastelera y cubierto de Crema                           |              2204 |          1574.4   | Pastelería Gourmet   |
| 412 | Rosca              | ROS0041       | Rosca Extra grande sabor Caramelo relleno de Crema de frutilla y cubierto de Chocolate negro      |              3819 |          2747.51  | Dolce Gusto S.R.L.   |
| 413 | Queque             | QUE0054       | Queque Extra grande sabor Pera relleno de Crema de chocolate y cubierto de Chocolate negro        |              5368 |          3220.15  | Dolce Gusto S.R.L.   |
| 414 | Cupcake            | CUP0045       | Cupcake Individual sabor Chocolate relleno de Crema de chocolate y cubierto de Crema              |              9492 |          7203.85  | Pastelería Gourmet   |
| 415 | Dona               | DON0055       | Dona Mini sabor Café relleno de Crema de frutilla y cubierto de Crema                             |              6264 |          4303.37  | Repostería Selecta   |
| 416 | Dona               | DON0056       | Dona Extra grande sabor Caramelo relleno de Manjar y cubierto de Nuez                             |              4493 |          2978.2   | Dolce Gusto S.R.L.   |
| 417 | Galleta            | GAL0049       | Galleta Extra grande sabor Caramelo relleno de Crema de frutilla y cubierto de Chocolate blanco   |              2765 |          1514.41  | Dolce Gusto S.R.L.   |
| 418 | Torta              | TOR0045       | Torta Individual sabor Pistacho relleno de Dulce de leche y cubierto de Merengue                  |              1929 |          1156.12  | Dolce Gusto S.R.L.   |
| 419 | Galleta            | GAL0050       | Galleta Individual sabor Vainilla relleno de Dulce de leche y cubierto de Merengue                |              3138 |          2378.43  | Pastelería Gourmet   |
| 420 | Tarta              | TAR0045       | Tarta Individual sabor Nuez relleno de Crema de chocolate y cubierto de Mani                      |              6214 |          4780.22  | Repostería Selecta   |
| 421 | Brownie            | BRO0048       | Brownie Mini sabor Coco relleno de Crema pastelera y cubierto de Chocolate blanco                 |              5294 |          3855.36  | Delicias Artesanas   |
| 422 | Tarta              | TAR0046       | Tarta Individual sabor Nuez relleno de Crema de frutilla y cubierto de Mani                       |              5959 |          4524.76  | Pan y Más            |
| 423 | Tarta              | TAR0047       | Tarta Individual sabor Maracuya relleno de Manjar y cubierto de Crema                             |              3546 |          1934.62  | Delicias Artesanas   |
| 424 | Queque             | QUE0055       | Queque Individual sabor Coco relleno de Crema de chocolate y cubierto de Chocolate negro          |              3422 |          2410.33  | Pan y Más            |
| 425 | Rosca              | ROS0042       | Rosca Grande sabor Maracuya relleno de Crema pastelera y cubierto de Chocolate blanco             |              9464 |          5458.7   | Repostería Selecta   |
| 426 | Queque             | QUE0056       | Queque Individual sabor Mora relleno de Crema de frutilla y cubierto de Chocolate negro           |              2428 |          1737.65  | Dolce Gusto S.R.L.   |
| 427 | Brownie            | BRO0049       | Brownie Extra grande sabor Canela relleno de Crema pastelera y cubierto de Nuez                   |              7576 |          5631.67  | Delicias Artesanas   |
| 428 | Torta              | TOR0046       | Torta Individual sabor Chocolate relleno de Dulce de leche y cubierto de Crema                    |              8155 |          5585.07  | Repostería Selecta   |
| 429 | Magdalena          | MAG0039       | Magdalena Extra grande sabor Nuez relleno de Manjar y cubierto de Chocolate negro                 |              3981 |          2947.14  | Panadería Fina & Co. |
| 430 | Queque             | QUE0057       | Queque Grande sabor Caramelo relleno de Crema pastelera y cubierto de Almendras                   |              8438 |          5790.01  | Pastelería Gourmet   |
| 431 | Galleta            | GAL0051       | Galleta Extra grande sabor Vainilla relleno de Crema pastelera y cubierto de Merengue             |              8007 |          4121.94  | Dolce Gusto S.R.L.   |
| 432 | Galleta            | GAL0052       | Galleta Extra grande sabor Nuez relleno de Crema de frutilla y cubierto de Chocolate blanco       |              1851 |          1094.49  | Dolce Gusto S.R.L.   |
| 433 | Torta              | TOR0047       | Torta Extra grande sabor Pera relleno de Crema pastelera y cubierto de Chocolate blanco           |              6523 |          3285.29  | Pan y Más            |
| 434 | Dona               | DON0057       | Dona Individual sabor Plátano relleno de Crema pastelera y cubierto de Chocolate negro            |              1391 |          1060.69  | Dolce Gusto S.R.L.   |
| 435 | Tarta              | TAR0048       | Tarta Grande sabor Caramelo relleno de Crema pastelera y cubierto de Crema                        |              3762 |          1929.11  | Dolce Gusto S.R.L.   |
| 436 | Magdalena          | MAG0040       | Magdalena Grande sabor Pistacho relleno de Dulce de leche y cubierto de Merengue                  |              4364 |          3179     | Dolce Gusto S.R.L.   |
| 437 | Galleta            | GAL0053       | Galleta Extra grande sabor Pistacho relleno de Dulce de leche y cubierto de Chocolate blanco      |              6776 |          3818.57  | Dolce Gusto S.R.L.   |
| 438 | Tarta              | TAR0049       | Tarta Individual sabor Vainilla relleno de Crema de chocolate y cubierto de Chocolate negro       |              7307 |          3761.67  | Pastelería Gourmet   |
| 439 | Rosca              | ROS0043       | Rosca Grande sabor Pistacho relleno de Crema pastelera y cubierto de Almendras                    |              8796 |          6480.09  | Delicias Artesanas   |
| 440 | Brownie            | BRO0050       | Brownie Individual sabor Caramelo relleno de Crema de chocolate y cubierto de Crema               |              4302 |          3354.82  | Pastelería Gourmet   |
| 441 | Magdalena          | MAG0041       | Magdalena Grande sabor Pistacho relleno de Crema pastelera y cubierto de Chocolate blanco         |              4001 |          2088.52  | Delicias Artesanas   |
| 442 | Galleta            | GAL0054       | Galleta Extra grande sabor Mora relleno de Dulce de leche y cubierto de Crema                     |              8125 |          4631.77  | Dolce Gusto S.R.L.   |
| 443 | Queque             | QUE0058       | Queque Mini sabor Coco relleno de Dulce de leche y cubierto de Crema                              |              8114 |          5058.15  | Repostería Selecta   |
| 444 | Magdalena          | MAG0042       | Magdalena Individual sabor Mora relleno de Crema de frutilla y cubierto de Chocolate blanco       |              9325 |          5386.83  | Panadería Fina & Co. |
| 445 | Rosca              | ROS0044       | Rosca Extra grande sabor Café relleno de Crema pastelera y cubierto de Merengue                   |              7590 |          5541.49  | Panadería Fina & Co. |
| 446 | Cupcake            | CUP0046       | Cupcake Individual sabor Pera relleno de Crema pastelera y cubierto de Nuez                       |              8801 |          6323.34  | Pan y Más            |
| 447 | Galleta            | GAL0055       | Galleta Extra grande sabor Caramelo relleno de Manjar y cubierto de Nuez                          |              6407 |          4422.77  | Dolce Gusto S.R.L.   |
| 448 | Queque             | QUE0059       | Queque Mini sabor Limon relleno de Crema de chocolate y cubierto de Mani                          |              2679 |          2074.81  | Panadería Fina & Co. |
| 449 | Torta              | TOR0048       | Torta Individual sabor Vainilla relleno de Dulce de leche y cubierto de Nuez                      |              7218 |          3727.99  | Dolce Gusto S.R.L.   |
| 450 | Dona               | DON0058       | Dona Grande sabor Mango relleno de Crema de frutilla y cubierto de Mani                           |              8127 |          6403.43  | Dolce Gusto S.R.L.   |
| 451 | Galleta            | GAL0056       | Galleta Mini sabor Mora relleno de Dulce de leche y cubierto de Chocolate negro                   |              2381 |          1291.01  | Repostería Selecta   |
| 452 | Torta              | TOR0049       | Torta Individual sabor Caramelo relleno de Manjar y cubierto de Mani                              |              7552 |          4933.46  | Panadería Fina & Co. |
| 453 | Rosca              | ROS0045       | Rosca Mini sabor Mango relleno de Dulce de leche y cubierto de Crema                              |              5151 |          3988.68  | Repostería Selecta   |
| 454 | Dona               | DON0059       | Dona Extra grande sabor Mango relleno de Dulce de leche y cubierto de Chocolate blanco            |              3984 |          2816.5   | Panadería Fina & Co. |
| 455 | Torta              | TOR0050       | Torta Grande sabor Coco relleno de Crema de frutilla y cubierto de Chocolate blanco               |              2895 |          2158.89  | Pan y Más            |
| 456 | Torta              | TOR0051       | Torta Extra grande sabor Limon relleno de Crema pastelera y cubierto de Merengue                  |              8108 |          6267.32  | Delicias Artesanas   |
| 457 | Rosca              | ROS0046       | Rosca Individual sabor Chocolate relleno de Manjar y cubierto de Crema                            |              1123 |           655.309 | Delicias Artesanas   |
| 458 | Galleta            | GAL0057       | Galleta Extra grande sabor Caramelo relleno de Crema pastelera y cubierto de Almendras            |              6306 |          3920.05  | Pastelería Gourmet   |
| 459 | Tarta              | TAR0050       | Tarta Grande sabor Pera relleno de Crema de frutilla y cubierto de Nuez                           |              6869 |          3444.36  | Repostería Selecta   |
| 460 | Tarta              | TAR0051       | Tarta Mini sabor Café relleno de Crema de chocolate y cubierto de Chocolate negro                 |              3691 |          1999.62  | Delicias Artesanas   |
| 461 | Queque             | QUE0060       | Queque Extra grande sabor Chocolate relleno de Crema de chocolate y cubierto de Merengue          |              5620 |          3341.4   | Repostería Selecta   |
| 462 | Torta              | TOR0052       | Torta Individual sabor Mango relleno de Crema de frutilla y cubierto de Chocolate negro           |              4553 |          2989.85  | Panadería Fina & Co. |
| 463 | Dona               | DON0060       | Dona Mini sabor Frambuesa relleno de Manjar y cubierto de Nuez                                    |              2261 |          1606.41  | Panadería Fina & Co. |
| 464 | Queque             | QUE0061       | Queque Extra grande sabor Pistacho relleno de Dulce de leche y cubierto de Nuez                   |              4902 |          2937.04  | Repostería Selecta   |
| 465 | Cupcake            | CUP0047       | Cupcake Extra grande sabor Mango relleno de Crema de chocolate y cubierto de Chocolate blanco     |              5145 |          2618.57  | Pan y Más            |
| 466 | Galleta            | GAL0058       | Galleta Extra grande sabor Frutilla relleno de Crema pastelera y cubierto de Chocolate blanco     |              5298 |          2827.83  | Pan y Más            |
| 467 | Torta              | TOR0053       | Torta Individual sabor Plátano relleno de Dulce de leche y cubierto de Chocolate negro            |              7320 |          5085.15  | Repostería Selecta   |
| 468 | Galleta            | GAL0059       | Galleta Individual sabor Frutilla relleno de Dulce de leche y cubierto de Nuez                    |              9746 |          7191.68  | Pan y Más            |
| 469 | Queque             | QUE0062       | Queque Individual sabor Caramelo relleno de Crema de frutilla y cubierto de Merengue              |              5682 |          4309.94  | Delicias Artesanas   |
| 470 | Dona               | DON0061       | Dona Individual sabor Pistacho relleno de Crema de chocolate y cubierto de Merengue               |              4715 |          2807.24  | Pastelería Gourmet   |
| 471 | Queque             | QUE0063       | Queque Mini sabor Plátano relleno de Manjar y cubierto de Almendras                               |              9279 |          5089.91  | Pastelería Gourmet   |
| 472 | Tarta              | TAR0052       | Tarta Mini sabor Vainilla relleno de Dulce de leche y cubierto de Nuez                            |              9128 |          6764.29  | Dolce Gusto S.R.L.   |
| 473 | Rosca              | ROS0047       | Rosca Extra grande sabor Limon relleno de Crema de chocolate y cubierto de Mani                   |              3422 |          2252.85  | Pastelería Gourmet   |
| 474 | Brownie            | BRO0051       | Brownie Extra grande sabor Pera relleno de Crema de frutilla y cubierto de Crema                  |              1851 |           983.135 | Repostería Selecta   |
| 475 | Queque             | QUE0064       | Queque Mini sabor Pistacho relleno de Manjar y cubierto de Nuez                                   |              9962 |          6061.29  | Delicias Artesanas   |
| 476 | Magdalena          | MAG0043       | Magdalena Mini sabor Chocolate relleno de Dulce de leche y cubierto de Merengue                   |              1802 |          1052.31  | Pan y Más            |
| 477 | Magdalena          | MAG0044       | Magdalena Mini sabor Mango relleno de Crema de chocolate y cubierto de Crema                      |              1577 |           789.371 | Panadería Fina & Co. |
| 478 | Queque             | QUE0065       | Queque Individual sabor Coco relleno de Manjar y cubierto de Merengue                             |              9474 |          7114.31  | Dolce Gusto S.R.L.   |
| 479 | Cupcake            | CUP0048       | Cupcake Extra grande sabor Mango relleno de Crema de frutilla y cubierto de Chocolate blanco      |              8433 |          6005.17  | Delicias Artesanas   |
| 480 | Queque             | QUE0066       | Queque Mini sabor Frambuesa relleno de Crema de chocolate y cubierto de Almendras                 |              7400 |          4581.08  | Dolce Gusto S.R.L.   |
| 481 | Queque             | QUE0067       | Queque Extra grande sabor Frutilla relleno de Crema de frutilla y cubierto de Chocolate blanco    |              3791 |          2321.93  | Delicias Artesanas   |
| 482 | Torta              | TOR0054       | Torta Mini sabor Nuez relleno de Manjar y cubierto de Merengue                                    |              8212 |          4961.5   | Pan y Más            |
| 483 | Queque             | QUE0068       | Queque Individual sabor Canela relleno de Crema pastelera y cubierto de Chocolate blanco          |              7780 |          5196.3   | Delicias Artesanas   |
| 484 | Tarta              | TAR0053       | Tarta Individual sabor Café relleno de Crema de chocolate y cubierto de Chocolate negro           |              9996 |          6598.04  | Repostería Selecta   |
| 485 | Brownie            | BRO0052       | Brownie Grande sabor Pera relleno de Crema de frutilla y cubierto de Mani                         |              4694 |          3325.68  | Panadería Fina & Co. |
| 486 | Magdalena          | MAG0045       | Magdalena Individual sabor Pistacho relleno de Dulce de leche y cubierto de Chocolate blanco      |              9418 |          5861.24  | Repostería Selecta   |
| 487 | Brownie            | BRO0053       | Brownie Extra grande sabor Limon relleno de Dulce de leche y cubierto de Crema                    |              1289 |           992.495 | Repostería Selecta   |
| 488 | Torta              | TOR0055       | Torta Individual sabor Frambuesa relleno de Dulce de leche y cubierto de Almendras                |              7577 |          5093.81  | Repostería Selecta   |
| 489 | Rosca              | ROS0048       | Rosca Individual sabor Caramelo relleno de Crema de frutilla y cubierto de Chocolate blanco       |              6879 |          4876.36  | Repostería Selecta   |
| 490 | Queque             | QUE0069       | Queque Grande sabor Coco relleno de Dulce de leche y cubierto de Almendras                        |              9097 |          7050.06  | Delicias Artesanas   |
| 491 | Galleta            | GAL0060       | Galleta Extra grande sabor Pera relleno de Crema de frutilla y cubierto de Crema                  |              4136 |          2472.23  | Pastelería Gourmet   |
| 492 | Rosca              | ROS0049       | Rosca Individual sabor Chocolate relleno de Crema de frutilla y cubierto de Chocolate negro       |              8047 |          6315.02  | Pan y Más            |
| 493 | Cupcake            | CUP0049       | Cupcake Individual sabor Maracuya relleno de Dulce de leche y cubierto de Chocolate blanco        |              5767 |          3378.12  | Panadería Fina & Co. |
| 494 | Torta              | TOR0056       | Torta Individual sabor Limon relleno de Manjar y cubierto de Nuez                                 |              2044 |          1022.81  | Pan y Más            |
| 495 | Cupcake            | CUP0050       | Cupcake Grande sabor Frambuesa relleno de Crema pastelera y cubierto de Almendras                 |              7297 |          3883.17  | Pan y Más            |
| 496 | Brownie            | BRO0054       | Brownie Grande sabor Café relleno de Dulce de leche y cubierto de Mani                            |              6728 |          3844.18  | Dolce Gusto S.R.L.   |
| 497 | Magdalena          | MAG0046       | Magdalena Mini sabor Vainilla relleno de Crema de chocolate y cubierto de Chocolate blanco        |              2234 |          1286.39  | Delicias Artesanas   |
| 498 | Galleta            | GAL0061       | Galleta Individual sabor Coco relleno de Dulce de leche y cubierto de Merengue                    |              9738 |          7052.95  | Pastelería Gourmet   |
| 499 | Queque             | QUE0070       | Queque Grande sabor Limon relleno de Crema pastelera y cubierto de Chocolate blanco               |              2555 |          1305.43  | Pan y Más            |
| 500 | Queque             | QUE0071       | Queque Extra grande sabor Frutilla relleno de Dulce de leche y cubierto de Chocolate negro        |              7717 |          4014.95  | Panadería Fina & Co. |
| 501 | Galleta            | GAL0062       | Galleta Grande sabor Caramelo relleno de Dulce de leche y cubierto de Mani                        |              5453 |          2727.62  | Dolce Gusto S.R.L.   |
| 502 | Magdalena          | MAG0047       | Magdalena Individual sabor Pistacho relleno de Crema de chocolate y cubierto de Chocolate blanco  |              7257 |          4031.66  | Delicias Artesanas   |
| 503 | Tarta              | TAR0054       | Tarta Mini sabor Chocolate relleno de Manjar y cubierto de Chocolate blanco                       |              4107 |          2640.88  | Pastelería Gourmet   |
| 504 | Torta              | TOR0057       | Torta Extra grande sabor Maracuya relleno de Crema de chocolate y cubierto de Chocolate negro     |              6770 |          4524.52  | Panadería Fina & Co. |
| 505 | Dona               | DON0062       | Dona Extra grande sabor Nuez relleno de Crema pastelera y cubierto de Nuez                        |              2034 |          1169.2   | Pan y Más            |
| 506 | Tarta              | TAR0055       | Tarta Grande sabor Plátano relleno de Dulce de leche y cubierto de Chocolate blanco               |              7663 |          4677.34  | Repostería Selecta   |
| 507 | Tarta              | TAR0056       | Tarta Extra grande sabor Maracuya relleno de Crema de frutilla y cubierto de Mani                 |              7557 |          4016.4   | Delicias Artesanas   |
| 508 | Tarta              | TAR0057       | Tarta Individual sabor Café relleno de Dulce de leche y cubierto de Crema                         |              6204 |          4296.38  | Dolce Gusto S.R.L.   |
| 509 | Rosca              | ROS0050       | Rosca Individual sabor Caramelo relleno de Crema de frutilla y cubierto de Mani                   |              3811 |          2158.52  | Pastelería Gourmet   |
| 510 | Galleta            | GAL0063       | Galleta Grande sabor Limon relleno de Dulce de leche y cubierto de Almendras                      |              4327 |          2485.26  | Delicias Artesanas   |
| 511 | Queque             | QUE0072       | Queque Extra grande sabor Pistacho relleno de Dulce de leche y cubierto de Chocolate blanco       |              5865 |          3601.49  | Pastelería Gourmet   |
| 512 | Rosca              | ROS0051       | Rosca Extra grande sabor Coco relleno de Manjar y cubierto de Chocolate blanco                    |              3224 |          2412.35  | Repostería Selecta   |
| 513 | Brownie            | BRO0055       | Brownie Extra grande sabor Vainilla relleno de Crema de chocolate y cubierto de Merengue          |              7388 |          4586.56  | Repostería Selecta   |
| 514 | Brownie            | BRO0056       | Brownie Mini sabor Nuez relleno de Crema pastelera y cubierto de Almendras                        |              8390 |          4602.45  | Delicias Artesanas   |
| 515 | Dona               | DON0063       | Dona Extra grande sabor Caramelo relleno de Crema de frutilla y cubierto de Merengue              |              4240 |          3266.77  | Pastelería Gourmet   |
| 516 | Rosca              | ROS0052       | Rosca Extra grande sabor Coco relleno de Crema de chocolate y cubierto de Crema                   |              6400 |          4750.27  | Repostería Selecta   |
| 517 | Cupcake            | CUP0051       | Cupcake Mini sabor Coco relleno de Manjar y cubierto de Nuez                                      |              1153 |           800.993 | Pastelería Gourmet   |
| 518 | Tarta              | TAR0058       | Tarta Extra grande sabor Vainilla relleno de Crema pastelera y cubierto de Mani                   |              7242 |          3789.22  | Repostería Selecta   |
| 519 | Magdalena          | MAG0048       | Magdalena Grande sabor Chocolate relleno de Crema de frutilla y cubierto de Nuez                  |              1479 |          1074.21  | Repostería Selecta   |
| 520 | Tarta              | TAR0059       | Tarta Mini sabor Coco relleno de Crema pastelera y cubierto de Nuez                               |              7994 |          4013.9   | Pastelería Gourmet   |
| 521 | Magdalena          | MAG0049       | Magdalena Mini sabor Café relleno de Crema pastelera y cubierto de Mani                           |              9352 |          5944.11  | Pan y Más            |
| 522 | Tarta              | TAR0060       | Tarta Grande sabor Limon relleno de Crema de frutilla y cubierto de Chocolate negro               |              9229 |          6731.64  | Delicias Artesanas   |
| 523 | Cupcake            | CUP0052       | Cupcake Extra grande sabor Caramelo relleno de Crema de frutilla y cubierto de Nuez               |              3279 |          1676.21  | Pastelería Gourmet   |
| 524 | Tarta              | TAR0061       | Tarta Grande sabor Mango relleno de Crema de chocolate y cubierto de Almendras                    |              2051 |          1246.41  | Pan y Más            |
| 525 | Tarta              | TAR0062       | Tarta Individual sabor Pera relleno de Crema pastelera y cubierto de Almendras                    |              8020 |          4212.77  | Delicias Artesanas   |
| 526 | Tarta              | TAR0063       | Tarta Grande sabor Pistacho relleno de Crema de chocolate y cubierto de Nuez                      |              6265 |          5005.94  | Pastelería Gourmet   |
| 527 | Torta              | TOR0058       | Torta Extra grande sabor Pistacho relleno de Dulce de leche y cubierto de Crema                   |              4684 |          3468.65  | Repostería Selecta   |
| 528 | Tarta              | TAR0064       | Tarta Grande sabor Limon relleno de Crema de chocolate y cubierto de Almendras                    |              5155 |          3091.35  | Pastelería Gourmet   |
| 529 | Tarta              | TAR0065       | Tarta Grande sabor Maracuya relleno de Dulce de leche y cubierto de Nuez                          |              4780 |          3118.01  | Pastelería Gourmet   |
| 530 | Dona               | DON0064       | Dona Individual sabor Nuez relleno de Crema de frutilla y cubierto de Merengue                    |              1337 |          1061.22  | Dolce Gusto S.R.L.   |
| 531 | Galleta            | GAL0064       | Galleta Grande sabor Mora relleno de Crema de frutilla y cubierto de Merengue                     |              9848 |          7777.15  | Panadería Fina & Co. |
| 532 | Dona               | DON0065       | Dona Grande sabor Plátano relleno de Dulce de leche y cubierto de Crema                           |              1472 |          1136.91  | Panadería Fina & Co. |
| 533 | Galleta            | GAL0065       | Galleta Grande sabor Plátano relleno de Dulce de leche y cubierto de Merengue                     |              9360 |          5667.55  | Panadería Fina & Co. |
| 534 | Dona               | DON0066       | Dona Grande sabor Chocolate relleno de Crema de chocolate y cubierto de Chocolate blanco          |              3342 |          2124.37  | Delicias Artesanas   |
| 535 | Cupcake            | CUP0053       | Cupcake Individual sabor Maracuya relleno de Dulce de leche y cubierto de Crema                   |              3993 |          2976.59  | Panadería Fina & Co. |
| 536 | Cupcake            | CUP0054       | Cupcake Mini sabor Chocolate relleno de Crema pastelera y cubierto de Nuez                        |              2871 |          1627.17  | Repostería Selecta   |
| 537 | Torta              | TOR0059       | Torta Individual sabor Pistacho relleno de Crema de frutilla y cubierto de Chocolate negro        |              5959 |          3768.59  | Dolce Gusto S.R.L.   |
| 538 | Rosca              | ROS0053       | Rosca Grande sabor Coco relleno de Crema pastelera y cubierto de Nuez                             |              8816 |          6272.54  | Dolce Gusto S.R.L.   |
| 539 | Brownie            | BRO0057       | Brownie Mini sabor Frutilla relleno de Crema de chocolate y cubierto de Nuez                      |              6086 |          3993.19  | Pastelería Gourmet   |
| 540 | Torta              | TOR0060       | Torta Grande sabor Canela relleno de Crema de frutilla y cubierto de Merengue                     |              3461 |          1748.49  | Dolce Gusto S.R.L.   |
| 541 | Brownie            | BRO0058       | Brownie Mini sabor Mora relleno de Crema pastelera y cubierto de Almendras                        |              9428 |          5505.42  | Delicias Artesanas   |
| 542 | Tarta              | TAR0066       | Tarta Grande sabor Coco relleno de Crema de frutilla y cubierto de Mani                           |              3584 |          2284.96  | Delicias Artesanas   |
| 543 | Dona               | DON0067       | Dona Individual sabor Caramelo relleno de Crema pastelera y cubierto de Chocolate negro           |              6201 |          3599.28  | Panadería Fina & Co. |
| 544 | Galleta            | GAL0066       | Galleta Extra grande sabor Frambuesa relleno de Dulce de leche y cubierto de Chocolate blanco     |              6323 |          3952.67  | Dolce Gusto S.R.L.   |
| 545 | Dona               | DON0068       | Dona Grande sabor Plátano relleno de Crema de chocolate y cubierto de Almendras                   |              5621 |          4484.53  | Dolce Gusto S.R.L.   |
| 546 | Dona               | DON0069       | Dona Mini sabor Maracuya relleno de Dulce de leche y cubierto de Mani                             |              8683 |          4457.8   | Dolce Gusto S.R.L.   |
| 547 | Magdalena          | MAG0050       | Magdalena Extra grande sabor Mora relleno de Crema pastelera y cubierto de Mani                   |              4596 |          2475.48  | Pastelería Gourmet   |
| 548 | Rosca              | ROS0054       | Rosca Extra grande sabor Plátano relleno de Crema de chocolate y cubierto de Nuez                 |              1509 |           992.209 | Dolce Gusto S.R.L.   |
| 549 | Tarta              | TAR0067       | Tarta Individual sabor Mango relleno de Crema de chocolate y cubierto de Mani                     |              1698 |           999.455 | Pastelería Gourmet   |
| 550 | Dona               | DON0070       | Dona Mini sabor Maracuya relleno de Crema pastelera y cubierto de Merengue                        |              5336 |          3925.76  | Panadería Fina & Co. |
| 551 | Torta              | TOR0061       | Torta Grande sabor Vainilla relleno de Crema de chocolate y cubierto de Crema                     |              7665 |          4687.93  | Pan y Más            |
| 552 | Rosca              | ROS0055       | Rosca Grande sabor Vainilla relleno de Crema de chocolate y cubierto de Mani                      |              3633 |          2446.53  | Repostería Selecta   |
| 553 | Tarta              | TAR0068       | Tarta Individual sabor Pistacho relleno de Crema de chocolate y cubierto de Chocolate blanco      |              1638 |          1141.61  | Delicias Artesanas   |
| 554 | Rosca              | ROS0056       | Rosca Grande sabor Canela relleno de Crema de frutilla y cubierto de Merengue                     |              4070 |          2051.35  | Pastelería Gourmet   |
| 555 | Torta              | TOR0062       | Torta Mini sabor Frambuesa relleno de Crema de frutilla y cubierto de Chocolate negro             |              1856 |          1372.6   | Panadería Fina & Co. |
| 556 | Galleta            | GAL0067       | Galleta Mini sabor Limon relleno de Manjar y cubierto de Crema                                    |              7648 |          5430.36  | Pan y Más            |
| 557 | Torta              | TOR0063       | Torta Grande sabor Mango relleno de Crema pastelera y cubierto de Chocolate negro                 |              2984 |          1976.69  | Delicias Artesanas   |
| 558 | Tarta              | TAR0069       | Tarta Mini sabor Nuez relleno de Manjar y cubierto de Mani                                        |              2959 |          2356     | Pan y Más            |
| 559 | Queque             | QUE0073       | Queque Grande sabor Frambuesa relleno de Crema de chocolate y cubierto de Merengue                |              4114 |          3126.15  | Pan y Más            |
| 560 | Rosca              | ROS0057       | Rosca Mini sabor Maracuya relleno de Crema de frutilla y cubierto de Chocolate negro              |              3932 |          3026.45  | Pastelería Gourmet   |
| 561 | Brownie            | BRO0059       | Brownie Extra grande sabor Pera relleno de Manjar y cubierto de Mani                              |              6935 |          4173.94  | Pan y Más            |
| 562 | Brownie            | BRO0060       | Brownie Mini sabor Vainilla relleno de Manjar y cubierto de Chocolate blanco                      |              1267 |           997.556 | Repostería Selecta   |
| 563 | Cupcake            | CUP0055       | Cupcake Extra grande sabor Nuez relleno de Crema pastelera y cubierto de Almendras                |              2182 |          1707.96  | Delicias Artesanas   |
| 564 | Magdalena          | MAG0051       | Magdalena Mini sabor Plátano relleno de Dulce de leche y cubierto de Crema                        |              2901 |          2000.28  | Repostería Selecta   |
| 565 | Brownie            | BRO0061       | Brownie Grande sabor Pera relleno de Crema de frutilla y cubierto de Nuez                         |              8523 |          5286.72  | Pastelería Gourmet   |
| 566 | Cupcake            | CUP0056       | Cupcake Individual sabor Canela relleno de Crema pastelera y cubierto de Mani                     |              2205 |          1601.72  | Pastelería Gourmet   |
| 567 | Tarta              | TAR0070       | Tarta Grande sabor Canela relleno de Crema pastelera y cubierto de Chocolate negro                |              3812 |          2652.5   | Repostería Selecta   |
| 568 | Galleta            | GAL0068       | Galleta Individual sabor Café relleno de Manjar y cubierto de Almendras                           |              2604 |          1954.83  | Pan y Más            |
| 569 | Tarta              | TAR0071       | Tarta Extra grande sabor Vainilla relleno de Crema de frutilla y cubierto de Merengue             |              8340 |          6533.37  | Dolce Gusto S.R.L.   |
| 570 | Brownie            | BRO0062       | Brownie Grande sabor Canela relleno de Manjar y cubierto de Crema                                 |              4557 |          3213.25  | Pastelería Gourmet   |
| 571 | Queque             | QUE0074       | Queque Individual sabor Vainilla relleno de Crema de chocolate y cubierto de Merengue             |              5068 |          3020.19  | Dolce Gusto S.R.L.   |
| 572 | Brownie            | BRO0063       | Brownie Extra grande sabor Frambuesa relleno de Crema de chocolate y cubierto de Merengue         |              1452 |           950.777 | Panadería Fina & Co. |
| 573 | Galleta            | GAL0069       | Galleta Individual sabor Canela relleno de Crema pastelera y cubierto de Chocolate negro          |              4050 |          2301.45  | Pastelería Gourmet   |
| 574 | Rosca              | ROS0058       | Rosca Individual sabor Canela relleno de Crema de frutilla y cubierto de Almendras                |              3253 |          2136.76  | Pan y Más            |
| 575 | Torta              | TOR0064       | Torta Extra grande sabor Frambuesa relleno de Manjar y cubierto de Nuez                           |              1444 |           725.192 | Pan y Más            |
| 576 | Dona               | DON0071       | Dona Extra grande sabor Pera relleno de Manjar y cubierto de Mani                                 |              5837 |          4040.78  | Pan y Más            |
| 577 | Cupcake            | CUP0057       | Cupcake Grande sabor Mango relleno de Dulce de leche y cubierto de Crema                          |              7840 |          5282.37  | Pan y Más            |
| 578 | Dona               | DON0072       | Dona Extra grande sabor Frutilla relleno de Crema de frutilla y cubierto de Mani                  |              5104 |          2656.94  | Pastelería Gourmet   |
| 579 | Torta              | TOR0065       | Torta Individual sabor Limon relleno de Crema de chocolate y cubierto de Crema                    |              6475 |          4942.66  | Pan y Más            |
| 580 | Galleta            | GAL0070       | Galleta Grande sabor Mango relleno de Crema de frutilla y cubierto de Crema                       |              7213 |          3953.48  | Panadería Fina & Co. |
| 581 | Galleta            | GAL0071       | Galleta Grande sabor Pistacho relleno de Manjar y cubierto de Nuez                                |              7446 |          3977.71  | Pastelería Gourmet   |
| 582 | Cupcake            | CUP0058       | Cupcake Individual sabor Caramelo relleno de Dulce de leche y cubierto de Crema                   |              4324 |          3164.39  | Pan y Más            |
| 583 | Torta              | TOR0066       | Torta Extra grande sabor Plátano relleno de Crema de chocolate y cubierto de Crema                |              8960 |          6312.6   | Dolce Gusto S.R.L.   |
| 584 | Cupcake            | CUP0059       | Cupcake Grande sabor Mora relleno de Crema de chocolate y cubierto de Crema                       |              6392 |          3222.71  | Repostería Selecta   |
| 585 | Brownie            | BRO0064       | Brownie Individual sabor Limon relleno de Dulce de leche y cubierto de Mani                       |              1634 |          1003.68  | Repostería Selecta   |
| 586 | Cupcake            | CUP0060       | Cupcake Extra grande sabor Frambuesa relleno de Crema pastelera y cubierto de Almendras           |              5172 |          3921.77  | Pastelería Gourmet   |
| 587 | Tarta              | TAR0072       | Tarta Individual sabor Limon relleno de Manjar y cubierto de Chocolate negro                      |              7285 |          4584.08  | Pastelería Gourmet   |
| 588 | Dona               | DON0073       | Dona Extra grande sabor Canela relleno de Crema de frutilla y cubierto de Nuez                    |              3806 |          2782.22  | Delicias Artesanas   |
| 589 | Dona               | DON0074       | Dona Extra grande sabor Caramelo relleno de Manjar y cubierto de Nuez                             |              1248 |           951.849 | Pan y Más            |
| 590 | Tarta              | TAR0073       | Tarta Extra grande sabor Pera relleno de Crema de chocolate y cubierto de Crema                   |              6251 |          4390.7   | Delicias Artesanas   |
| 591 | Brownie            | BRO0065       | Brownie Mini sabor Caramelo relleno de Crema de frutilla y cubierto de Chocolate blanco           |              5163 |          3660.19  | Pan y Más            |
| 592 | Queque             | QUE0075       | Queque Extra grande sabor Mango relleno de Crema de frutilla y cubierto de Crema                  |              2751 |          2194.58  | Delicias Artesanas   |
| 593 | Rosca              | ROS0059       | Rosca Extra grande sabor Caramelo relleno de Manjar y cubierto de Crema                           |              6573 |          4712.59  | Delicias Artesanas   |
| 594 | Magdalena          | MAG0052       | Magdalena Individual sabor Plátano relleno de Manjar y cubierto de Chocolate blanco               |              1834 |          1105.4   | Delicias Artesanas   |
| 595 | Dona               | DON0075       | Dona Individual sabor Caramelo relleno de Dulce de leche y cubierto de Chocolate blanco           |              9614 |          7178.56  | Repostería Selecta   |
| 596 | Rosca              | ROS0060       | Rosca Mini sabor Maracuya relleno de Dulce de leche y cubierto de Chocolate negro                 |              5856 |          4272.31  | Panadería Fina & Co. |
| 597 | Magdalena          | MAG0053       | Magdalena Mini sabor Plátano relleno de Dulce de leche y cubierto de Chocolate negro              |              1871 |           990.571 | Delicias Artesanas   |
| 598 | Queque             | QUE0076       | Queque Grande sabor Vainilla relleno de Dulce de leche y cubierto de Merengue                     |              5650 |          2973.25  | Pastelería Gourmet   |
| 599 | Queque             | QUE0077       | Queque Individual sabor Chocolate relleno de Manjar y cubierto de Chocolate negro                 |              5261 |          3007.76  | Delicias Artesanas   |
| 600 | Cupcake            | CUP0061       | Cupcake Grande sabor Mora relleno de Dulce de leche y cubierto de Merengue                        |              9878 |          6908.2   | Dolce Gusto S.R.L.   |
| 601 | Tarta              | TAR0074       | Tarta Individual sabor Chocolate relleno de Manjar y cubierto de Nuez                             |              7801 |          4496.98  | Repostería Selecta   |
| 602 | Brownie            | BRO0066       | Brownie Grande sabor Pistacho relleno de Crema pastelera y cubierto de Merengue                   |              9508 |          5072.22  | Pastelería Gourmet   |
| 603 | Cupcake            | CUP0062       | Cupcake Individual sabor Canela relleno de Dulce de leche y cubierto de Nuez                      |              9457 |          7259.62  | Dolce Gusto S.R.L.   |
| 604 | Magdalena          | MAG0054       | Magdalena Grande sabor Vainilla relleno de Crema de chocolate y cubierto de Nuez                  |              9930 |          5497.09  | Dolce Gusto S.R.L.   |
| 605 | Galleta            | GAL0072       | Galleta Mini sabor Mora relleno de Manjar y cubierto de Chocolate negro                           |              3736 |          2761.38  | Repostería Selecta   |
| 606 | Magdalena          | MAG0055       | Magdalena Individual sabor Frambuesa relleno de Manjar y cubierto de Mani                         |              2793 |          1698.59  | Panadería Fina & Co. |
| 607 | Torta              | TOR0067       | Torta Mini sabor Frambuesa relleno de Crema de frutilla y cubierto de Merengue                    |              7515 |          5150.84  | Repostería Selecta   |
| 608 | Dona               | DON0076       | Dona Extra grande sabor Limon relleno de Crema de chocolate y cubierto de Mani                    |              1771 |          1327.48  | Repostería Selecta   |
| 609 | Brownie            | BRO0067       | Brownie Extra grande sabor Maracuya relleno de Manjar y cubierto de Merengue                      |              8199 |          6184     | Dolce Gusto S.R.L.   |
| 610 | Dona               | DON0077       | Dona Extra grande sabor Canela relleno de Crema de frutilla y cubierto de Chocolate negro         |              7670 |          5529.05  | Pastelería Gourmet   |
| 611 | Galleta            | GAL0073       | Galleta Mini sabor Frambuesa relleno de Manjar y cubierto de Nuez                                 |              9724 |          5615.31  | Pastelería Gourmet   |
| 612 | Galleta            | GAL0074       | Galleta Mini sabor Pistacho relleno de Crema de frutilla y cubierto de Mani                       |              9639 |          7155.67  | Panadería Fina & Co. |
| 613 | Magdalena          | MAG0056       | Magdalena Extra grande sabor Limon relleno de Crema de frutilla y cubierto de Merengue            |              3429 |          2038.92  | Pan y Más            |
| 614 | Tarta              | TAR0075       | Tarta Extra grande sabor Frambuesa relleno de Dulce de leche y cubierto de Merengue               |              6006 |          3985.66  | Pastelería Gourmet   |
| 615 | Brownie            | BRO0068       | Brownie Grande sabor Pera relleno de Dulce de leche y cubierto de Nuez                            |              2417 |          1595.33  | Pan y Más            |
| 616 | Cupcake            | CUP0063       | Cupcake Grande sabor Pistacho relleno de Crema de frutilla y cubierto de Mani                     |              6886 |          3752.14  | Delicias Artesanas   |
| 617 | Galleta            | GAL0075       | Galleta Grande sabor Vainilla relleno de Crema de chocolate y cubierto de Nuez                    |              5954 |          4510.68  | Panadería Fina & Co. |
| 618 | Tarta              | TAR0076       | Tarta Extra grande sabor Chocolate relleno de Crema pastelera y cubierto de Chocolate blanco      |              3150 |          2200.24  | Repostería Selecta   |
| 619 | Magdalena          | MAG0057       | Magdalena Extra grande sabor Vainilla relleno de Crema de chocolate y cubierto de Nuez            |              4588 |          3216.53  | Panadería Fina & Co. |
| 620 | Rosca              | ROS0061       | Rosca Individual sabor Chocolate relleno de Crema de chocolate y cubierto de Nuez                 |              7434 |          5771.5   | Repostería Selecta   |
| 621 | Brownie            | BRO0069       | Brownie Extra grande sabor Chocolate relleno de Crema de frutilla y cubierto de Crema             |              4856 |          3675.38  | Repostería Selecta   |
| 622 | Torta              | TOR0068       | Torta Extra grande sabor Maracuya relleno de Crema de chocolate y cubierto de Nuez                |              9214 |          7159.36  | Dolce Gusto S.R.L.   |
| 623 | Queque             | QUE0078       | Queque Extra grande sabor Pistacho relleno de Manjar y cubierto de Chocolate blanco               |              3844 |          3026.28  | Dolce Gusto S.R.L.   |
| 624 | Cupcake            | CUP0064       | Cupcake Individual sabor Café relleno de Crema de frutilla y cubierto de Mani                     |              8394 |          5085.84  | Dolce Gusto S.R.L.   |
| 625 | Cupcake            | CUP0065       | Cupcake Individual sabor Frambuesa relleno de Crema de chocolate y cubierto de Merengue           |              3961 |          2737.89  | Pan y Más            |
| 626 | Galleta            | GAL0076       | Galleta Grande sabor Frutilla relleno de Dulce de leche y cubierto de Crema                       |              8903 |          4560.92  | Pan y Más            |
| 627 | Brownie            | BRO0070       | Brownie Grande sabor Pera relleno de Manjar y cubierto de Crema                                   |              4776 |          3117.23  | Repostería Selecta   |
| 628 | Tarta              | TAR0077       | Tarta Extra grande sabor Frutilla relleno de Crema de frutilla y cubierto de Chocolate blanco     |              4311 |          2670.84  | Repostería Selecta   |
| 629 | Galleta            | GAL0077       | Galleta Grande sabor Caramelo relleno de Dulce de leche y cubierto de Chocolate blanco            |              9419 |          5830     | Pastelería Gourmet   |
| 630 | Torta              | TOR0069       | Torta Grande sabor Pera relleno de Crema de frutilla y cubierto de Merengue                       |              7636 |          5724.55  | Pastelería Gourmet   |
| 631 | Brownie            | BRO0071       | Brownie Extra grande sabor Caramelo relleno de Dulce de leche y cubierto de Mani                  |              5838 |          3391.74  | Pastelería Gourmet   |
| 632 | Torta              | TOR0070       | Torta Extra grande sabor Chocolate relleno de Crema de chocolate y cubierto de Chocolate blanco   |              1862 |           934.374 | Delicias Artesanas   |
| 633 | Tarta              | TAR0078       | Tarta Grande sabor Maracuya relleno de Crema pastelera y cubierto de Mani                         |              3203 |          2215.03  | Repostería Selecta   |
| 634 | Queque             | QUE0079       | Queque Mini sabor Frutilla relleno de Dulce de leche y cubierto de Chocolate negro                |              7338 |          3826.22  | Pan y Más            |
| 635 | Rosca              | ROS0062       | Rosca Mini sabor Maracuya relleno de Crema de chocolate y cubierto de Chocolate blanco            |              8833 |          6925.91  | Repostería Selecta   |
| 636 | Galleta            | GAL0078       | Galleta Extra grande sabor Caramelo relleno de Dulce de leche y cubierto de Chocolate negro       |              7883 |          6261.42  | Pan y Más            |
| 637 | Tarta              | TAR0079       | Tarta Mini sabor Chocolate relleno de Manjar y cubierto de Chocolate blanco                       |              3546 |          2094.12  | Repostería Selecta   |
| 638 | Dona               | DON0078       | Dona Individual sabor Caramelo relleno de Manjar y cubierto de Crema                              |              4718 |          2657.5   | Repostería Selecta   |
| 639 | Brownie            | BRO0072       | Brownie Individual sabor Limon relleno de Crema pastelera y cubierto de Chocolate negro           |              2792 |          1884.19  | Delicias Artesanas   |
| 640 | Galleta            | GAL0079       | Galleta Grande sabor Chocolate relleno de Manjar y cubierto de Chocolate blanco                   |              9607 |          5436.64  | Pan y Más            |
| 641 | Torta              | TOR0071       | Torta Mini sabor Maracuya relleno de Manjar y cubierto de Nuez                                    |              1939 |          1260.59  | Pastelería Gourmet   |
| 642 | Torta              | TOR0072       | Torta Mini sabor Nuez relleno de Dulce de leche y cubierto de Chocolate blanco                    |              1024 |           641.094 | Repostería Selecta   |
| 643 | Brownie            | BRO0073       | Brownie Extra grande sabor Frutilla relleno de Crema pastelera y cubierto de Chocolate negro      |              9043 |          5783.07  | Repostería Selecta   |
| 644 | Galleta            | GAL0080       | Galleta Grande sabor Vainilla relleno de Crema pastelera y cubierto de Nuez                       |              6605 |          3310.52  | Pastelería Gourmet   |
| 645 | Galleta            | GAL0081       | Galleta Mini sabor Nuez relleno de Crema de frutilla y cubierto de Almendras                      |              2883 |          2131     | Repostería Selecta   |
| 646 | Galleta            | GAL0082       | Galleta Grande sabor Pistacho relleno de Crema pastelera y cubierto de Crema                      |              4840 |          3822.34  | Panadería Fina & Co. |
| 647 | Galleta            | GAL0083       | Galleta Extra grande sabor Limon relleno de Manjar y cubierto de Almendras                        |              3945 |          2750.62  | Panadería Fina & Co. |
| 648 | Tarta              | TAR0080       | Tarta Mini sabor Maracuya relleno de Manjar y cubierto de Chocolate blanco                        |              4761 |          2807.78  | Dolce Gusto S.R.L.   |
| 649 | Magdalena          | MAG0058       | Magdalena Extra grande sabor Café relleno de Dulce de leche y cubierto de Crema                   |              1158 |           654.179 | Dolce Gusto S.R.L.   |
| 650 | Queque             | QUE0080       | Queque Extra grande sabor Pistacho relleno de Crema pastelera y cubierto de Almendras             |              4595 |          3070.69  | Pan y Más            |
| 651 | Queque             | QUE0081       | Queque Individual sabor Coco relleno de Manjar y cubierto de Crema                                |              4964 |          3286.48  | Repostería Selecta   |
| 652 | Torta              | TOR0073       | Torta Individual sabor Coco relleno de Crema de chocolate y cubierto de Nuez                      |              3399 |          1970.14  | Pastelería Gourmet   |
| 653 | Queque             | QUE0082       | Queque Mini sabor Coco relleno de Crema pastelera y cubierto de Chocolate negro                   |              5648 |          3430.13  | Dolce Gusto S.R.L.   |
| 654 | Torta              | TOR0074       | Torta Mini sabor Limon relleno de Dulce de leche y cubierto de Crema                              |              5993 |          4115.89  | Repostería Selecta   |
| 655 | Brownie            | BRO0074       | Brownie Individual sabor Canela relleno de Crema de frutilla y cubierto de Nuez                   |              1998 |          1412.69  | Panadería Fina & Co. |
| 656 | Queque             | QUE0083       | Queque Mini sabor Plátano relleno de Crema de frutilla y cubierto de Merengue                     |              5737 |          2954.49  | Pan y Más            |
| 657 | Dona               | DON0079       | Dona Individual sabor Frutilla relleno de Manjar y cubierto de Almendras                          |              6004 |          4219.15  | Delicias Artesanas   |
| 658 | Cupcake            | CUP0066       | Cupcake Individual sabor Canela relleno de Crema pastelera y cubierto de Nuez                     |              6989 |          5043.79  | Repostería Selecta   |
| 659 | Galleta            | GAL0084       | Galleta Extra grande sabor Caramelo relleno de Dulce de leche y cubierto de Merengue              |              3656 |          2559.74  | Pastelería Gourmet   |
| 660 | Dona               | DON0080       | Dona Individual sabor Canela relleno de Manjar y cubierto de Mani                                 |              7527 |          4660.89  | Dolce Gusto S.R.L.   |
| 661 | Torta              | TOR0075       | Torta Grande sabor Coco relleno de Crema de frutilla y cubierto de Chocolate negro                |              7173 |          3961.07  | Dolce Gusto S.R.L.   |
| 662 | Rosca              | ROS0063       | Rosca Extra grande sabor Canela relleno de Crema pastelera y cubierto de Almendras                |              8615 |          4524.15  | Repostería Selecta   |
| 663 | Queque             | QUE0084       | Queque Extra grande sabor Plátano relleno de Crema de chocolate y cubierto de Merengue            |              3205 |          1987.83  | Delicias Artesanas   |
| 664 | Tarta              | TAR0081       | Tarta Individual sabor Frambuesa relleno de Dulce de leche y cubierto de Chocolate blanco         |              2230 |          1149.2   | Pastelería Gourmet   |
| 665 | Brownie            | BRO0075       | Brownie Extra grande sabor Café relleno de Dulce de leche y cubierto de Chocolate blanco          |              2173 |          1314.51  | Repostería Selecta   |
| 666 | Rosca              | ROS0064       | Rosca Individual sabor Pera relleno de Manjar y cubierto de Nuez                                  |              3711 |          2767.98  | Dolce Gusto S.R.L.   |
| 667 | Tarta              | TAR0082       | Tarta Grande sabor Canela relleno de Manjar y cubierto de Mani                                    |              1272 |           700.551 | Panadería Fina & Co. |
| 668 | Tarta              | TAR0083       | Tarta Grande sabor Mora relleno de Manjar y cubierto de Chocolate blanco                          |              9428 |          5210.53  | Delicias Artesanas   |
| 669 | Dona               | DON0081       | Dona Grande sabor Nuez relleno de Crema de chocolate y cubierto de Almendras                      |              1801 |          1283.32  | Delicias Artesanas   |
| 670 | Rosca              | ROS0065       | Rosca Grande sabor Maracuya relleno de Crema pastelera y cubierto de Crema                        |              8163 |          5523.56  | Pastelería Gourmet   |
| 671 | Dona               | DON0082       | Dona Extra grande sabor Vainilla relleno de Dulce de leche y cubierto de Mani                     |              2228 |          1431.53  | Panadería Fina & Co. |
| 672 | Cupcake            | CUP0067       | Cupcake Grande sabor Vainilla relleno de Crema pastelera y cubierto de Almendras                  |              5818 |          3981.88  | Pan y Más            |
| 673 | Cupcake            | CUP0068       | Cupcake Individual sabor Plátano relleno de Crema pastelera y cubierto de Almendras               |              2114 |          1500.87  | Dolce Gusto S.R.L.   |
| 674 | Queque             | QUE0085       | Queque Individual sabor Mango relleno de Crema de chocolate y cubierto de Nuez                    |              9144 |          5169.57  | Repostería Selecta   |
| 675 | Tarta              | TAR0084       | Tarta Mini sabor Vainilla relleno de Crema de chocolate y cubierto de Merengue                    |              3843 |          2256.53  | Repostería Selecta   |
| 676 | Torta              | TOR0076       | Torta Extra grande sabor Limon relleno de Crema pastelera y cubierto de Nuez                      |              1425 |           940.955 | Delicias Artesanas   |
| 677 | Galleta            | GAL0085       | Galleta Individual sabor Nuez relleno de Crema de frutilla y cubierto de Nuez                     |              9012 |          6903.12  | Dolce Gusto S.R.L.   |
| 678 | Galleta            | GAL0086       | Galleta Individual sabor Vainilla relleno de Manjar y cubierto de Chocolate negro                 |              3761 |          2943.42  | Panadería Fina & Co. |
| 679 | Dona               | DON0083       | Dona Mini sabor Maracuya relleno de Crema pastelera y cubierto de Almendras                       |              2932 |          2286.05  | Delicias Artesanas   |
| 680 | Brownie            | BRO0076       | Brownie Mini sabor Plátano relleno de Crema de frutilla y cubierto de Merengue                    |              6483 |          4329.06  | Panadería Fina & Co. |
| 681 | Brownie            | BRO0077       | Brownie Extra grande sabor Coco relleno de Crema pastelera y cubierto de Almendras                |              7195 |          4099.39  | Repostería Selecta   |
| 682 | Dona               | DON0084       | Dona Extra grande sabor Coco relleno de Crema pastelera y cubierto de Almendras                   |              6817 |          4643.73  | Pan y Más            |
| 683 | Rosca              | ROS0066       | Rosca Extra grande sabor Frambuesa relleno de Crema pastelera y cubierto de Chocolate negro       |              3919 |          2506.65  | Pastelería Gourmet   |
| 684 | Galleta            | GAL0087       | Galleta Grande sabor Nuez relleno de Manjar y cubierto de Chocolate blanco                        |              5802 |          4495.93  | Repostería Selecta   |
| 685 | Brownie            | BRO0078       | Brownie Individual sabor Café relleno de Crema de frutilla y cubierto de Chocolate negro          |              1738 |          1057.32  | Delicias Artesanas   |
| 686 | Brownie            | BRO0079       | Brownie Extra grande sabor Café relleno de Manjar y cubierto de Chocolate blanco                  |              6688 |          5162.18  | Repostería Selecta   |
| 687 | Cupcake            | CUP0069       | Cupcake Extra grande sabor Chocolate relleno de Dulce de leche y cubierto de Chocolate negro      |              3252 |          1881.98  | Delicias Artesanas   |
| 688 | Queque             | QUE0086       | Queque Extra grande sabor Vainilla relleno de Crema pastelera y cubierto de Mani                  |              7869 |          4548.96  | Repostería Selecta   |
| 689 | Queque             | QUE0087       | Queque Extra grande sabor Frambuesa relleno de Crema de frutilla y cubierto de Crema              |              6690 |          3587.04  | Pastelería Gourmet   |
| 690 | Galleta            | GAL0088       | Galleta Individual sabor Maracuya relleno de Dulce de leche y cubierto de Mani                    |              3136 |          1879.22  | Dolce Gusto S.R.L.   |
| 691 | Galleta            | GAL0089       | Galleta Mini sabor Pistacho relleno de Crema pastelera y cubierto de Mani                         |              3693 |          2809.88  | Pan y Más            |
| 692 | Rosca              | ROS0067       | Rosca Extra grande sabor Chocolate relleno de Manjar y cubierto de Merengue                       |              4827 |          2938.89  | Pan y Más            |
| 693 | Galleta            | GAL0090       | Galleta Individual sabor Pera relleno de Manjar y cubierto de Crema                               |              8262 |          4525.7   | Pan y Más            |
| 694 | Torta              | TOR0077       | Torta Grande sabor Limon relleno de Dulce de leche y cubierto de Almendras                        |              6900 |          4266.21  | Repostería Selecta   |
| 695 | Cupcake            | CUP0070       | Cupcake Extra grande sabor Mango relleno de Crema de frutilla y cubierto de Mani                  |              8457 |          4942.42  | Pan y Más            |
| 696 | Tarta              | TAR0085       | Tarta Mini sabor Nuez relleno de Crema de frutilla y cubierto de Chocolate blanco                 |              7713 |          5899.94  | Pastelería Gourmet   |
| 697 | Magdalena          | MAG0059       | Magdalena Individual sabor Pera relleno de Crema de frutilla y cubierto de Mani                   |              8195 |          6086.7   | Dolce Gusto S.R.L.   |
| 698 | Galleta            | GAL0091       | Galleta Grande sabor Mango relleno de Manjar y cubierto de Mani                                   |              7553 |          5840.56  | Pastelería Gourmet   |
| 699 | Rosca              | ROS0068       | Rosca Grande sabor Coco relleno de Crema de chocolate y cubierto de Merengue                      |              2914 |          1962.43  | Delicias Artesanas   |
| 700 | Torta              | TOR0078       | Torta Individual sabor Chocolate relleno de Manjar y cubierto de Chocolate blanco                 |              4052 |          2752.39  | Pastelería Gourmet   |
| 701 | Torta              | TOR0079       | Torta Individual sabor Limon relleno de Crema de chocolate y cubierto de Crema                    |              3635 |          2274.41  | Dolce Gusto S.R.L.   |
| 702 | Rosca              | ROS0069       | Rosca Mini sabor Maracuya relleno de Crema pastelera y cubierto de Mani                           |              9174 |          6219.19  | Repostería Selecta   |
| 703 | Tarta              | TAR0086       | Tarta Mini sabor Maracuya relleno de Crema de frutilla y cubierto de Nuez                         |              4029 |          2759.88  | Panadería Fina & Co. |
| 704 | Galleta            | GAL0092       | Galleta Individual sabor Mora relleno de Crema pastelera y cubierto de Crema                      |              6523 |          3918.77  | Pastelería Gourmet   |
| 705 | Tarta              | TAR0087       | Tarta Grande sabor Mora relleno de Crema de chocolate y cubierto de Mani                          |              6677 |          4193.22  | Repostería Selecta   |
| 706 | Magdalena          | MAG0060       | Magdalena Mini sabor Maracuya relleno de Crema pastelera y cubierto de Mani                       |              6526 |          3790.79  | Pastelería Gourmet   |
| 707 | Dona               | DON0085       | Dona Individual sabor Pistacho relleno de Crema pastelera y cubierto de Chocolate negro           |              5312 |          4208.95  | Repostería Selecta   |
| 708 | Queque             | QUE0088       | Queque Extra grande sabor Café relleno de Crema pastelera y cubierto de Almendras                 |              9157 |          4598.29  | Dolce Gusto S.R.L.   |
| 709 | Magdalena          | MAG0061       | Magdalena Individual sabor Pera relleno de Crema de chocolate y cubierto de Almendras             |              1146 |           575.484 | Panadería Fina & Co. |
| 710 | Cupcake            | CUP0071       | Cupcake Extra grande sabor Caramelo relleno de Crema pastelera y cubierto de Nuez                 |              1230 |           706.987 | Dolce Gusto S.R.L.   |
| 711 | Tarta              | TAR0088       | Tarta Extra grande sabor Limon relleno de Manjar y cubierto de Chocolate negro                    |              8945 |          5997.7   | Pastelería Gourmet   |
| 712 | Queque             | QUE0089       | Queque Mini sabor Canela relleno de Dulce de leche y cubierto de Nuez                             |              8112 |          4637.66  | Panadería Fina & Co. |
| 713 | Torta              | TOR0080       | Torta Extra grande sabor Plátano relleno de Crema pastelera y cubierto de Almendras               |              6229 |          3256.48  | Pastelería Gourmet   |
| 714 | Torta              | TOR0081       | Torta Grande sabor Vainilla relleno de Crema pastelera y cubierto de Almendras                    |              8888 |          6609.35  | Delicias Artesanas   |
| 715 | Brownie            | BRO0080       | Brownie Mini sabor Maracuya relleno de Crema de frutilla y cubierto de Chocolate negro            |              3917 |          2571.59  | Pan y Más            |
| 716 | Queque             | QUE0090       | Queque Extra grande sabor Frambuesa relleno de Dulce de leche y cubierto de Chocolate blanco      |              6855 |          3616.65  | Pastelería Gourmet   |
| 717 | Queque             | QUE0091       | Queque Mini sabor Frutilla relleno de Crema de chocolate y cubierto de Chocolate blanco           |              3425 |          2364.37  | Panadería Fina & Co. |
| 718 | Galleta            | GAL0093       | Galleta Mini sabor Canela relleno de Crema de frutilla y cubierto de Crema                        |              6681 |          3641.96  | Delicias Artesanas   |
| 719 | Galleta            | GAL0094       | Galleta Individual sabor Coco relleno de Crema de chocolate y cubierto de Nuez                    |              7686 |          4688.67  | Panadería Fina & Co. |
| 720 | Dona               | DON0086       | Dona Grande sabor Plátano relleno de Manjar y cubierto de Chocolate negro                         |              5103 |          3202.97  | Pastelería Gourmet   |
| 721 | Queque             | QUE0092       | Queque Grande sabor Nuez relleno de Dulce de leche y cubierto de Crema                            |              9430 |          5916.22  | Pan y Más            |
| 722 | Magdalena          | MAG0062       | Magdalena Mini sabor Canela relleno de Crema de frutilla y cubierto de Almendras                  |              3806 |          2896.09  | Delicias Artesanas   |
| 723 | Dona               | DON0087       | Dona Individual sabor Pera relleno de Crema de frutilla y cubierto de Nuez                        |              4064 |          3036.72  | Pastelería Gourmet   |
| 724 | Brownie            | BRO0081       | Brownie Extra grande sabor Nuez relleno de Crema pastelera y cubierto de Chocolate blanco         |              9116 |          5330.99  | Pastelería Gourmet   |
| 725 | Dona               | DON0088       | Dona Grande sabor Nuez relleno de Crema de frutilla y cubierto de Nuez                            |              3891 |          2144.2   | Pan y Más            |
| 726 | Galleta            | GAL0095       | Galleta Individual sabor Pera relleno de Crema de chocolate y cubierto de Chocolate blanco        |              8092 |          4795.75  | Repostería Selecta   |
| 727 | Magdalena          | MAG0063       | Magdalena Individual sabor Canela relleno de Manjar y cubierto de Mani                            |              7850 |          5769.13  | Repostería Selecta   |
| 728 | Queque             | QUE0093       | Queque Mini sabor Pera relleno de Dulce de leche y cubierto de Merengue                           |              5883 |          2964.96  | Repostería Selecta   |
| 729 | Brownie            | BRO0082       | Brownie Mini sabor Limon relleno de Crema pastelera y cubierto de Mani                            |              4553 |          3161.08  | Pastelería Gourmet   |
| 730 | Cupcake            | CUP0072       | Cupcake Grande sabor Coco relleno de Crema pastelera y cubierto de Merengue                       |              5839 |          3645.93  | Panadería Fina & Co. |
| 731 | Rosca              | ROS0070       | Rosca Extra grande sabor Pera relleno de Dulce de leche y cubierto de Merengue                    |              3687 |          2103     | Repostería Selecta   |
| 732 | Brownie            | BRO0083       | Brownie Individual sabor Mango relleno de Crema de frutilla y cubierto de Nuez                    |              2063 |          1288.27  | Dolce Gusto S.R.L.   |
| 733 | Cupcake            | CUP0073       | Cupcake Extra grande sabor Caramelo relleno de Crema de chocolate y cubierto de Crema             |              2477 |          1881.55  | Panadería Fina & Co. |
| 734 | Torta              | TOR0082       | Torta Extra grande sabor Café relleno de Dulce de leche y cubierto de Merengue                    |              8273 |          4450.32  | Delicias Artesanas   |
| 735 | Torta              | TOR0083       | Torta Individual sabor Vainilla relleno de Crema de frutilla y cubierto de Crema                  |              6995 |          4664.78  | Panadería Fina & Co. |
| 736 | Magdalena          | MAG0064       | Magdalena Individual sabor Café relleno de Dulce de leche y cubierto de Merengue                  |              1637 |          1077.54  | Dolce Gusto S.R.L.   |
| 737 | Rosca              | ROS0071       | Rosca Mini sabor Mango relleno de Dulce de leche y cubierto de Chocolate negro                    |              6372 |          4653.13  | Panadería Fina & Co. |
| 738 | Cupcake            | CUP0074       | Cupcake Extra grande sabor Frutilla relleno de Crema de frutilla y cubierto de Chocolate blanco   |              3031 |          2060.06  | Pastelería Gourmet   |
| 739 | Rosca              | ROS0072       | Rosca Grande sabor Mango relleno de Crema de frutilla y cubierto de Nuez                          |              9469 |          5361.16  | Dolce Gusto S.R.L.   |
| 740 | Magdalena          | MAG0065       | Magdalena Extra grande sabor Chocolate relleno de Crema pastelera y cubierto de Chocolate negro   |              5905 |          4069.86  | Delicias Artesanas   |
| 741 | Rosca              | ROS0073       | Rosca Grande sabor Nuez relleno de Crema de frutilla y cubierto de Nuez                           |              4688 |          2995.36  | Dolce Gusto S.R.L.   |
| 742 | Torta              | TOR0084       | Torta Mini sabor Canela relleno de Crema de chocolate y cubierto de Nuez                          |              6623 |          4755.54  | Dolce Gusto S.R.L.   |
| 743 | Galleta            | GAL0096       | Galleta Extra grande sabor Plátano relleno de Crema de chocolate y cubierto de Nuez               |              9640 |          6689     | Dolce Gusto S.R.L.   |
| 744 | Torta              | TOR0085       | Torta Individual sabor Frambuesa relleno de Manjar y cubierto de Merengue                         |              6234 |          4720.72  | Dolce Gusto S.R.L.   |
| 745 | Tarta              | TAR0089       | Tarta Grande sabor Coco relleno de Crema de frutilla y cubierto de Chocolate blanco               |              3093 |          2172.65  | Delicias Artesanas   |
| 746 | Dona               | DON0089       | Dona Individual sabor Vainilla relleno de Crema de chocolate y cubierto de Almendras              |              4128 |          2355.75  | Panadería Fina & Co. |
| 747 | Tarta              | TAR0090       | Tarta Individual sabor Chocolate relleno de Crema de chocolate y cubierto de Chocolate negro      |              3093 |          1689.57  | Pastelería Gourmet   |
| 748 | Torta              | TOR0086       | Torta Grande sabor Limon relleno de Dulce de leche y cubierto de Mani                             |              8594 |          4343.16  | Pan y Más            |
| 749 | Rosca              | ROS0074       | Rosca Individual sabor Vainilla relleno de Crema de chocolate y cubierto de Almendras             |              1773 |           916.271 | Repostería Selecta   |
| 750 | Cupcake            | CUP0075       | Cupcake Extra grande sabor Caramelo relleno de Dulce de leche y cubierto de Nuez                  |              8100 |          4954.75  | Pan y Más            |
| 751 | Cupcake            | CUP0076       | Cupcake Mini sabor Mora relleno de Crema de chocolate y cubierto de Chocolate blanco              |              7689 |          5941.56  | Panadería Fina & Co. |
| 752 | Cupcake            | CUP0077       | Cupcake Grande sabor Mora relleno de Crema de frutilla y cubierto de Crema                        |              1629 |          1247.74  | Delicias Artesanas   |
| 753 | Torta              | TOR0087       | Torta Individual sabor Chocolate relleno de Crema pastelera y cubierto de Almendras               |              7398 |          4056.67  | Delicias Artesanas   |
| 754 | Torta              | TOR0088       | Torta Individual sabor Frutilla relleno de Crema pastelera y cubierto de Chocolate blanco         |              9524 |          7258.94  | Repostería Selecta   |
| 755 | Galleta            | GAL0097       | Galleta Grande sabor Chocolate relleno de Manjar y cubierto de Chocolate negro                    |              1963 |          1433.36  | Pan y Más            |
| 756 | Brownie            | BRO0084       | Brownie Extra grande sabor Pera relleno de Crema de frutilla y cubierto de Crema                  |              3899 |          3035.71  | Pan y Más            |
| 757 | Dona               | DON0090       | Dona Grande sabor Vainilla relleno de Manjar y cubierto de Chocolate blanco                       |              9871 |          5870.1   | Delicias Artesanas   |
| 758 | Rosca              | ROS0075       | Rosca Extra grande sabor Limon relleno de Dulce de leche y cubierto de Almendras                  |              3433 |          2666.48  | Dolce Gusto S.R.L.   |
| 759 | Dona               | DON0091       | Dona Extra grande sabor Caramelo relleno de Crema pastelera y cubierto de Crema                   |              1997 |          1131.1   | Panadería Fina & Co. |
| 760 | Tarta              | TAR0091       | Tarta Extra grande sabor Frambuesa relleno de Crema de chocolate y cubierto de Chocolate negro    |              9246 |          6030.84  | Panadería Fina & Co. |
| 761 | Magdalena          | MAG0066       | Magdalena Mini sabor Mango relleno de Dulce de leche y cubierto de Chocolate negro                |              4003 |          3028.29  | Repostería Selecta   |
| 762 | Queque             | QUE0094       | Queque Individual sabor Caramelo relleno de Crema de chocolate y cubierto de Almendras            |              5789 |          2918.85  | Dolce Gusto S.R.L.   |
| 763 | Galleta            | GAL0098       | Galleta Extra grande sabor Café relleno de Crema de chocolate y cubierto de Merengue              |              1091 |           867.868 | Delicias Artesanas   |
| 764 | Brownie            | BRO0085       | Brownie Mini sabor Chocolate relleno de Manjar y cubierto de Nuez                                 |              5504 |          4284.81  | Delicias Artesanas   |
| 765 | Torta              | TOR0089       | Torta Individual sabor Plátano relleno de Crema pastelera y cubierto de Chocolate negro           |              8260 |          4535.24  | Panadería Fina & Co. |
| 766 | Tarta              | TAR0092       | Tarta Individual sabor Coco relleno de Manjar y cubierto de Nuez                                  |              4447 |          2939.26  | Dolce Gusto S.R.L.   |
| 767 | Magdalena          | MAG0067       | Magdalena Mini sabor Café relleno de Manjar y cubierto de Crema                                   |              8454 |          4587.78  | Pan y Más            |
| 768 | Dona               | DON0092       | Dona Individual sabor Chocolate relleno de Crema pastelera y cubierto de Crema                    |              4020 |          2944.64  | Pan y Más            |
| 769 | Dona               | DON0093       | Dona Individual sabor Coco relleno de Crema pastelera y cubierto de Chocolate negro               |              6507 |          4576.49  | Pastelería Gourmet   |
| 770 | Tarta              | TAR0093       | Tarta Mini sabor Plátano relleno de Crema de frutilla y cubierto de Nuez                          |              7569 |          5524.57  | Pastelería Gourmet   |
| 771 | Cupcake            | CUP0078       | Cupcake Mini sabor Frutilla relleno de Crema de frutilla y cubierto de Crema                      |              9868 |          7089.78  | Pastelería Gourmet   |
| 772 | Galleta            | GAL0099       | Galleta Mini sabor Mora relleno de Crema de chocolate y cubierto de Chocolate negro               |              8989 |          6586.18  | Delicias Artesanas   |
| 773 | Brownie            | BRO0086       | Brownie Mini sabor Vainilla relleno de Crema pastelera y cubierto de Chocolate blanco             |              2749 |          1764.2   | Delicias Artesanas   |
| 774 | Tarta              | TAR0094       | Tarta Extra grande sabor Plátano relleno de Crema de frutilla y cubierto de Merengue              |              5106 |          3726.33  | Dolce Gusto S.R.L.   |
| 775 | Cupcake            | CUP0079       | Cupcake Extra grande sabor Mora relleno de Crema de chocolate y cubierto de Almendras             |              9284 |          4746.97  | Delicias Artesanas   |
| 776 | Queque             | QUE0095       | Queque Grande sabor Pistacho relleno de Crema de frutilla y cubierto de Mani                      |              1085 |           798.384 | Pan y Más            |
| 777 | Brownie            | BRO0087       | Brownie Mini sabor Caramelo relleno de Crema de frutilla y cubierto de Crema                      |              3275 |          1775.19  | Dolce Gusto S.R.L.   |
| 778 | Cupcake            | CUP0080       | Cupcake Grande sabor Pera relleno de Crema de chocolate y cubierto de Crema                       |              1279 |           867.113 | Delicias Artesanas   |
| 779 | Rosca              | ROS0076       | Rosca Mini sabor Limon relleno de Crema de frutilla y cubierto de Chocolate negro                 |              6113 |          4534.34  | Pastelería Gourmet   |
| 780 | Galleta            | GAL0100       | Galleta Extra grande sabor Mango relleno de Manjar y cubierto de Chocolate negro                  |              2724 |          1627.97  | Dolce Gusto S.R.L.   |
| 781 | Dona               | DON0094       | Dona Individual sabor Pistacho relleno de Manjar y cubierto de Chocolate blanco                   |              6262 |          3862.12  | Repostería Selecta   |
| 782 | Rosca              | ROS0077       | Rosca Mini sabor Mora relleno de Crema de frutilla y cubierto de Merengue                         |              2356 |          1493.13  | Panadería Fina & Co. |
| 783 | Brownie            | BRO0088       | Brownie Individual sabor Vainilla relleno de Dulce de leche y cubierto de Merengue                |              2918 |          1972.8   | Panadería Fina & Co. |
| 784 | Brownie            | BRO0089       | Brownie Grande sabor Pera relleno de Dulce de leche y cubierto de Mani                            |              6071 |          4069.37  | Delicias Artesanas   |
| 785 | Queque             | QUE0096       | Queque Extra grande sabor Plátano relleno de Manjar y cubierto de Chocolate negro                 |              8651 |          6621.1   | Pan y Más            |
| 786 | Torta              | TOR0090       | Torta Individual sabor Mango relleno de Crema de frutilla y cubierto de Merengue                  |              6584 |          4854.08  | Dolce Gusto S.R.L.   |
| 787 | Queque             | QUE0097       | Queque Extra grande sabor Mango relleno de Crema de frutilla y cubierto de Chocolate negro        |              8679 |          5886.11  | Pastelería Gourmet   |
| 788 | Cupcake            | CUP0081       | Cupcake Individual sabor Frambuesa relleno de Crema de chocolate y cubierto de Merengue           |              5805 |          4012.83  | Delicias Artesanas   |
| 789 | Magdalena          | MAG0068       | Magdalena Mini sabor Chocolate relleno de Manjar y cubierto de Chocolate negro                    |              1414 |           777.121 | Panadería Fina & Co. |
| 790 | Magdalena          | MAG0069       | Magdalena Mini sabor Maracuya relleno de Crema pastelera y cubierto de Crema                      |              4819 |          3557.55  | Pastelería Gourmet   |
| 791 | Magdalena          | MAG0070       | Magdalena Individual sabor Pistacho relleno de Manjar y cubierto de Mani                          |              1176 |           834.498 | Delicias Artesanas   |
| 792 | Queque             | QUE0098       | Queque Grande sabor Frambuesa relleno de Manjar y cubierto de Chocolate blanco                    |              4662 |          3436.1   | Panadería Fina & Co. |
| 793 | Magdalena          | MAG0071       | Magdalena Grande sabor Maracuya relleno de Crema de frutilla y cubierto de Chocolate negro        |              5531 |          3362.43  | Pan y Más            |
| 794 | Magdalena          | MAG0072       | Magdalena Extra grande sabor Vainilla relleno de Crema de chocolate y cubierto de Merengue        |              2652 |          2043.86  | Panadería Fina & Co. |
| 795 | Dona               | DON0095       | Dona Grande sabor Plátano relleno de Crema de frutilla y cubierto de Crema                        |              7003 |          4972.7   | Panadería Fina & Co. |
| 796 | Brownie            | BRO0090       | Brownie Mini sabor Frambuesa relleno de Manjar y cubierto de Chocolate blanco                     |              9177 |          6855.33  | Panadería Fina & Co. |
| 797 | Brownie            | BRO0091       | Brownie Individual sabor Mora relleno de Crema pastelera y cubierto de Chocolate blanco           |              9280 |          6036.58  | Pan y Más            |
| 798 | Queque             | QUE0099       | Queque Individual sabor Mango relleno de Crema de frutilla y cubierto de Mani                     |              2882 |          2087.12  | Pastelería Gourmet   |
| 799 | Rosca              | ROS0078       | Rosca Individual sabor Frambuesa relleno de Dulce de leche y cubierto de Chocolate negro          |              1915 |          1321.82  | Pastelería Gourmet   |
| 800 | Queque             | QUE0100       | Queque Extra grande sabor Plátano relleno de Crema pastelera y cubierto de Almendras              |              6991 |          4344.19  | Panadería Fina & Co. |
| 801 | Galleta            | GAL0101       | Galleta Grande sabor Maracuya relleno de Manjar y cubierto de Almendras                           |              4701 |          2358.92  | Pan y Más            |
| 802 | Galleta            | GAL0102       | Galleta Extra grande sabor Mango relleno de Dulce de leche y cubierto de Chocolate blanco         |              4724 |          3364.47  | Delicias Artesanas   |
| 803 | Tarta              | TAR0095       | Tarta Extra grande sabor Vainilla relleno de Manjar y cubierto de Mani                            |              8066 |          6119.01  | Dolce Gusto S.R.L.   |
| 804 | Tarta              | TAR0096       | Tarta Grande sabor Maracuya relleno de Crema de chocolate y cubierto de Chocolate blanco          |              6963 |          4084.15  | Pan y Más            |
| 805 | Torta              | TOR0091       | Torta Grande sabor Maracuya relleno de Dulce de leche y cubierto de Merengue                      |              8167 |          5091.8   | Pastelería Gourmet   |
| 806 | Magdalena          | MAG0073       | Magdalena Individual sabor Chocolate relleno de Crema de chocolate y cubierto de Merengue         |              8494 |          5575.17  | Dolce Gusto S.R.L.   |
| 807 | Magdalena          | MAG0074       | Magdalena Grande sabor Nuez relleno de Dulce de leche y cubierto de Crema                         |              6056 |          3978.71  | Repostería Selecta   |
| 808 | Queque             | QUE0101       | Queque Grande sabor Frambuesa relleno de Crema pastelera y cubierto de Chocolate blanco           |              6906 |          4080.65  | Repostería Selecta   |
| 809 | Dona               | DON0096       | Dona Individual sabor Coco relleno de Crema de chocolate y cubierto de Crema                      |              3220 |          2293.1   | Dolce Gusto S.R.L.   |
| 810 | Queque             | QUE0102       | Queque Mini sabor Mora relleno de Manjar y cubierto de Merengue                                   |              7659 |          5764.93  | Repostería Selecta   |
| 811 | Brownie            | BRO0092       | Brownie Grande sabor Plátano relleno de Manjar y cubierto de Chocolate blanco                     |              4695 |          2572.41  | Repostería Selecta   |
| 812 | Queque             | QUE0103       | Queque Mini sabor Pera relleno de Crema de frutilla y cubierto de Crema                           |              5539 |          3885.19  | Delicias Artesanas   |
| 813 | Brownie            | BRO0093       | Brownie Mini sabor Frambuesa relleno de Crema de frutilla y cubierto de Crema                     |              6383 |          3475.32  | Repostería Selecta   |
| 814 | Tarta              | TAR0097       | Tarta Mini sabor Plátano relleno de Crema de chocolate y cubierto de Mani                         |              6065 |          3671.87  | Pan y Más            |
| 815 | Tarta              | TAR0098       | Tarta Mini sabor Pistacho relleno de Crema de chocolate y cubierto de Crema                       |              1841 |          1424.31  | Repostería Selecta   |
| 816 | Torta              | TOR0092       | Torta Grande sabor Frutilla relleno de Dulce de leche y cubierto de Merengue                      |              6280 |          4873.54  | Repostería Selecta   |
| 817 | Galleta            | GAL0103       | Galleta Mini sabor Coco relleno de Crema pastelera y cubierto de Mani                             |              4212 |          2345.77  | Dolce Gusto S.R.L.   |
| 818 | Cupcake            | CUP0082       | Cupcake Grande sabor Nuez relleno de Dulce de leche y cubierto de Chocolate blanco                |              9707 |          7676.78  | Panadería Fina & Co. |
| 819 | Cupcake            | CUP0083       | Cupcake Extra grande sabor Mango relleno de Dulce de leche y cubierto de Crema                    |              6176 |          4405.16  | Repostería Selecta   |
| 820 | Queque             | QUE0104       | Queque Individual sabor Vainilla relleno de Dulce de leche y cubierto de Almendras                |              4026 |          3047.34  | Pastelería Gourmet   |
| 821 | Brownie            | BRO0094       | Brownie Individual sabor Pera relleno de Crema de chocolate y cubierto de Merengue                |              2092 |          1516.74  | Panadería Fina & Co. |
| 822 | Cupcake            | CUP0084       | Cupcake Extra grande sabor Café relleno de Crema pastelera y cubierto de Chocolate negro          |              5627 |          4385.89  | Repostería Selecta   |
| 823 | Magdalena          | MAG0075       | Magdalena Mini sabor Nuez relleno de Crema de chocolate y cubierto de Chocolate blanco            |              2511 |          1960     | Delicias Artesanas   |
| 824 | Cupcake            | CUP0085       | Cupcake Mini sabor Chocolate relleno de Crema pastelera y cubierto de Mani                        |              8637 |          6762.16  | Pastelería Gourmet   |
| 825 | Magdalena          | MAG0076       | Magdalena Extra grande sabor Mora relleno de Crema pastelera y cubierto de Mani                   |              7087 |          4648.24  | Pastelería Gourmet   |
| 826 | Rosca              | ROS0079       | Rosca Individual sabor Caramelo relleno de Crema pastelera y cubierto de Almendras                |              6099 |          3666.41  | Pastelería Gourmet   |
| 827 | Tarta              | TAR0099       | Tarta Grande sabor Coco relleno de Manjar y cubierto de Almendras                                 |              5621 |          3629.57  | Panadería Fina & Co. |
| 828 | Galleta            | GAL0104       | Galleta Grande sabor Mora relleno de Dulce de leche y cubierto de Crema                           |              4921 |          3915.1   | Delicias Artesanas   |
| 829 | Brownie            | BRO0095       | Brownie Grande sabor Pistacho relleno de Crema de chocolate y cubierto de Almendras               |              6630 |          3361.17  | Panadería Fina & Co. |
| 830 | Galleta            | GAL0105       | Galleta Individual sabor Café relleno de Crema de chocolate y cubierto de Crema                   |              2266 |          1343.42  | Delicias Artesanas   |
| 831 | Dona               | DON0097       | Dona Extra grande sabor Plátano relleno de Dulce de leche y cubierto de Chocolate blanco          |              5083 |          2916.01  | Repostería Selecta   |
| 832 | Queque             | QUE0105       | Queque Individual sabor Maracuya relleno de Crema de chocolate y cubierto de Chocolate negro      |              1618 |           985.34  | Pan y Más            |
| 833 | Brownie            | BRO0096       | Brownie Mini sabor Caramelo relleno de Crema pastelera y cubierto de Almendras                    |              8700 |          6227.94  | Delicias Artesanas   |
| 834 | Torta              | TOR0093       | Torta Mini sabor Pera relleno de Crema de chocolate y cubierto de Merengue                        |              1279 |           791.385 | Panadería Fina & Co. |
| 835 | Queque             | QUE0106       | Queque Grande sabor Caramelo relleno de Crema pastelera y cubierto de Almendras                   |              2011 |          1211.52  | Dolce Gusto S.R.L.   |
| 836 | Rosca              | ROS0080       | Rosca Mini sabor Pistacho relleno de Crema pastelera y cubierto de Mani                           |              3507 |          2728.5   | Panadería Fina & Co. |
| 837 | Dona               | DON0098       | Dona Extra grande sabor Pera relleno de Manjar y cubierto de Mani                                 |              4126 |          2331.14  | Pastelería Gourmet   |
| 838 | Magdalena          | MAG0077       | Magdalena Extra grande sabor Canela relleno de Crema de chocolate y cubierto de Nuez              |              8212 |          4489.36  | Panadería Fina & Co. |
| 839 | Galleta            | GAL0106       | Galleta Individual sabor Mora relleno de Crema pastelera y cubierto de Chocolate negro            |              6140 |          3529.11  | Pan y Más            |
| 840 | Dona               | DON0099       | Dona Mini sabor Mora relleno de Crema de chocolate y cubierto de Nuez                             |              9659 |          7607.95  | Dolce Gusto S.R.L.   |
| 841 | Cupcake            | CUP0086       | Cupcake Mini sabor Pistacho relleno de Crema pastelera y cubierto de Chocolate blanco             |              9669 |          5279.09  | Pastelería Gourmet   |
| 842 | Tarta              | TAR0100       | Tarta Grande sabor Caramelo relleno de Crema de chocolate y cubierto de Mani                      |              7044 |          5090.71  | Pan y Más            |
| 843 | Cupcake            | CUP0087       | Cupcake Extra grande sabor Frambuesa relleno de Crema pastelera y cubierto de Crema               |              6356 |          4755.74  | Pastelería Gourmet   |
| 844 | Dona               | DON0100       | Dona Individual sabor Limon relleno de Crema de frutilla y cubierto de Crema                      |              3064 |          2396.08  | Panadería Fina & Co. |
| 845 | Brownie            | BRO0097       | Brownie Extra grande sabor Frutilla relleno de Manjar y cubierto de Nuez                          |              9521 |          6040.77  | Delicias Artesanas   |
| 846 | Cupcake            | CUP0088       | Cupcake Individual sabor Pistacho relleno de Manjar y cubierto de Merengue                        |              2598 |          1839.8   | Pan y Más            |
| 847 | Tarta              | TAR0101       | Tarta Individual sabor Maracuya relleno de Dulce de leche y cubierto de Mani                      |              4345 |          2337.71  | Repostería Selecta   |
| 848 | Brownie            | BRO0098       | Brownie Individual sabor Mora relleno de Crema pastelera y cubierto de Almendras                  |              2643 |          1579.51  | Pan y Más            |
| 849 | Queque             | QUE0107       | Queque Extra grande sabor Frutilla relleno de Crema pastelera y cubierto de Mani                  |              4720 |          3711.5   | Pan y Más            |
| 850 | Cupcake            | CUP0089       | Cupcake Mini sabor Vainilla relleno de Dulce de leche y cubierto de Merengue                      |              6195 |          4518.35  | Repostería Selecta   |
| 851 | Cupcake            | CUP0090       | Cupcake Extra grande sabor Frambuesa relleno de Crema pastelera y cubierto de Nuez                |              4666 |          3639.89  | Panadería Fina & Co. |
| 852 | Queque             | QUE0108       | Queque Mini sabor Pera relleno de Crema de frutilla y cubierto de Nuez                            |              6658 |          4859.8   | Pan y Más            |
| 853 | Rosca              | ROS0081       | Rosca Grande sabor Plátano relleno de Crema de chocolate y cubierto de Merengue                   |              8997 |          5686.72  | Pastelería Gourmet   |
| 854 | Cupcake            | CUP0091       | Cupcake Extra grande sabor Limon relleno de Manjar y cubierto de Mani                             |              7781 |          4333.79  | Pastelería Gourmet   |
| 855 | Galleta            | GAL0107       | Galleta Individual sabor Pistacho relleno de Crema de frutilla y cubierto de Nuez                 |              2124 |          1213.18  | Delicias Artesanas   |
| 856 | Tarta              | TAR0102       | Tarta Mini sabor Limon relleno de Dulce de leche y cubierto de Merengue                           |              9614 |          6105.88  | Pastelería Gourmet   |
| 857 | Queque             | QUE0109       | Queque Grande sabor Mango relleno de Crema pastelera y cubierto de Chocolate negro                |              1859 |          1426.64  | Pastelería Gourmet   |
| 858 | Rosca              | ROS0082       | Rosca Mini sabor Coco relleno de Crema pastelera y cubierto de Mani                               |              1345 |          1073.64  | Dolce Gusto S.R.L.   |
| 859 | Cupcake            | CUP0092       | Cupcake Extra grande sabor Mora relleno de Crema de chocolate y cubierto de Mani                  |              5766 |          3597.96  | Panadería Fina & Co. |
| 860 | Dona               | DON0101       | Dona Grande sabor Frambuesa relleno de Crema de frutilla y cubierto de Merengue                   |              8391 |          5086.93  | Pan y Más            |
| 861 | Magdalena          | MAG0078       | Magdalena Grande sabor Vainilla relleno de Manjar y cubierto de Chocolate negro                   |              8551 |          5667.3   | Pan y Más            |
| 862 | Torta              | TOR0094       | Torta Extra grande sabor Canela relleno de Dulce de leche y cubierto de Chocolate blanco          |              6896 |          3626.63  | Delicias Artesanas   |
| 863 | Rosca              | ROS0083       | Rosca Mini sabor Pistacho relleno de Crema de frutilla y cubierto de Chocolate negro              |              2419 |          1894.08  | Dolce Gusto S.R.L.   |
| 864 | Galleta            | GAL0108       | Galleta Individual sabor Limon relleno de Crema de chocolate y cubierto de Mani                   |              9425 |          7174.11  | Pastelería Gourmet   |
| 865 | Galleta            | GAL0109       | Galleta Individual sabor Mora relleno de Manjar y cubierto de Chocolate blanco                    |              4805 |          2755.92  | Pastelería Gourmet   |
| 866 | Cupcake            | CUP0093       | Cupcake Individual sabor Chocolate relleno de Crema de frutilla y cubierto de Nuez                |              8105 |          4112.69  | Repostería Selecta   |
| 867 | Cupcake            | CUP0094       | Cupcake Mini sabor Frambuesa relleno de Manjar y cubierto de Almendras                            |              9285 |          6715.35  | Pan y Más            |
| 868 | Galleta            | GAL0110       | Galleta Grande sabor Pistacho relleno de Dulce de leche y cubierto de Nuez                        |              6584 |          3345.43  | Delicias Artesanas   |
| 869 | Galleta            | GAL0111       | Galleta Individual sabor Canela relleno de Manjar y cubierto de Chocolate negro                   |              4374 |          3267.95  | Repostería Selecta   |
| 870 | Magdalena          | MAG0079       | Magdalena Grande sabor Frutilla relleno de Dulce de leche y cubierto de Almendras                 |              7994 |          5843.72  | Pastelería Gourmet   |
| 871 | Rosca              | ROS0084       | Rosca Extra grande sabor Pistacho relleno de Dulce de leche y cubierto de Chocolate negro         |              4680 |          2377.95  | Dolce Gusto S.R.L.   |
| 872 | Magdalena          | MAG0080       | Magdalena Grande sabor Frutilla relleno de Dulce de leche y cubierto de Almendras                 |              9671 |          6761.75  | Repostería Selecta   |
| 873 | Cupcake            | CUP0095       | Cupcake Extra grande sabor Frambuesa relleno de Dulce de leche y cubierto de Crema                |              7310 |          3898.81  | Repostería Selecta   |
| 874 | Magdalena          | MAG0081       | Magdalena Individual sabor Maracuya relleno de Crema de chocolate y cubierto de Mani              |              5926 |          4616.88  | Delicias Artesanas   |
| 875 | Queque             | QUE0110       | Queque Grande sabor Pera relleno de Crema de chocolate y cubierto de Almendras                    |              7850 |          4851.23  | Dolce Gusto S.R.L.   |
| 876 | Cupcake            | CUP0096       | Cupcake Grande sabor Mora relleno de Crema de chocolate y cubierto de Crema                       |              2855 |          2223.59  | Pastelería Gourmet   |
| 877 | Torta              | TOR0095       | Torta Extra grande sabor Pistacho relleno de Dulce de leche y cubierto de Mani                    |              6878 |          5160.72  | Delicias Artesanas   |
| 878 | Torta              | TOR0096       | Torta Extra grande sabor Plátano relleno de Crema pastelera y cubierto de Nuez                    |              9470 |          5953.14  | Panadería Fina & Co. |
| 879 | Queque             | QUE0111       | Queque Extra grande sabor Pera relleno de Dulce de leche y cubierto de Nuez                       |              7693 |          4405.61  | Pastelería Gourmet   |
| 880 | Brownie            | BRO0099       | Brownie Individual sabor Pistacho relleno de Dulce de leche y cubierto de Nuez                    |              3145 |          1759.17  | Dolce Gusto S.R.L.   |
| 881 | Dona               | DON0102       | Dona Extra grande sabor Mora relleno de Manjar y cubierto de Chocolate negro                      |              5264 |          3622.56  | Dolce Gusto S.R.L.   |
| 882 | Dona               | DON0103       | Dona Individual sabor Frambuesa relleno de Crema de chocolate y cubierto de Nuez                  |              3300 |          2114.67  | Pastelería Gourmet   |
| 883 | Magdalena          | MAG0082       | Magdalena Grande sabor Maracuya relleno de Crema de frutilla y cubierto de Nuez                   |              5702 |          2896.5   | Repostería Selecta   |
| 884 | Cupcake            | CUP0097       | Cupcake Individual sabor Café relleno de Crema pastelera y cubierto de Chocolate blanco           |              5256 |          3017.62  | Pan y Más            |
| 885 | Dona               | DON0104       | Dona Individual sabor Frambuesa relleno de Manjar y cubierto de Chocolate negro                   |              6698 |          3408.33  | Panadería Fina & Co. |
| 886 | Magdalena          | MAG0083       | Magdalena Mini sabor Chocolate relleno de Crema de frutilla y cubierto de Merengue                |              6618 |          3491.23  | Repostería Selecta   |
| 887 | Rosca              | ROS0085       | Rosca Grande sabor Nuez relleno de Crema de frutilla y cubierto de Chocolate blanco               |              1013 |           593.39  | Pastelería Gourmet   |
| 888 | Torta              | TOR0097       | Torta Extra grande sabor Caramelo relleno de Dulce de leche y cubierto de Merengue                |              9757 |          7592.67  | Pan y Más            |
| 889 | Queque             | QUE0112       | Queque Extra grande sabor Frambuesa relleno de Crema pastelera y cubierto de Chocolate blanco     |              2235 |          1165.34  | Pan y Más            |
| 890 | Torta              | TOR0098       | Torta Individual sabor Chocolate relleno de Crema de frutilla y cubierto de Chocolate negro       |              2757 |          1806.01  | Panadería Fina & Co. |
| 891 | Cupcake            | CUP0098       | Cupcake Grande sabor Mango relleno de Crema de frutilla y cubierto de Chocolate negro             |              3674 |          2561.75  | Pastelería Gourmet   |
| 892 | Queque             | QUE0113       | Queque Individual sabor Nuez relleno de Crema de chocolate y cubierto de Merengue                 |              4147 |          2568.33  | Delicias Artesanas   |
| 893 | Queque             | QUE0114       | Queque Individual sabor Pera relleno de Crema de frutilla y cubierto de Mani                      |              6399 |          4748.02  | Pastelería Gourmet   |
| 894 | Queque             | QUE0115       | Queque Extra grande sabor Limon relleno de Dulce de leche y cubierto de Mani                      |              3759 |          2900.14  | Panadería Fina & Co. |
| 895 | Magdalena          | MAG0084       | Magdalena Grande sabor Nuez relleno de Crema de frutilla y cubierto de Mani                       |              8873 |          5295.94  | Pastelería Gourmet   |
| 896 | Galleta            | GAL0112       | Galleta Grande sabor Café relleno de Crema pastelera y cubierto de Almendras                      |              6310 |          3875.92  | Pastelería Gourmet   |
| 897 | Magdalena          | MAG0085       | Magdalena Individual sabor Plátano relleno de Crema de frutilla y cubierto de Merengue            |              6019 |          3891.45  | Repostería Selecta   |
| 898 | Magdalena          | MAG0086       | Magdalena Mini sabor Chocolate relleno de Dulce de leche y cubierto de Nuez                       |              9045 |          4859.97  | Pan y Más            |
| 899 | Torta              | TOR0099       | Torta Mini sabor Mango relleno de Crema pastelera y cubierto de Mani                              |              5442 |          3628.98  | Panadería Fina & Co. |
| 900 | Magdalena          | MAG0087       | Magdalena Mini sabor Canela relleno de Manjar y cubierto de Chocolate blanco                      |              6018 |          3894.73  | Pan y Más            |
| 901 | Magdalena          | MAG0088       | Magdalena Individual sabor Plátano relleno de Dulce de leche y cubierto de Mani                   |              3453 |          2689.93  | Panadería Fina & Co. |
| 902 | Dona               | DON0105       | Dona Individual sabor Maracuya relleno de Crema de frutilla y cubierto de Chocolate negro         |              2985 |          1565.35  | Pan y Más            |
| 903 | Queque             | QUE0116       | Queque Individual sabor Coco relleno de Crema pastelera y cubierto de Chocolate negro             |              5739 |          3395.25  | Delicias Artesanas   |
| 904 | Queque             | QUE0117       | Queque Mini sabor Limon relleno de Crema pastelera y cubierto de Chocolate blanco                 |              1505 |           771.004 | Pan y Más            |
| 905 | Dona               | DON0106       | Dona Extra grande sabor Nuez relleno de Manjar y cubierto de Chocolate negro                      |              8431 |          6115.02  | Delicias Artesanas   |
| 906 | Magdalena          | MAG0089       | Magdalena Mini sabor Mango relleno de Dulce de leche y cubierto de Mani                           |              5691 |          3460.54  | Dolce Gusto S.R.L.   |
| 907 | Rosca              | ROS0086       | Rosca Mini sabor Mango relleno de Crema de frutilla y cubierto de Nuez                            |              4981 |          2576.65  | Delicias Artesanas   |
| 908 | Galleta            | GAL0113       | Galleta Mini sabor Mango relleno de Manjar y cubierto de Chocolate negro                          |              9886 |          5912.39  | Repostería Selecta   |
| 909 | Dona               | DON0107       | Dona Extra grande sabor Vainilla relleno de Manjar y cubierto de Chocolate negro                  |              8141 |          4656.73  | Pan y Más            |
| 910 | Dona               | DON0108       | Dona Grande sabor Nuez relleno de Crema de frutilla y cubierto de Mani                            |              3728 |          2283.43  | Pastelería Gourmet   |
| 911 | Queque             | QUE0118       | Queque Grande sabor Caramelo relleno de Crema de frutilla y cubierto de Crema                     |              2634 |          1358.84  | Repostería Selecta   |
| 912 | Brownie            | BRO0100       | Brownie Extra grande sabor Nuez relleno de Crema de frutilla y cubierto de Crema                  |              3528 |          2036.08  | Pan y Más            |
| 913 | Dona               | DON0109       | Dona Mini sabor Mango relleno de Dulce de leche y cubierto de Almendras                           |              7762 |          5028.49  | Panadería Fina & Co. |
| 914 | Torta              | TOR0100       | Torta Individual sabor Café relleno de Crema de frutilla y cubierto de Chocolate blanco           |              9285 |          5169.51  | Panadería Fina & Co. |
| 915 | Cupcake            | CUP0099       | Cupcake Individual sabor Plátano relleno de Crema pastelera y cubierto de Almendras               |              3066 |          1761.16  | Panadería Fina & Co. |
| 916 | Tarta              | TAR0103       | Tarta Extra grande sabor Canela relleno de Crema de chocolate y cubierto de Merengue              |              5606 |          3918.85  | Delicias Artesanas   |
| 917 | Cupcake            | CUP0100       | Cupcake Extra grande sabor Mango relleno de Crema de chocolate y cubierto de Almendras            |              3433 |          2716.72  | Pastelería Gourmet   |
| 918 | Tarta              | TAR0104       | Tarta Extra grande sabor Nuez relleno de Crema de frutilla y cubierto de Crema                    |              1060 |           828.511 | Dolce Gusto S.R.L.   |
| 919 | Dona               | DON0110       | Dona Extra grande sabor Vainilla relleno de Manjar y cubierto de Chocolate negro                  |              6187 |          4850.1   | Delicias Artesanas   |
| 920 | Torta              | TOR0101       | Torta Grande sabor Café relleno de Crema de chocolate y cubierto de Merengue                      |              4287 |          3030.76  | Delicias Artesanas   |
| 921 | Magdalena          | MAG0090       | Magdalena Extra grande sabor Maracuya relleno de Crema de chocolate y cubierto de Nuez            |              1463 |           967.349 | Pan y Más            |
| 922 | Brownie            | BRO0101       | Brownie Extra grande sabor Canela relleno de Dulce de leche y cubierto de Chocolate blanco        |              3605 |          1946.9   | Pastelería Gourmet   |
| 923 | Galleta            | GAL0114       | Galleta Mini sabor Canela relleno de Dulce de leche y cubierto de Crema                           |              7970 |          4323.02  | Dolce Gusto S.R.L.   |
| 924 | Queque             | QUE0119       | Queque Individual sabor Coco relleno de Crema de chocolate y cubierto de Chocolate negro          |              6801 |          3607.73  | Delicias Artesanas   |
| 925 | Cupcake            | CUP0101       | Cupcake Individual sabor Frutilla relleno de Crema de frutilla y cubierto de Nuez                 |              5622 |          4413.12  | Dolce Gusto S.R.L.   |
| 926 | Galleta            | GAL0115       | Galleta Extra grande sabor Frambuesa relleno de Crema de frutilla y cubierto de Chocolate negro   |              6742 |          4276.92  | Pastelería Gourmet   |
| 927 | Cupcake            | CUP0102       | Cupcake Individual sabor Mora relleno de Crema de frutilla y cubierto de Almendras                |              6786 |          3857.44  | Dolce Gusto S.R.L.   |
| 928 | Torta              | TOR0102       | Torta Mini sabor Frutilla relleno de Crema de chocolate y cubierto de Merengue                    |              3233 |          2316.73  | Delicias Artesanas   |
| 929 | Torta              | TOR0103       | Torta Grande sabor Frutilla relleno de Crema de chocolate y cubierto de Almendras                 |              1889 |          1108.06  | Pastelería Gourmet   |
| 930 | Magdalena          | MAG0091       | Magdalena Individual sabor Pistacho relleno de Crema de chocolate y cubierto de Almendras         |              6044 |          4014.43  | Delicias Artesanas   |
| 931 | Dona               | DON0111       | Dona Individual sabor Chocolate relleno de Crema de chocolate y cubierto de Mani                  |              3921 |          2615.34  | Repostería Selecta   |
| 932 | Magdalena          | MAG0092       | Magdalena Individual sabor Nuez relleno de Crema de chocolate y cubierto de Almendras             |              2953 |          1534.02  | Pastelería Gourmet   |
| 933 | Torta              | TOR0104       | Torta Grande sabor Limon relleno de Crema pastelera y cubierto de Chocolate blanco                |              8681 |          6313.9   | Repostería Selecta   |
| 934 | Queque             | QUE0120       | Queque Extra grande sabor Canela relleno de Crema de frutilla y cubierto de Nuez                  |              6199 |          4680     | Delicias Artesanas   |
| 935 | Magdalena          | MAG0093       | Magdalena Individual sabor Pistacho relleno de Crema de chocolate y cubierto de Chocolate blanco  |              9996 |          7424.84  | Pan y Más            |
| 936 | Queque             | QUE0121       | Queque Grande sabor Pera relleno de Crema de frutilla y cubierto de Almendras                     |              1856 |          1121.37  | Pan y Más            |
| 937 | Dona               | DON0112       | Dona Mini sabor Coco relleno de Crema de frutilla y cubierto de Chocolate negro                   |              4356 |          2241.36  | Delicias Artesanas   |
| 938 | Rosca              | ROS0087       | Rosca Individual sabor Mango relleno de Dulce de leche y cubierto de Nuez                         |              5597 |          4320.43  | Panadería Fina & Co. |
| 939 | Dona               | DON0113       | Dona Mini sabor Chocolate relleno de Crema de chocolate y cubierto de Almendras                   |              9912 |          5931.37  | Dolce Gusto S.R.L.   |
| 940 | Brownie            | BRO0102       | Brownie Individual sabor Vainilla relleno de Crema pastelera y cubierto de Chocolate blanco       |              1515 |           884.942 | Pan y Más            |
| 941 | Tarta              | TAR0105       | Tarta Grande sabor Caramelo relleno de Dulce de leche y cubierto de Nuez                          |              8635 |          4362.5   | Delicias Artesanas   |
| 942 | Torta              | TOR0105       | Torta Individual sabor Vainilla relleno de Manjar y cubierto de Crema                             |              3600 |          1817.17  | Delicias Artesanas   |
| 943 | Tarta              | TAR0106       | Tarta Extra grande sabor Maracuya relleno de Manjar y cubierto de Mani                            |              3795 |          2975.29  | Panadería Fina & Co. |
| 944 | Brownie            | BRO0103       | Brownie Mini sabor Frutilla relleno de Crema de frutilla y cubierto de Nuez                       |              6254 |          4113.75  | Panadería Fina & Co. |
| 945 | Torta              | TOR0106       | Torta Grande sabor Mora relleno de Manjar y cubierto de Chocolate blanco                          |              2728 |          1526.24  | Delicias Artesanas   |
| 946 | Magdalena          | MAG0094       | Magdalena Extra grande sabor Pera relleno de Crema pastelera y cubierto de Merengue               |              9326 |          5589.52  | Dolce Gusto S.R.L.   |
| 947 | Galleta            | GAL0116       | Galleta Mini sabor Vainilla relleno de Crema pastelera y cubierto de Chocolate negro              |              6313 |          4826.86  | Pan y Más            |
| 948 | Queque             | QUE0122       | Queque Extra grande sabor Pera relleno de Dulce de leche y cubierto de Chocolate blanco           |              7941 |          5304.83  | Delicias Artesanas   |
| 949 | Tarta              | TAR0107       | Tarta Individual sabor Coco relleno de Crema de chocolate y cubierto de Merengue                  |              2711 |          2055.01  | Repostería Selecta   |
| 950 | Tarta              | TAR0108       | Tarta Individual sabor Frambuesa relleno de Manjar y cubierto de Chocolate negro                  |              9610 |          5398.97  | Dolce Gusto S.R.L.   |
| 951 | Cupcake            | CUP0103       | Cupcake Extra grande sabor Caramelo relleno de Crema de chocolate y cubierto de Crema             |              7154 |          3768.6   | Repostería Selecta   |
| 952 | Torta              | TOR0107       | Torta Extra grande sabor Frambuesa relleno de Dulce de leche y cubierto de Crema                  |              6990 |          5576     | Dolce Gusto S.R.L.   |
| 953 | Dona               | DON0114       | Dona Extra grande sabor Vainilla relleno de Crema de frutilla y cubierto de Almendras             |              5272 |          3063     | Dolce Gusto S.R.L.   |
| 954 | Dona               | DON0115       | Dona Extra grande sabor Maracuya relleno de Crema de chocolate y cubierto de Almendras            |              5560 |          4304.86  | Panadería Fina & Co. |
| 955 | Galleta            | GAL0117       | Galleta Extra grande sabor Vainilla relleno de Crema de chocolate y cubierto de Chocolate blanco  |              8645 |          5378.02  | Pan y Más            |
| 956 | Brownie            | BRO0104       | Brownie Mini sabor Frambuesa relleno de Crema pastelera y cubierto de Almendras                   |              6227 |          3190.58  | Pastelería Gourmet   |
| 957 | Cupcake            | CUP0104       | Cupcake Grande sabor Maracuya relleno de Dulce de leche y cubierto de Nuez                        |              1024 |           672.941 | Panadería Fina & Co. |
| 958 | Dona               | DON0116       | Dona Individual sabor Mora relleno de Crema pastelera y cubierto de Nuez                          |              9285 |          6637.78  | Panadería Fina & Co. |
| 959 | Torta              | TOR0108       | Torta Individual sabor Maracuya relleno de Dulce de leche y cubierto de Chocolate blanco          |              3451 |          2391.43  | Repostería Selecta   |
| 960 | Magdalena          | MAG0095       | Magdalena Extra grande sabor Mora relleno de Manjar y cubierto de Chocolate negro                 |              9168 |          5766.71  | Pastelería Gourmet   |
| 961 | Torta              | TOR0109       | Torta Mini sabor Limon relleno de Crema pastelera y cubierto de Crema                             |              8571 |          4607.15  | Dolce Gusto S.R.L.   |
| 962 | Torta              | TOR0110       | Torta Individual sabor Canela relleno de Manjar y cubierto de Crema                               |              2664 |          1484.39  | Pan y Más            |
| 963 | Magdalena          | MAG0096       | Magdalena Individual sabor Limon relleno de Crema de frutilla y cubierto de Chocolate negro       |              7618 |          5046.24  | Repostería Selecta   |
| 964 | Tarta              | TAR0109       | Tarta Individual sabor Vainilla relleno de Dulce de leche y cubierto de Almendras                 |              6719 |          4082.97  | Delicias Artesanas   |
| 965 | Rosca              | ROS0088       | Rosca Grande sabor Pera relleno de Dulce de leche y cubierto de Mani                              |              6532 |          4239.45  | Panadería Fina & Co. |
| 966 | Brownie            | BRO0105       | Brownie Extra grande sabor Plátano relleno de Manjar y cubierto de Crema                          |              4479 |          3478     | Delicias Artesanas   |
| 967 | Magdalena          | MAG0097       | Magdalena Individual sabor Frambuesa relleno de Crema de frutilla y cubierto de Nuez              |              1941 |          1110     | Delicias Artesanas   |
| 968 | Dona               | DON0117       | Dona Individual sabor Caramelo relleno de Manjar y cubierto de Crema                              |              6779 |          5140.25  | Pastelería Gourmet   |
| 969 | Dona               | DON0118       | Dona Mini sabor Plátano relleno de Crema pastelera y cubierto de Chocolate blanco                 |              9658 |          5742.87  | Repostería Selecta   |
| 970 | Tarta              | TAR0110       | Tarta Grande sabor Limon relleno de Manjar y cubierto de Mani                                     |              2746 |          1641.7   | Dolce Gusto S.R.L.   |
| 971 | Magdalena          | MAG0098       | Magdalena Mini sabor Pistacho relleno de Crema de chocolate y cubierto de Mani                    |              4720 |          2719.3   | Dolce Gusto S.R.L.   |
| 972 | Rosca              | ROS0089       | Rosca Individual sabor Vainilla relleno de Dulce de leche y cubierto de Chocolate blanco          |              2388 |          1787.22  | Pastelería Gourmet   |
| 973 | Cupcake            | CUP0105       | Cupcake Mini sabor Frutilla relleno de Crema de chocolate y cubierto de Crema                     |              1532 |          1070.98  | Dolce Gusto S.R.L.   |
| 974 | Tarta              | TAR0111       | Tarta Individual sabor Coco relleno de Crema de frutilla y cubierto de Merengue                   |              8653 |          5896.46  | Dolce Gusto S.R.L.   |
| 975 | Cupcake            | CUP0106       | Cupcake Extra grande sabor Vainilla relleno de Crema de frutilla y cubierto de Nuez               |              5812 |          3376.44  | Repostería Selecta   |
| 976 | Magdalena          | MAG0099       | Magdalena Mini sabor Frambuesa relleno de Crema pastelera y cubierto de Crema                     |              4207 |          3360.87  | Pastelería Gourmet   |
| 977 | Torta              | TOR0111       | Torta Individual sabor Nuez relleno de Crema de frutilla y cubierto de Almendras                  |              4964 |          2613.22  | Delicias Artesanas   |
| 978 | Dona               | DON0119       | Dona Mini sabor Frutilla relleno de Manjar y cubierto de Crema                                    |              2594 |          1777.7   | Repostería Selecta   |
| 979 | Dona               | DON0120       | Dona Grande sabor Mango relleno de Dulce de leche y cubierto de Nuez                              |              9580 |          6645.7   | Delicias Artesanas   |
| 980 | Brownie            | BRO0106       | Brownie Individual sabor Caramelo relleno de Crema de frutilla y cubierto de Merengue             |              9816 |          6998.63  | Pan y Más            |
| 981 | Rosca              | ROS0090       | Rosca Individual sabor Pistacho relleno de Dulce de leche y cubierto de Chocolate blanco          |              9587 |          7131.23  | Panadería Fina & Co. |
| 982 | Queque             | QUE0123       | Queque Extra grande sabor Coco relleno de Dulce de leche y cubierto de Mani                       |              6076 |          3212.68  | Panadería Fina & Co. |
| 983 | Queque             | QUE0124       | Queque Grande sabor Chocolate relleno de Manjar y cubierto de Chocolate negro                     |              1493 |          1152.23  | Panadería Fina & Co. |
| 984 | Tarta              | TAR0112       | Tarta Mini sabor Frambuesa relleno de Crema pastelera y cubierto de Mani                          |              8892 |          6615.34  | Pastelería Gourmet   |
| 985 | Rosca              | ROS0091       | Rosca Grande sabor Nuez relleno de Dulce de leche y cubierto de Crema                             |              7803 |          5560.39  | Repostería Selecta   |
| 986 | Rosca              | ROS0092       | Rosca Extra grande sabor Frambuesa relleno de Crema pastelera y cubierto de Nuez                  |              1443 |          1147.71  | Delicias Artesanas   |
| 987 | Galleta            | GAL0118       | Galleta Extra grande sabor Nuez relleno de Crema de frutilla y cubierto de Chocolate blanco       |              4333 |          2647.61  | Repostería Selecta   |
| 988 | Rosca              | ROS0093       | Rosca Grande sabor Frutilla relleno de Manjar y cubierto de Chocolate negro                       |              1823 |          1127.28  | Pastelería Gourmet   |
| 989 | Tarta              | TAR0113       | Tarta Mini sabor Canela relleno de Crema pastelera y cubierto de Nuez                             |              3079 |          2273.25  | Delicias Artesanas   |
| 990 | Magdalena          | MAG0100       | Magdalena Individual sabor Nuez relleno de Dulce de leche y cubierto de Chocolate blanco          |              4382 |          3487.19  | Delicias Artesanas   |
| 991 | Tarta              | TAR0114       | Tarta Mini sabor Frambuesa relleno de Manjar y cubierto de Mani                                   |              7146 |          3983.8   | Delicias Artesanas   |
| 992 | Magdalena          | MAG0101       | Magdalena Extra grande sabor Café relleno de Crema de chocolate y cubierto de Crema               |              8262 |          4779.27  | Panadería Fina & Co. |
| 993 | Magdalena          | MAG0102       | Magdalena Grande sabor Vainilla relleno de Dulce de leche y cubierto de Chocolate negro           |              9187 |          6773.27  | Pastelería Gourmet   |
| 994 | Dona               | DON0121       | Dona Mini sabor Mora relleno de Crema de chocolate y cubierto de Crema                            |              8699 |          4751.99  | Pastelería Gourmet   |
| 995 | Tarta              | TAR0115       | Tarta Mini sabor Frambuesa relleno de Crema de frutilla y cubierto de Crema                       |              4583 |          3537.1   | Pan y Más            |
| 996 | Dona               | DON0122       | Dona Mini sabor Nuez relleno de Dulce de leche y cubierto de Nuez                                 |              6920 |          5471.86  | Pan y Más            |
| 997 | Rosca              | ROS0094       | Rosca Grande sabor Café relleno de Crema pastelera y cubierto de Chocolate negro                  |              6899 |          4150.72  | Pan y Más            |
| 998 | Tarta              | TAR0116       | Tarta Individual sabor Caramelo relleno de Crema pastelera y cubierto de Chocolate negro          |              4792 |          2731.52  | Repostería Selecta   |
| 999 | Tarta              | TAR0117       | Tarta Grande sabor Coco relleno de Crema de frutilla y cubierto de Almendras                      |              3778 |          2654.18  | Dolce Gusto S.R.L.   |

# Generacion de base de datos para ventas

## Leer base de datos de clientes
Esto es un paso necesario para porder asociar los clientes ya generados y de los cuales tenemos informacion, con los productos en inventario.

In [89]:
df_clientes = pd.read_excel(path_info_clientes)

In [90]:
df_productos = pd.read_excel(path_inventario)

In [91]:
# Establecer un rango realista para las fechas de compras
fecha_actual = datetime.now()

inicio_rango = fecha_actual - timedelta (days = 180)

Se genera una tabla que contenga una fecha realista para la demanda del producto, un nombre de cliente extraido de la base de datos y un tipo de producto que desee. Cabe destacar que de esta forma y usando la funcion 'random.choice' es que puede existir clientes que demanden mas de un tipo de producto, lo cual es completamente valido y cumple con una situacion realista. 

In [92]:
def generar_demanda():
  fecha_compra = fake.date_between_dates(date_start=inicio_rango, date_end=fecha_actual)
  
  # Selecciona una fila aleatoria del DataFrame de productos
  producto = df_productos.sample(1).iloc[0]
  tipo_producto = producto['Tipo de producto']
  codigo_producto = producto['ID Producto']

  cliente = df_clientes.sample(1).iloc[0]
  nombre_cliente = cliente['Nombre']
  rut = cliente['Rut']
  
  id_compra = uuid.uuid4().hex[:6]

  return {"ID Compra": id_compra, "Fecha de compra": fecha_compra, "Nombre": nombre_cliente,"Rut": rut, "Producto demandado": tipo_producto, "ID Producto": codigo_producto}

num_demanda_clientes = 1000

demanda = [generar_demanda() for _ in range (num_demanda_clientes)]

df = pd.DataFrame(demanda)

# Guardar el archivo
df.to_excel(path_demanda, index=False)

# Visualizar en markdown
display(Markdown(df.to_markdown()))

|     | ID Compra   | Fecha de compra   | Nombre                                   | Rut        | Producto demandado   | ID Producto   |
|----:|:------------|:------------------|:-----------------------------------------|:-----------|:---------------------|:--------------|
|   0 | 4811f4      | 2023-06-15        | Isabel Elsa Cáceres Valdés               | 15417214-3 | Cupcake              | CUP0097       |
|   1 | 796f1d      | 2023-11-15        | Alejandro Dennis Escobar Rojas           | 16494057-6 | Queque               | QUE0030       |
|   2 | e9ce7e      | 2023-06-10        | Jenny Contreras                          | 18682188-7 | Dona                 | DON0031       |
|   3 | 93cf82      | 2023-06-19        | Cristóbal Santos                         | 13693193-9 | Magdalena            | MAG0070       |
|   4 | dee3c8      | 2023-08-25        | Alfredo Juan Zapata Aracena              | 18723165-9 | Dona                 | DON0043       |
|   5 | 5c6a64      | 2023-06-26        | Fernando Benjamín Pérez Morales          | 18853525-3 | Brownie              | BRO0033       |
|   6 | 328d37      | 2023-08-07        | Pablo Valdés                             | 16390909-3 | Magdalena            | MAG0065       |
|   7 | 01c1f5      | 2023-09-06        | Claudio Aguirre Bahamondes               | 19287148-9 | Tarta                | TAR0080       |
|   8 | 354265      | 2023-09-17        | José Rosales                             | 17262915-5 | Dona                 | DON0093       |
|   9 | c0b488      | 2023-09-27        | Horacio Aliaga Barrera                   | 18850082-0 | Queque               | QUE0015       |
|  10 | e93a65      | 2023-08-05        | Clara Claudia González Seguel            | 16229934-4 | Brownie              | BRO0038       |
|  11 | 92f2fe      | 2023-08-15        | Mercedes Margarita Farías Vergara        | 10854778-8 | Galleta              | GAL0044       |
|  12 | e4a229      | 2023-07-06        | Zulema Yessenia Sandoval Gálvez          | 16917327-4 | Cupcake              | CUP0013       |
|  13 | 42adea      | 2023-07-14        | Carmen Anna Montenegro Reyes             | 17930086-9 | Queque               | QUE0123       |
|  14 | 3e799d      | 2023-09-26        | Nicole Iris Ovalle Miranda               | 16369557-4 | Brownie              | BRO0018       |
|  15 | 14e484      | 2023-08-01        | Tomás Brayan Varas Sánchez               | 10202443-8 | Cupcake              | CUP0056       |
|  16 | e46afe      | 2023-11-23        | Claudio Martín Machuca Rojas             | 17666644-2 | Magdalena            | MAG0033       |
|  17 | 2b4d9f      | 2023-10-02        | Ricardo Sepúlveda                        | 17078397-9 | Dona                 | DON0071       |
|  18 | 518b39      | 2023-10-05        | Manuel Olivares                          | 17556367-6 | Magdalena            | MAG0073       |
|  19 | 17d615      | 2023-10-13        | Luis Correa Carmona                      | 10667109-5 | Dona                 | DON0050       |
|  20 | 386f0f      | 2023-08-19        | Manuel Olivares                          | 17556367-6 | Galleta              | GAL0077       |
|  21 | 7e82fa      | 2023-07-06        | Víctor Alex Céspedes Cabrera             | 11278234-5 | Dona                 | DON0061       |
|  22 | bb49fa      | 2023-09-06        | Héctor Vásquez                           | 15937139-9 | Dona                 | DON0096       |
|  23 | 70a00f      | 2023-06-28        | Gloria Andrea Herrera Luengo             | 12307497-2 | Queque               | QUE0066       |
|  24 | 07d810      | 2023-07-28        | Marisol Valentina López Elgueta          | 11930224-3 | Galleta              | GAL0073       |
|  25 | da25a1      | 2023-08-04        | Zulema Yessenia Sandoval Gálvez          | 16917327-4 | Torta                | TOR0043       |
|  26 | d7229e      | 2023-11-13        | Angelina Alejandra Benítez Bernal        | 10234228-K | Tarta                | TAR0112       |
|  27 | 4ee096      | 2023-09-08        | Nicol Trinidad Tapia Morales             | 16462899-2 | Torta                | TOR0090       |
|  28 | 143e90      | 2023-07-20        | Gisselle Ana Vergara Bahamondes          | 18945471-0 | Rosca                | ROS0054       |
|  29 | 051704      | 2023-09-21        | Juan Ángel Cornejo Vergara               | 13600421-K | Dona                 | DON0112       |
|  30 | adc2d8      | 2023-11-14        | Adriana Neira                            | 19333958-7 | Dona                 | DON0002       |
|  31 | 2b9a30      | 2023-06-15        | Víctor Osvaldo Medina Flores             | 10218209-0 | Dona                 | DON0083       |
|  32 | 035706      | 2023-08-30        | Benjamín Contreras Rubio                 | 19659087-6 | Magdalena            | MAG0009       |
|  33 | 145956      | 2023-06-30        | Aída Ana Fernández Alarcón               | 19715122-7 | Brownie              | BRO0053       |
|  34 | c4fd4b      | 2023-08-11        | Tatiana Mancilla Cuevas                  | 12495331-6 | Magdalena            | MAG0093       |
|  35 | b6be7a      | 2023-09-13        | Isabel Andrade Ríos                      | 17374680-2 | Cupcake              | CUP0021       |
|  36 | b0456f      | 2023-11-01        | Delia Ximena Montes López                | 18572344-K | Queque               | QUE0095       |
|  37 | eb632b      | 2023-09-24        | Catalina Riquelme-Barraza                | 17566730-1 | Galleta              | GAL0071       |
|  38 | 10986b      | 2023-08-25        | Jorge Acosta Moreno                      | 18768673-K | Queque               | QUE0124       |
|  39 | 7d6d05      | 2023-09-19        | Elsa Carmen Molina Riquelme              | 18461872-6 | Dona                 | DON0067       |
|  40 | 35e137      | 2023-06-24        | Alejandro Bastián Jiménez Azócar         | 17375249-K | Dona                 | DON0072       |
|  41 | fcd157      | 2023-07-30        | Juan Vera                                | 14762231-6 | Tarta                | TAR0037       |
|  42 | 28cf01      | 2023-10-01        | Elías Moya Pérez                         | 10169733-9 | Dona                 | DON0020       |
|  43 | 651a27      | 2023-06-23        | Jacqueline Ruth Vera Sepúlveda           | 15029190-4 | Tarta                | TAR0032       |
|  44 | 2b8fd4      | 2023-07-21        | Luciano Torres Araya                     | 11800774-K | Brownie              | BRO0074       |
|  45 | 24d4e4      | 2023-06-03        | Valeria Marcela Bascuñán Vega            | 14885864-3 | Dona                 | DON0089       |
|  46 | 61b869      | 2023-07-23        | Tatiana Graciela Fuentealba Burgos       | 18232977-5 | Tarta                | TAR0114       |
|  47 | 9fa689      | 2023-07-21        | Benjamin Henríquez Rojas                 | 11818130-7 | Queque               | QUE0102       |
|  48 | 568a54      | 2023-10-01        | Georgina Saavedra                        | 10727068-5 | Torta                | TOR0031       |
|  49 | 8b6fd5      | 2023-06-15        | Javiera Agustina Calderón Guerrero       | 19641357-0 | Brownie              | BRO0072       |
|  50 | a9fd32      | 2023-10-05        | Fredy Felipe Flores Fuentes              | 13477970-6 | Brownie              | BRO0014       |
|  51 | 5a46df      | 2023-09-24        | Ingrid Lucía Lagos Coronado              | 12225848-1 | Cupcake              | CUP0025       |
|  52 | 0bbee7      | 2023-06-06        | Luis Aburto                              | 11725463-8 | Rosca                | ROS0025       |
|  53 | 9bf26d      | 2023-11-09        | Enzo Bernardo Barrera López              | 13976698-9 | Queque               | QUE0066       |
|  54 | 6e3a6e      | 2023-10-25        | Pedro Henríquez Sáez                     | 15951001-4 | Torta                | TOR0056       |
|  55 | 6ea5d8      | 2023-10-29        | Elizabeth Burgos                         | 11218211-6 | Brownie              | BRO0007       |
|  56 | 8ad378      | 2023-06-11        | Mario Cerda Pino                         | 14747753-2 | Cupcake              | CUP0084       |
|  57 | a5c0a5      | 2023-06-04        | Elba Jacqueline Bravo Saavedra           | 14598507-3 | Galleta              | GAL0028       |
|  58 | 0fa785      | 2023-08-04        | Daniel Juan Miranda Martínez             | 19548925-3 | Magdalena            | MAG0029       |
|  59 | 546ab3      | 2023-10-18        | Miguel Francisco Carrasco Solís          | 15117087-4 | Cupcake              | CUP0090       |
|  60 | 997d5f      | 2023-11-01        | María Mansilla                           | 18870501-4 | Queque               | QUE0094       |
|  61 | 39daa9      | 2023-07-05        | Antonia Rivera Peralta                   | 16353978-2 | Torta                | TOR0090       |
|  62 | bb255b      | 2023-11-09        | Isabel Elsa Cáceres Valdés               | 15417214-3 | Galleta              | GAL0103       |
|  63 | 2ac215      | 2023-08-11        | Bastián Lucas Retamal Aguilera           | 16936297-4 | Queque               | QUE0007       |
|  64 | ab498a      | 2023-08-14        | Gustavo Pablo Martínez San Martín        | 14264158-6 | Queque               | QUE0063       |
|  65 | 2f2c0c      | 2023-07-31        | Luis Parra                               | 12982056-4 | Cupcake              | CUP0068       |
|  66 | 249042      | 2023-07-08        | Luz María Vega Ferrada                   | 16298047-3 | Cupcake              | CUP0043       |
|  67 | abe7e5      | 2023-08-20        | Sara Nicole Romero Muñoz                 | 19953646-7 | Magdalena            | MAG0067       |
|  68 | 829de7      | 2023-08-23        | Maximiliano Sánchez                      | 15319157-5 | Brownie              | BRO0018       |
|  69 | 9fdddf      | 2023-09-06        | Maximiliano Bustos                       | 12588084-0 | Magdalena            | MAG0024       |
|  70 | 187ec4      | 2023-07-26        | Patricia Segura Ávila                    | 13786441-3 | Tarta                | TAR0028       |
|  71 | efa45b      | 2023-09-08        | Miguel León Rivas                        | 13662344-4 | Galleta              | GAL0027       |
|  72 | bafa91      | 2023-07-08        | Dylan Muñoz Matamala                     | 12977677-K | Queque               | QUE0065       |
|  73 | 7ec4bf      | 2023-09-19        | Susana Beatriz Aravena Quezada           | 14375730-8 | Magdalena            | MAG0080       |
|  74 | 379cb0      | 2023-07-26        | Jacinta Marcela Acevedo-Cuevas Gómez     | 11641240-0 | Tarta                | TAR0018       |
|  75 | 4ce7e3      | 2023-07-27        | Aníbal Roberto Candia Valenzuela         | 14491143-3 | Cupcake              | CUP0050       |
|  76 | f3669e      | 2023-10-27        | Edison Castillo Cruz                     | 14650302-4 | Torta                | TOR0084       |
|  77 | 392c53      | 2023-11-26        | Gloria Andrea Herrera Luengo             | 12307497-2 | Cupcake              | CUP0075       |
|  78 | 3753bd      | 2023-07-14        | Cristóbal Galaz                          | 11661696-7 | Magdalena            | MAG0098       |
|  79 | 0cc87b      | 2023-08-09        | Nicole Castillo Vallejos                 | 15544044-K | Torta                | TOR0011       |
|  80 | 9658eb      | 2023-07-21        | Mauricio Parraguez Morales               | 12015586-5 | Magdalena            | MAG0034       |
|  81 | a37b59      | 2023-06-26        | Gustavo Javier Gatica Silva              | 11392266-K | Magdalena            | MAG0055       |
|  82 | 96bba4      | 2023-06-02        | Daniella Silva Ramos                     | 16814144-3 | Magdalena            | MAG0050       |
|  83 | 1a0e9b      | 2023-11-09        | Génesis Carrasco Ibarra                  | 12436825-8 | Brownie              | BRO0036       |
|  84 | 4db242      | 2023-09-28        | Pascual Jorge Díaz Muñoz                 | 11375408-0 | Brownie              | BRO0039       |
|  85 | d54cbb      | 2023-10-07        | Washington Jorge Rubilar Pereira         | 16475766-3 | Magdalena            | MAG0016       |
|  86 | 609480      | 2023-06-01        | Viviana Leiva                            | 12812236-0 | Brownie              | BRO0079       |
|  87 | 54579d      | 2023-11-23        | María Mónica Rojas Castillo              | 10505831-7 | Galleta              | GAL0092       |
|  88 | fd2dac      | 2023-11-08        | Jorge Emilio Rodríguez Ayala             | 17018659-4 | Queque               | QUE0077       |
|  89 | 3f3da4      | 2023-07-18        | José Gerardo Salgado Zúñiga              | 10946061-4 | Dona                 | DON0103       |
|  90 | 4fff85      | 2023-07-16        | Angelo Marín                             | 13995820-5 | Magdalena            | MAG0030       |
|  91 | 045e5c      | 2023-06-27        | Luis Mondaca Becerra                     | 18869290-2 | Brownie              | BRO0079       |
|  92 | df8f1f      | 2023-10-14        | Patrick Inostroza                        | 19908525-2 | Dona                 | DON0039       |
|  93 | 206817      | 2023-09-11        | Segundo Mejías Olguín                    | 16454005-3 | Brownie              | BRO0102       |
|  94 | 3ebee5      | 2023-06-14        | Antonia Catalina Verdugo Cabello         | 11684977-6 | Torta                | TOR0083       |
|  95 | 288e2a      | 2023-06-21        | Gonzalo Daniel García Romero             | 11717642-2 | Dona                 | DON0112       |
|  96 | 17981d      | 2023-08-16        | Hernán Neira Henríquez                   | 14921629-9 | Queque               | QUE0032       |
|  97 | 2e6b5e      | 2023-08-17        | Georgina Saavedra                        | 10727068-5 | Torta                | TOR0063       |
|  98 | bc06ec      | 2023-07-13        | Gabriela Sepúlveda                       | 17111160-9 | Tarta                | TAR0012       |
|  99 | d519d9      | 2023-06-10        | Flor Urrutia Marchant                    | 13802835-4 | Galleta              | GAL0033       |
| 100 | a0e28e      | 2023-11-12        | Francisco Patricio Mancilla Quinteros    | 15404882-3 | Magdalena            | MAG0074       |
| 101 | 79474b      | 2023-07-16        | Luis Sebastián Miranda Miranda           | 18401988-5 | Galleta              | GAL0010       |
| 102 | 1ef8fe      | 2023-10-12        | Patricio Muñoz                           | 15477758-7 | Galleta              | GAL0114       |
| 103 | bbdfb7      | 2023-06-25        | Manuel Oyarce                            | 13549533-8 | Galleta              | GAL0014       |
| 104 | e2fcce      | 2023-06-29        | Fernanda Carolina Rebolledo Arcos        | 18426636-7 | Brownie              | BRO0077       |
| 105 | 412360      | 2023-06-04        | Néstor Aguilera                          | 16307646-8 | Torta                | TOR0048       |
| 106 | 53ee0f      | 2023-06-07        | Camila Carolina Espinosa Torres          | 14360083-5 | Galleta              | GAL0002       |
| 107 | cb99ad      | 2023-06-06        | Valeria Vania González Arancibia         | 13160972-K | Torta                | TOR0033       |
| 108 | 411ce1      | 2023-06-22        | Elsa Cecilia Bravo Celis                 | 16950266-3 | Torta                | TOR0100       |
| 109 | d7203b      | 2023-11-22        | Francisco Patricio Mancilla Quinteros    | 15404882-3 | Queque               | QUE0091       |
| 110 | a636ec      | 2023-09-17        | Karen Yáñez Castillo                     | 12996347-0 | Dona                 | DON0003       |
| 111 | 8924a1      | 2023-09-07        | Marcia Erika Morales Arancibia           | 16626844-0 | Galleta              | GAL0118       |
| 112 | c28ee1      | 2023-10-14        | Bruno González Bastías                   | 14271010-1 | Magdalena            | MAG0077       |
| 113 | a2bdd4      | 2023-09-26        | Carlos Durán Olguín                      | 17234779-K | Cupcake              | CUP0091       |
| 114 | 7ebd6d      | 2023-06-12        | Benjamin José Quiroga Alfaro             | 12002754-9 | Brownie              | BRO0096       |
| 115 | 3c11fd      | 2023-11-14        | Moisés Núñez                             | 10918277-2 | Galleta              | GAL0076       |
| 116 | 5e4b6b      | 2023-09-16        | José Rosales                             | 17262915-5 | Cupcake              | CUP0055       |
| 117 | 6a1b1f      | 2023-06-21        | Reinaldo Héctor Vergara Rojas            | 16107282-3 | Rosca                | ROS0023       |
| 118 | 0b6d37      | 2023-10-24        | Sergio Gabriel Ortiz González            | 19625039-0 | Galleta              | GAL0054       |
| 119 | 7c8d84      | 2023-10-17        | Tatiana Carolina López Correa            | 18281389-4 | Magdalena            | MAG0036       |
| 120 | 09a273      | 2023-06-08        | Gustavo Novoa                            | 17323759-K | Galleta              | GAL0089       |
| 121 | 0fda49      | 2023-06-27        | Gerónimo Medina Araya                    | 11217325-3 | Galleta              | GAL0103       |
| 122 | 7972b4      | 2023-09-30        | Juan Luis Domínguez Tapia                | 19204831-9 | Torta                | TOR0032       |
| 123 | 6f722c      | 2023-09-30        | Emilio Carreño                           | 16550225-5 | Galleta              | GAL0003       |
| 124 | 1523f8      | 2023-11-04        | Elizabeth Burgos                         | 11218211-6 | Dona                 | DON0068       |
| 125 | af0c41      | 2023-06-18        | Silvia Makarena Osses Guerrero           | 11368502-2 | Queque               | QUE0117       |
| 126 | ac0858      | 2023-10-26        | Hilda Eloísa Solís Pinto                 | 19009134-3 | Cupcake              | CUP0061       |
| 127 | c708d2      | 2023-11-11        | Sebastián Luis Aguilar León              | 18457741-K | Magdalena            | MAG0087       |
| 128 | afdf3f      | 2023-07-09        | Víctor Cruz Tapia                        | 17363696-0 | Magdalena            | MAG0053       |
| 129 | 197249      | 2023-06-29        | Luis Cisternas Celis                     | 12197013-9 | Cupcake              | CUP0007       |
| 130 | 96230d      | 2023-07-01        | Isidora Muñoz Quintana                   | 13880430-4 | Rosca                | ROS0005       |
| 131 | 654b34      | 2023-09-27        | Ricardo Sepúlveda                        | 17078397-9 | Torta                | TOR0062       |
| 132 | 01af83      | 2023-09-18        | Claudio Albornoz Araneda                 | 13461749-1 | Cupcake              | CUP0048       |
| 133 | e95df7      | 2023-09-08        | Segundo Daniel Cisternas Cruz            | 12166348-3 | Galleta              | GAL0014       |
| 134 | 2628a2      | 2023-06-14        | Cristian Kevin Vera Aguilera             | 16137322-4 | Queque               | QUE0042       |
| 135 | 001500      | 2023-09-14        | Antonio Jaime Pizarro Estay              | 16418398-4 | Magdalena            | MAG0093       |
| 136 | e69956      | 2023-07-26        | Luciana Díaz-Fuentes Sáez                | 13484419-0 | Torta                | TOR0054       |
| 137 | bc4b11      | 2023-09-12        | Jonathan Yáñez                           | 15746440-6 | Rosca                | ROS0001       |
| 138 | 65e23e      | 2023-11-16        | Sergio Medina Sepúlveda                  | 13392723-K | Queque               | QUE0004       |
| 139 | 6e4580      | 2023-07-31        | Héctor Guerrero                          | 15467629-3 | Brownie              | BRO0028       |
| 140 | db9c28      | 2023-06-14        | Oscar Castillo                           | 13996407-9 | Tarta                | TAR0051       |
| 141 | d5d777      | 2023-10-25        | Marcela Muñoz Salgado                    | 10310235-5 | Brownie              | BRO0096       |
| 142 | badd2c      | 2023-09-14        | Cristian Wladimir Estay Cornejo          | 18285928-5 | Torta                | TOR0011       |
| 143 | defd2f      | 2023-10-11        | Erika Hurtado                            | 13403990-6 | Rosca                | ROS0089       |
| 144 | 74ec5d      | 2023-09-06        | Karen López                              | 17461972-2 | Cupcake              | CUP0005       |
| 145 | dcc866      | 2023-11-06        | Osvaldo Vicente Hernández Uribe          | 10839812-4 | Tarta                | TAR0027       |
| 146 | 4580d0      | 2023-11-27        | Berta Beatriz Acevedo González           | 12312433-6 | Dona                 | DON0027       |
| 147 | 689630      | 2023-07-13        | Paola Bernardita Herrera González        | 15977741-0 | Queque               | QUE0022       |
| 148 | d51ce5      | 2023-10-08        | María Romero Fernández                   | 18782065-3 | Magdalena            | MAG0020       |
| 149 | b113bf      | 2023-08-14        | Sofía María Jara Torres                  | 12863536-3 | Rosca                | ROS0077       |
| 150 | 9dcef8      | 2023-07-20        | Lionel Gallegos                          | 14804494-5 | Dona                 | DON0061       |
| 151 | c51f70      | 2023-07-05        | Antonio Jaime Pizarro Estay              | 16418398-4 | Queque               | QUE0019       |
| 152 | 262a86      | 2023-11-19        | Rosa Romina Díaz Guerrero                | 13372470-9 | Tarta                | TAR0064       |
| 153 | 2b7b96      | 2023-11-16        | Roberto Darío Santander Cortés           | 18669242-9 | Rosca                | ROS0026       |
| 154 | 07ad93      | 2023-07-12        | Verónica Albertina Cornejo-Hurtado Gómez | 19312123-8 | Dona                 | DON0080       |
| 155 | 471b9e      | 2023-08-16        | Pedro José Ahumada Concha                | 15355786-5 | Galleta              | GAL0015       |
| 156 | a7c1a7      | 2023-09-01        | Edward Cortés Sandoval                   | 12030609-1 | Tarta                | TAR0054       |
| 157 | 7f88f2      | 2023-09-16        | Raquel Monserrat Jofré Paz               | 16052899-1 | Rosca                | ROS0030       |
| 158 | acdcef      | 2023-10-26        | Jorge Guzmán Moraga                      | 12681819-7 | Brownie              | BRO0040       |
| 159 | a9af91      | 2023-10-20        | Erika Toledo                             | 12335263-8 | Torta                | TOR0108       |
| 160 | 7bbd64      | 2023-10-07        | Miguel Francisco Carrasco Solís          | 15117087-4 | Galleta              | GAL0040       |
| 161 | 218a85      | 2023-08-30        | María Antonia Chávez Arancibia           | 13670120-7 | Rosca                | ROS0082       |
| 162 | a88d62      | 2023-09-16        | Ximena Stephanie Sánchez Fredes          | 12634759-2 | Torta                | TOR0029       |
| 163 | 2ba8ff      | 2023-07-23        | Alfonso Arias                            | 10099398-3 | Tarta                | TAR0012       |
| 164 | 24713d      | 2023-10-20        | Ernesto Medina Martínez                  | 18776970-7 | Rosca                | ROS0006       |
| 165 | 090274      | 2023-08-26        | Matilde Marcela Figueroa Paredes         | 12571572-3 | Queque               | QUE0095       |
| 166 | 6154b2      | 2023-07-31        | Francisco Tapia Gajardo                  | 15994333-5 | Queque               | QUE0025       |
| 167 | 7dab5c      | 2023-07-05        | Segundo Daniel Cisternas Cruz            | 12166348-3 | Brownie              | BRO0098       |
| 168 | a6147f      | 2023-11-03        | Sergio Héctor Andrade Villagrán          | 11975867-2 | Tarta                | TAR0086       |
| 169 | 86d485      | 2023-09-05        | Alejandro Dennis Escobar Rojas           | 16494057-6 | Cupcake              | CUP0090       |
| 170 | 453b98      | 2023-08-16        | Elsa Carmen Molina Riquelme              | 18461872-6 | Dona                 | DON0101       |
| 171 | 428717      | 2023-10-14        | Cristian José Brito Ruiz                 | 11514458-7 | Cupcake              | CUP0042       |
| 172 | 7e6f0d      | 2023-06-18        | Fiorella Ramírez Vargas                  | 12208674-1 | Rosca                | ROS0045       |
| 173 | ba1e26      | 2023-10-13        | Sofía Contreras                          | 17218317-3 | Rosca                | ROS0048       |
| 174 | f4e15b      | 2023-06-06        | Marcela Muñoz Salgado                    | 10310235-5 | Tarta                | TAR0031       |
| 175 | fef2ef      | 2023-07-29        | María Patricia Ortega Molina             | 13562982-2 | Queque               | QUE0114       |
| 176 | 72e977      | 2023-09-18        | Jeannette Viviana Mora Aedo              | 14063807-3 | Galleta              | GAL0011       |
| 177 | 1b36a3      | 2023-10-30        | Roxana Marcela Vergara Escobar           | 10201801-5 | Rosca                | ROS0065       |
| 178 | 100506      | 2023-06-17        | Katherine Alba Barahona Jara             | 19019088-6 | Dona                 | DON0048       |
| 179 | 66f346      | 2023-11-11        | María Karen Troncoso Erices              | 18251783-2 | Queque               | QUE0109       |
| 180 | 160864      | 2023-11-07        | Roberto Juan Espinoza Valenzuela         | 18290653-1 | Galleta              | GAL0070       |
| 181 | 3c85ea      | 2023-08-25        | Ximena Stephanie Sánchez Fredes          | 12634759-2 | Dona                 | DON0077       |
| 182 | 1a94bb      | 2023-09-08        | Ian José Sepúlveda Gutiérrez             | 12993717-7 | Galleta              | GAL0073       |
| 183 | 4e1720      | 2023-06-15        | Georgina Saavedra                        | 10727068-5 | Dona                 | DON0007       |
| 184 | 80dc3f      | 2023-09-12        | Jeannette Sanhueza Norambuena            | 12152763-0 | Torta                | TOR0047       |
| 185 | c036f9      | 2023-08-26        | Nibaldo José Castro Guzmán               | 12032582-2 | Galleta              | GAL0013       |
| 186 | a4fb3a      | 2023-11-15        | Carolina Díaz Muñoz                      | 11848319-4 | Cupcake              | CUP0065       |
| 187 | a8107e      | 2023-11-21        | Lorena Arenas Medel                      | 11224973-8 | Dona                 | DON0043       |
| 188 | 2c3ff0      | 2023-10-01        | Tatiana Carolina López Correa            | 18281389-4 | Dona                 | DON0006       |
| 189 | edd5ec      | 2023-10-22        | Johnny Catalán Lorca                     | 17816404-8 | Cupcake              | CUP0029       |
| 190 | 4edc39      | 2023-08-28        | Hilda Eloísa Solís Pinto                 | 19009134-3 | Galleta              | GAL0062       |
| 191 | 5af204      | 2023-07-08        | Jaime Guillermo González Lizana          | 17102902-8 | Torta                | TOR0046       |
| 192 | e10bbc      | 2023-06-09        | Rodolfo Martín Valenzuela Olivares       | 15129422-4 | Cupcake              | CUP0092       |
| 193 | b8ba5a      | 2023-10-18        | Mateo Loyola                             | 12290805-4 | Brownie              | BRO0034       |
| 194 | f3df61      | 2023-06-18        | José Gerardo Salgado Zúñiga              | 10946061-4 | Torta                | TOR0024       |
| 195 | 97c39c      | 2023-08-13        | María Candia Farías                      | 15608191-7 | Tarta                | TAR0042       |
| 196 | fa6226      | 2023-07-30        | Segundo Mejías Olguín                    | 16454005-3 | Torta                | TOR0049       |
| 197 | 2799de      | 2023-07-29        | Luz Matilda Carmona Romero               | 11607803-5 | Cupcake              | CUP0105       |
| 198 | 729647      | 2023-06-07        | Valentina Ávila Salazar                  | 13679189-7 | Rosca                | ROS0027       |
| 199 | c79f28      | 2023-10-12        | Moisés José Aguilar Reyes                | 12492238-9 | Rosca                | ROS0053       |
| 200 | 12a4e3      | 2023-11-03        | Isabel Emilia González Lagos             | 15433080-6 | Tarta                | TAR0012       |
| 201 | 549336      | 2023-10-10        | Camila Mansilla Valderrama               | 13658398-7 | Brownie              | BRO0086       |
| 202 | 68e0fc      | 2023-08-28        | Alexander Fabián Rivera Díaz             | 17087770-8 | Brownie              | BRO0022       |
| 203 | 576382      | 2023-08-18        | Dylan Muñoz Matamala                     | 12977677-K | Cupcake              | CUP0013       |
| 204 | 6e6f69      | 2023-10-21        | Leonor María Cisterna González           | 14999203-2 | Brownie              | BRO0073       |
| 205 | 71af5c      | 2023-08-21        | Sandra Rojas                             | 14645621-9 | Tarta                | TAR0094       |
| 206 | 019fb9      | 2023-06-03        | Pedro Rojas                              | 16072434-9 | Tarta                | TAR0085       |
| 207 | 8757f3      | 2023-08-23        | Sergio Medina Sepúlveda                  | 13392723-K | Queque               | QUE0107       |
| 208 | 8a7b7e      | 2023-11-06        | María Osses Huerta                       | 10202024-7 | Torta                | TOR0082       |
| 209 | 656bcf      | 2023-07-30        | Macarena Pérez Águila                    | 14918445-4 | Queque               | QUE0038       |
| 210 | 01dfff      | 2023-07-20        | María Teresa Sánchez Valencia            | 14977234-7 | Torta                | TOR0023       |
| 211 | ed4588      | 2023-08-05        | Hilda Jofré                              | 18513213-3 | Queque               | QUE0065       |
| 212 | 257ba4      | 2023-06-25        | Javiera Carvajal Cáceres                 | 17734602-0 | Queque               | QUE0021       |
| 213 | 5a39b6      | 2023-09-26        | Jorge Luis Flores Parraguez              | 16012905-K | Cupcake              | CUP0103       |
| 214 | 5d273c      | 2023-10-18        | Pedro José Ahumada Concha                | 15355786-5 | Tarta                | TAR0090       |
| 215 | f33399      | 2023-09-11        | Héctor Bastián Garrido Salas             | 16211159-K | Magdalena            | MAG0009       |
| 216 | 1cacb1      | 2023-07-17        | Pascal Flores Osorio                     | 19866986-3 | Brownie              | BRO0072       |
| 217 | b875b4      | 2023-06-27        | Rodolfo Pérez Ramírez                    | 12894047-4 | Rosca                | ROS0048       |
| 218 | ea7743      | 2023-06-28        | Valeria Marcela Bascuñán Vega            | 14885864-3 | Galleta              | GAL0001       |
| 219 | c8c1d0      | 2023-08-09        | Tomás Luis Velásquez Soto                | 11910866-7 | Galleta              | GAL0032       |
| 220 | e56062      | 2023-11-26        | César Vivanco Quintana                   | 16783840-6 | Magdalena            | MAG0012       |
| 221 | ad8ff8      | 2023-07-29        | Paula Teresa Martínez Pereira            | 11793333-9 | Tarta                | TAR0062       |
| 222 | 7bea85      | 2023-07-27        | Manuel Olivares                          | 17556367-6 | Tarta                | TAR0047       |
| 223 | a2321d      | 2023-09-28        | Alejandro Julián Hernández Álvarez       | 11735285-2 | Rosca                | ROS0006       |
| 224 | 5e1090      | 2023-11-10        | Gabriel Quezada                          | 14777118-4 | Brownie              | BRO0013       |
| 225 | 58c620      | 2023-09-21        | Nicole Mariana Cárcamo González          | 10441857-0 | Queque               | QUE0018       |
| 226 | 7ca3d7      | 2023-11-05        | Zulema Yessenia Sandoval Gálvez          | 16917327-4 | Galleta              | GAL0097       |
| 227 | 409a3a      | 2023-06-17        | Norma Jessica Valenzuela Figueroa        | 13743359-2 | Cupcake              | CUP0025       |
| 228 | d677aa      | 2023-06-13        | Elcira Araya                             | 18505255-K | Torta                | TOR0102       |
| 229 | a6ad6a      | 2023-10-31        | Alexis Arteaga Rivas                     | 18331400-1 | Dona                 | DON0111       |
| 230 | 36b6f1      | 2023-09-24        | Noah Iván Valenzuela Hermosilla          | 19598915-2 | Cupcake              | CUP0079       |
| 231 | 8ca343      | 2023-09-14        | Christian Aranda                         | 16899941-9 | Rosca                | ROS0026       |
| 232 | 9ee00b      | 2023-07-25        | Javier Silva                             | 19720890-2 | Cupcake              | CUP0031       |
| 233 | 33b688      | 2023-11-17        | Miguel Diego Cuevas Rosas                | 19586485-7 | Queque               | QUE0120       |
| 234 | a28171      | 2023-06-03        | Clara Claudia González Seguel            | 16229934-4 | Rosca                | ROS0076       |
| 235 | a25fc5      | 2023-08-17        | Natalia Fuentes Ramírez                  | 12349206-2 | Torta                | TOR0028       |
| 236 | 7163d2      | 2023-06-14        | Ana Flores                               | 16320125-2 | Brownie              | BRO0059       |
| 237 | 512217      | 2023-06-23        | Edmundo Javier Arriagada Flores          | 11837686-2 | Magdalena            | MAG0007       |
| 238 | 390ef5      | 2023-08-26        | Luis Juan Morales Saldivia               | 18480418-9 | Galleta              | GAL0115       |
| 239 | 9fb6b4      | 2023-10-20        | Patricio Oscar Valdés Carrasco           | 17310189-3 | Galleta              | GAL0091       |
| 240 | 85fe35      | 2023-09-07        | Luis Cristopher Pérez Jerez              | 11052060-1 | Torta                | TOR0010       |
| 241 | 59b341      | 2023-10-15        | Luis Garay Acuña                         | 12836581-1 | Dona                 | DON0027       |
| 242 | 133d34      | 2023-10-27        | Gaspar Jorge Sepúlveda Vásquez           | 13934218-8 | Cupcake              | CUP0096       |
| 243 | e1111b      | 2023-06-27        | Gloria María Garcés Ortega               | 18545458-9 | Galleta              | GAL0054       |
| 244 | 8ba5f0      | 2023-07-31        | Javier Silva                             | 19720890-2 | Rosca                | ROS0057       |
| 245 | 80ba59      | 2023-07-03        | José Rosales                             | 17262915-5 | Galleta              | GAL0041       |
| 246 | b080dc      | 2023-06-29        | Elena Soto                               | 19236253-9 | Cupcake              | CUP0008       |
| 247 | 75c1dd      | 2023-10-31        | Pedro Muñoz-Torres                       | 17195280-0 | Queque               | QUE0015       |
| 248 | c1cf7f      | 2023-08-19        | Edita Zúñiga Leiva                       | 18643314-K | Brownie              | BRO0084       |
| 249 | 9da730      | 2023-08-08        | Lucas Cisternas Burgos                   | 19858682-6 | Cupcake              | CUP0070       |
| 250 | 7559f4      | 2023-07-10        | Marco Ruiz                               | 11215531-K | Tarta                | TAR0002       |
| 251 | 66da44      | 2023-07-28        | Paula Kiara Aravena Mendoza              | 18960039-1 | Torta                | TOR0081       |
| 252 | 45442a      | 2023-06-27        | Cristóbal Galaz                          | 11661696-7 | Cupcake              | CUP0089       |
| 253 | 635b25      | 2023-07-11        | Eloísa Blanca Cortés Muñoz               | 14858844-4 | Tarta                | TAR0069       |
| 254 | ccc26b      | 2023-07-31        | Ernesto Medina Martínez                  | 18776970-7 | Tarta                | TAR0074       |
| 255 | 01ff68      | 2023-06-12        | Cristóbal Santos                         | 13693193-9 | Cupcake              | CUP0013       |
| 256 | ce44c8      | 2023-07-06        | Alexander José Venegas Ramírez           | 19647307-7 | Queque               | QUE0006       |
| 257 | 8b5444      | 2023-08-11        | Scarlett García                          | 12624121-0 | Magdalena            | MAG0047       |
| 258 | d768a8      | 2023-08-30        | Julieta Lilian Aravena Medina            | 19295002-9 | Magdalena            | MAG0032       |
| 259 | 58061d      | 2023-06-28        | Juan Elías Vergara Mendoza               | 19158626-2 | Dona                 | DON0042       |
| 260 | b3805a      | 2023-10-20        | Adrián Vicente Toro Estrada              | 16322904-4 | Queque               | QUE0050       |
| 261 | d09437      | 2023-11-19        | María Julieta Pino Muñoz                 | 12851711-K | Brownie              | BRO0016       |
| 262 | b964f0      | 2023-08-31        | Emilia Paulina González Parra            | 12592975-3 | Dona                 | DON0054       |
| 263 | 91644c      | 2023-07-21        | Patricio Muñoz                           | 15477758-7 | Rosca                | ROS0009       |
| 264 | 18328e      | 2023-08-02        | Manuel Francisco Ulloa León              | 12494895-9 | Cupcake              | CUP0101       |
| 265 | 3e33f4      | 2023-10-31        | Luis Gastón Morán Carrasco               | 15291725-3 | Torta                | TOR0028       |
| 266 | d621e6      | 2023-09-30        | Sergio Gabriel Ortiz González            | 19625039-0 | Rosca                | ROS0032       |
| 267 | ef3e5e      | 2023-06-14        | Claudio Jaime Díaz Donoso                | 10799922-2 | Queque               | QUE0024       |
| 268 | 468311      | 2023-07-06        | Antonia Catalina Verdugo Cabello         | 11684977-6 | Magdalena            | MAG0024       |
| 269 | 740898      | 2023-09-22        | Danilo Joaquín Oyarce Urrutia            | 18487901-0 | Torta                | TOR0011       |
| 270 | dda7c6      | 2023-06-28        | Santos Rodrigo Villanueva Cofré          | 19002152-8 | Tarta                | TAR0051       |
| 271 | c71991      | 2023-11-25        | Raúl José Ortega Villanueva              | 13325550-2 | Galleta              | GAL0053       |
| 272 | cc03e0      | 2023-08-07        | Adriana Neira                            | 19333958-7 | Cupcake              | CUP0059       |
| 273 | 221f1b      | 2023-09-16        | Cristian Luis Vera Zavala                | 14246872-6 | Torta                | TOR0029       |
| 274 | 28657a      | 2023-06-20        | Camilo Toloza                            | 14775036-8 | Torta                | TOR0066       |
| 275 | d6bad9      | 2023-10-18        | Marco Julio Rojas Fernández              | 15686865-2 | Tarta                | TAR0059       |
| 276 | 97f9bd      | 2023-06-26        | Enrique Luis Órdenes Astorga             | 10722218-9 | Galleta              | GAL0095       |
| 277 | 01e7c0      | 2023-07-10        | María Ossandón Castillo                  | 18429404-9 | Dona                 | DON0016       |
| 278 | 023c3b      | 2023-07-21        | Luis Gastón Morán Carrasco               | 15291725-3 | Torta                | TOR0085       |
| 279 | cb7fe4      | 2023-06-17        | Cristian Alberto Sáez Muñoz              | 18943313-9 | Queque               | QUE0115       |
| 280 | 192d6c      | 2023-10-10        | Paola Bernardita Herrera González        | 15977741-0 | Cupcake              | CUP0103       |
| 281 | ac5969      | 2023-11-20        | María Troncoso                           | 12409530-2 | Dona                 | DON0034       |
| 282 | 79a268      | 2023-07-20        | Mónica María Moreno Quiroz               | 11875998-4 | Dona                 | DON0119       |
| 283 | 96e925      | 2023-07-07        | Jhoan Oscar Tapia Acevedo                | 15713006-9 | Brownie              | BRO0056       |
| 284 | 8ee4a3      | 2023-10-14        | Soraya Morales Arriagada                 | 11578196-K | Galleta              | GAL0114       |
| 285 | 67b2a5      | 2023-08-23        | María Adriana Cárdenas Acevedo           | 16940836-5 | Tarta                | TAR0003       |
| 286 | 26bc0e      | 2023-11-12        | Yolanda Soto                             | 11078988-9 | Rosca                | ROS0071       |
| 287 | bb5fb9      | 2023-11-14        | Julieta Lilian Aravena Medina            | 19295002-9 | Cupcake              | CUP0098       |
| 288 | 1f5554      | 2023-06-18        | Mayte Natacha Ulloa Fuenzalida           | 14417208-8 | Cupcake              | CUP0103       |
| 289 | 7de6b0      | 2023-08-26        | Rosa Elizabeth Herrera Rojas             | 16043136-6 | Dona                 | DON0039       |
| 290 | a4708a      | 2023-09-05        | Clara Verónica Martínez Pérez            | 19890285-7 | Dona                 | DON0060       |
| 291 | 317a9a      | 2023-09-09        | Marilyn Irene Ramírez Opazo              | 15983726-1 | Brownie              | BRO0096       |
| 292 | 909022      | 2023-06-12        | Pedro Benjamín Díaz Torres               | 11287384-0 | Rosca                | ROS0034       |
| 293 | 879b32      | 2023-06-10        | Inés Alarcón Gutiérrez                   | 19036614-8 | Galleta              | GAL0064       |
| 294 | 44d73b      | 2023-06-06        | Marcelo Rubio Zúñiga                     | 12100935-7 | Magdalena            | MAG0090       |
| 295 | 0d1acd      | 2023-07-15        | Ruth Rojas                               | 12671847-6 | Queque               | QUE0065       |
| 296 | b61582      | 2023-10-23        | Aída Ana Fernández Alarcón               | 19715122-7 | Dona                 | DON0043       |
| 297 | 6ffbce      | 2023-10-17        | Bernardo Peña González                   | 18225302-4 | Brownie              | BRO0012       |
| 298 | 8da312      | 2023-11-24        | Hernán Arenas Toro                       | 13730313-0 | Tarta                | TAR0116       |
| 299 | 76a98c      | 2023-10-07        | Mauricio Sandoval                        | 15613050-3 | Queque               | QUE0003       |
| 300 | 476c53      | 2023-09-02        | Eduardo Tobar                            | 12343657-0 | Queque               | QUE0021       |
| 301 | 84a771      | 2023-08-06        | Daniel Juan Miranda Martínez             | 19548925-3 | Dona                 | DON0006       |
| 302 | 53443d      | 2023-07-24        | Humberto Luis Galleguillos Silva         | 14333065-6 | Dona                 | DON0060       |
| 303 | 107977      | 2023-07-24        | Sebastián Francisco Ávila López          | 13161078-1 | Cupcake              | CUP0028       |
| 304 | 449ef8      | 2023-08-16        | Guillermo Armijo                         | 13717361-2 | Tarta                | TAR0080       |
| 305 | 42c566      | 2023-09-18        | Edison Castillo Cruz                     | 14650302-4 | Tarta                | TAR0104       |
| 306 | 0da8d4      | 2023-09-10        | Katherine Villegas Plaza                 | 16023174-8 | Tarta                | TAR0029       |
| 307 | 9dfe83      | 2023-11-19        | María Varas Luna                         | 16677355-2 | Torta                | TOR0067       |
| 308 | 5983c7      | 2023-09-12        | María Antonia Chávez Arancibia           | 13670120-7 | Dona                 | DON0071       |
| 309 | 19e3f2      | 2023-07-15        | Noemí Martina Navarro-Olivares Rivera    | 12364781-4 | Magdalena            | MAG0052       |
| 310 | be34fc      | 2023-07-24        | Miguel Tomás Gallardo Canales            | 11302001-4 | Dona                 | DON0018       |
| 311 | 4fccea      | 2023-10-03        | Ricardo Araya Angulo                     | 12284399-2 | Dona                 | DON0021       |
| 312 | 7e1eca      | 2023-09-05        | Claudio Aguirre Bahamondes               | 19287148-9 | Torta                | TOR0051       |
| 313 | c4f40b      | 2023-10-31        | Fabiana Laura Espinoza Rivas             | 10226842-1 | Brownie              | BRO0091       |
| 314 | 828d7e      | 2023-07-03        | Mónica Cortés Sáez                       | 17793089-2 | Cupcake              | CUP0009       |
| 315 | 239a2b      | 2023-07-28        | Mireya Romina Hidalgo Candia             | 17814777-7 | Queque               | QUE0023       |
| 316 | b48882      | 2023-09-08        | Javiera María Salinas Hernández          | 17228097-3 | Queque               | QUE0050       |
| 317 | fd4fa3      | 2023-09-13        | Cristian Alberto Sáez Muñoz              | 18943313-9 | Tarta                | TAR0072       |
| 318 | b8e4eb      | 2023-09-30        | María Adriana Cárdenas Acevedo           | 16940836-5 | Magdalena            | MAG0072       |
| 319 | 07911b      | 2023-08-22        | Irene Ibáñez Acuña                       | 16381467-3 | Magdalena            | MAG0015       |
| 320 | fc8672      | 2023-11-21        | Juan Adán Pérez Saavedra                 | 10141662-2 | Galleta              | GAL0080       |
| 321 | e93b73      | 2023-08-31        | Joaquín Barrera                          | 15956314-6 | Cupcake              | CUP0023       |
| 322 | 4e05b1      | 2023-11-13        | Luis Cid Verdugo                         | 18353524-1 | Magdalena            | MAG0044       |
| 323 | f3ce3b      | 2023-09-28        | Isabel Emilia González Lagos             | 15433080-6 | Galleta              | GAL0030       |
| 324 | bba870      | 2023-08-22        | Thomas Ian Aguilera Cáceres              | 16524128-3 | Magdalena            | MAG0040       |
| 325 | 27e66d      | 2023-07-19        | Matías Josue Cortés Mora                 | 19600256-0 | Torta                | TOR0048       |
| 326 | a446e7      | 2023-09-08        | Patricia Eugenia Concha González         | 17490750-0 | Dona                 | DON0088       |
| 327 | 84463c      | 2023-06-14        | Jacqueline Laura Peña Cortés             | 17395854-0 | Brownie              | BRO0024       |
| 328 | 3601ff      | 2023-07-31        | Jorge Guzmán Moraga                      | 12681819-7 | Magdalena            | MAG0070       |
| 329 | 8c7860      | 2023-08-21        | Héctor Bastián Garrido Salas             | 16211159-K | Tarta                | TAR0048       |
| 330 | d3bc0e      | 2023-11-20        | Sergio Aguilar Gómez                     | 17393020-8 | Dona                 | DON0090       |
| 331 | 5223c3      | 2023-08-08        | Martha Pamela Orellana Reyes             | 19721578-K | Torta                | TOR0109       |
| 332 | 2b1bfa      | 2023-09-08        | María Karen Troncoso Erices              | 18251783-2 | Rosca                | ROS0024       |
| 333 | 7f0598      | 2023-10-06        | Hugo Quispe Ceballos                     | 10800929-9 | Magdalena            | MAG0010       |
| 334 | da1b18      | 2023-11-25        | Eduardo Vidal Sepúlveda                  | 10309039-3 | Tarta                | TAR0067       |
| 335 | c75e27      | 2023-11-20        | Víctor Diego González Segovia            | 15442719-5 | Dona                 | DON0008       |
| 336 | b616cf      | 2023-07-19        | Héctor Guerrero                          | 15467629-3 | Queque               | QUE0110       |
| 337 | 0c4cdd      | 2023-11-15        | Irma María Tapia Gutiérrez               | 12222431-5 | Tarta                | TAR0094       |
| 338 | 5cf7cd      | 2023-09-29        | Clara Isidora Navarrete Oyarzo           | 18839836-9 | Queque               | QUE0023       |
| 339 | bac233      | 2023-10-15        | César Aguilera Sánchez                   | 13108674-2 | Brownie              | BRO0064       |
| 340 | cab5d4      | 2023-08-30        | Mauricio Parraguez Morales               | 12015586-5 | Cupcake              | CUP0046       |
| 341 | 0dc603      | 2023-06-24        | Martina Rivera                           | 17703009-8 | Galleta              | GAL0058       |
| 342 | 7c83ab      | 2023-06-03        | Alma Álvarez                             | 12661877-5 | Queque               | QUE0035       |
| 343 | 31e978      | 2023-06-07        | Viviana Valenzuela Rodríguez             | 13901121-2 | Queque               | QUE0105       |
| 344 | fef792      | 2023-09-11        | Cecilia Salas Contreras                  | 17394359-8 | Torta                | TOR0036       |
| 345 | d75fb6      | 2023-07-02        | Cristofer Salinas                        | 13343669-6 | Queque               | QUE0080       |
| 346 | 617609      | 2023-10-01        | Vania Clara Carreño Bustos               | 14745890-K | Torta                | TOR0003       |
| 347 | 0a9588      | 2023-11-20        | Juan Luis Domínguez Tapia                | 19204831-9 | Galleta              | GAL0013       |
| 348 | 8390a4      | 2023-08-15        | Violeta Victoria Barra Troncoso          | 18900459-K | Torta                | TOR0096       |
| 349 | 66165f      | 2023-08-26        | Yesica Emilia Soto González              | 11235747-6 | Galleta              | GAL0091       |
| 350 | a68ef9      | 2023-09-07        | Guillermo Armijo                         | 13717361-2 | Dona                 | DON0058       |
| 351 | e6bfb6      | 2023-09-24        | Patricia Leslie Solís Retamal            | 16319007-8 | Torta                | TOR0108       |
| 352 | 0d97d9      | 2023-09-22        | Juan Fuentes                             | 18647257-7 | Rosca                | ROS0066       |
| 353 | c2f3d8      | 2023-07-16        | María Margarita Morales Díaz             | 13040753-9 | Queque               | QUE0113       |
| 354 | 394c98      | 2023-10-09        | Julio Troncoso Álvarez                   | 18127839-K | Galleta              | GAL0032       |
| 355 | 885b7a      | 2023-09-22        | Patricio Oscar Valdés Carrasco           | 17310189-3 | Tarta                | TAR0047       |
| 356 | 3deedc      | 2023-10-25        | José González Fuentes                    | 18046339-1 | Magdalena            | MAG0089       |
| 357 | e83cfb      | 2023-07-25        | Norma Valeska Martínez Paredes           | 17878773-8 | Dona                 | DON0075       |
| 358 | 1e7562      | 2023-08-11        | Luciano José Vera Villegas               | 10930426-3 | Dona                 | DON0059       |
| 359 | f26081      | 2023-10-01        | Marcia Erika Morales Arancibia           | 16626844-0 | Magdalena            | MAG0011       |
| 360 | 55d78e      | 2023-08-04        | Tomás Luis Velásquez Soto                | 11910866-7 | Cupcake              | CUP0029       |
| 361 | 13ec32      | 2023-06-13        | Bernardita Isabel Espinoza Pereira       | 14558019-1 | Rosca                | ROS0041       |
| 362 | 542aa4      | 2023-09-12        | Isabel Andrade Ríos                      | 17374680-2 | Tarta                | TAR0116       |
| 363 | d9e526      | 2023-08-24        | Sebastián Hernán Molina Campos           | 17929121-9 | Dona                 | DON0040       |
| 364 | 12d6b4      | 2023-08-09        | Enzo Bernardo Barrera López              | 13976698-9 | Cupcake              | CUP0069       |
| 365 | dbc7c9      | 2023-07-06        | Sonia Ulloa-Araya López                  | 13948315-5 | Galleta              | GAL0113       |
| 366 | 177815      | 2023-08-20        | Flor Urrutia Marchant                    | 13802835-4 | Galleta              | GAL0031       |
| 367 | 2595aa      | 2023-09-19        | Mateo Campos Cuevas                      | 13032840-8 | Cupcake              | CUP0044       |
| 368 | 579e99      | 2023-10-26        | Pilar Cortés González                    | 18701866-4 | Torta                | TOR0078       |
| 369 | 3e6ee0      | 2023-09-07        | Juana Videla Gálvez                      | 15096852-1 | Rosca                | ROS0044       |
| 370 | 73e586      | 2023-11-08        | Juan Gutiérrez Navarro                   | 15468715-3 | Magdalena            | MAG0077       |
| 371 | af3550      | 2023-07-16        | Carlos Víctor Cubillos Bravo             | 15902765-6 | Rosca                | ROS0089       |
| 372 | 6a2bd6      | 2023-09-09        | Claudia Pérez Pavez                      | 17961642-9 | Brownie              | BRO0036       |
| 373 | 588eab      | 2023-07-07        | Claudio Sebastián Pinto Valdés           | 14847100-6 | Cupcake              | CUP0084       |
| 374 | 7ea6e7      | 2023-11-22        | Alberto Ortega Sepúlveda                 | 13618898-3 | Cupcake              | CUP0079       |
| 375 | 2b344f      | 2023-08-30        | Sandra Ximena Matus Baeza                | 11131094-7 | Dona                 | DON0095       |
| 376 | fd6205      | 2023-11-27        | Raúl Urbina                              | 15616810-3 | Cupcake              | CUP0013       |
| 377 | b07181      | 2023-08-07        | Paula Antonia Guerrero Gallegos          | 10141898-4 | Rosca                | ROS0044       |
| 378 | a81406      | 2023-06-30        | Francisca Cárdenas Cerda                 | 16249434-7 | Queque               | QUE0013       |
| 379 | 93523d      | 2023-10-31        | Sergio Héctor Andrade Villagrán          | 11975867-2 | Magdalena            | MAG0075       |
| 380 | f668e7      | 2023-11-03        | Anahí Reyes                              | 10678640-6 | Tarta                | TAR0060       |
| 381 | f5c3a0      | 2023-08-21        | Jocelyn Eliana Vera Acevedo              | 15512192-9 | Torta                | TOR0014       |
| 382 | b0fcff      | 2023-11-27        | Cindy María Arce Muñoz                   | 14464461-8 | Queque               | QUE0053       |
| 383 | dca08e      | 2023-10-11        | Marta Sandra Suárez Torres               | 14806923-5 | Cupcake              | CUP0038       |
| 384 | bcb52a      | 2023-09-13        | Juan Ricardo Rubilar Santana             | 19513318-4 | Rosca                | ROS0092       |
| 385 | df5631      | 2023-06-10        | Rubén Federico Ruiz Villarroel           | 16649520-8 | Galleta              | GAL0036       |
| 386 | d264ad      | 2023-09-16        | José Gerardo Salgado Zúñiga              | 10946061-4 | Dona                 | DON0053       |
| 387 | eab7be      | 2023-09-21        | Berta Beatriz Acevedo González           | 12312433-6 | Rosca                | ROS0035       |
| 388 | 3f00ca      | 2023-07-16        | Douglas Kevin Mansilla Fuentes           | 10676755-9 | Magdalena            | MAG0093       |
| 389 | f2b65f      | 2023-07-05        | Catherine Andrea Duarte Vilches          | 19243571-8 | Cupcake              | CUP0015       |
| 390 | d15143      | 2023-10-07        | Manuel Patricio Muñoz Ceballos           | 11859023-1 | Magdalena            | MAG0014       |
| 391 | 8b0367      | 2023-08-17        | Patricio Carlos Ramírez Reyes            | 15650652-3 | Queque               | QUE0081       |
| 392 | 23b6ab      | 2023-07-08        | Katherine Julia Burgos Martínez          | 11057774-8 | Brownie              | BRO0050       |
| 393 | cb289f      | 2023-06-26        | Carolina Díaz Muñoz                      | 11848319-4 | Cupcake              | CUP0104       |
| 394 | f372ce      | 2023-09-02        | Jonathan Jorge Solar Moraga              | 19498423-6 | Magdalena            | MAG0084       |
| 395 | 43da98      | 2023-07-22        | Sergio Gabriel Ortiz González            | 19625039-0 | Brownie              | BRO0103       |
| 396 | 7698d2      | 2023-08-26        | Edith Zúñiga Jorquera                    | 16197294-0 | Tarta                | TAR0106       |
| 397 | d2e99a      | 2023-09-12        | María Riquelme Riquelme                  | 18845725-4 | Brownie              | BRO0044       |
| 398 | f5ab9c      | 2023-08-16        | Patricia Natalia Pérez Acuña             | 10296111-K | Magdalena            | MAG0063       |
| 399 | 99049b      | 2023-08-14        | Yolanda Belén Sepúlveda Cerda            | 16068155-6 | Magdalena            | MAG0034       |
| 400 | 79326b      | 2023-11-19        | Claudio Sebastián Sánchez Calderón       | 12687889-1 | Tarta                | TAR0094       |
| 401 | 4cbf73      | 2023-06-02        | Arturo Luis Guerra Seguel                | 15889140-7 | Tarta                | TAR0044       |
| 402 | 996acb      | 2023-07-19        | María Tamara Castro Núñez                | 18796712-1 | Brownie              | BRO0011       |
| 403 | a3e7c0      | 2023-08-25        | Héctor Guerrero                          | 15467629-3 | Magdalena            | MAG0065       |
| 404 | c3fa82      | 2023-06-07        | Magdalena Corvalán                       | 18714888-K | Magdalena            | MAG0082       |
| 405 | bdc32a      | 2023-07-30        | Pascuala Zúñiga Zamora                   | 12238581-2 | Magdalena            | MAG0046       |
| 406 | 2c594c      | 2023-08-26        | Mauricio Meza Órdenes                    | 15975545-4 | Brownie              | BRO0062       |
| 407 | 714c8f      | 2023-09-27        | Pedro Henríquez Sáez                     | 15951001-4 | Tarta                | TAR0044       |
| 408 | 5bfbbd      | 2023-11-11        | Wladimir Francisco Orellana Lara         | 15994304-0 | Dona                 | DON0092       |
| 409 | 6a7b71      | 2023-07-23        | María Alejandra Poblete Henríquez        | 12250458-9 | Rosca                | ROS0087       |
| 410 | 2c96af      | 2023-07-05        | Martina Calderón Espinoza                | 18950594-3 | Queque               | QUE0099       |
| 411 | 1768f2      | 2023-11-05        | Camilo Bruno Alvarado Silva              | 18661366-2 | Brownie              | BRO0106       |
| 412 | 33472d      | 2023-09-08        | Susana Beatriz Aravena Quezada           | 14375730-8 | Rosca                | ROS0087       |
| 413 | 35956f      | 2023-06-14        | Nicole Iris Ovalle Miranda               | 16369557-4 | Cupcake              | CUP0033       |
| 414 | 598b32      | 2023-10-27        | César Toro Barría                        | 15563165-6 | Dona                 | DON0048       |
| 415 | 6f2deb      | 2023-07-13        | Eduardo Vidal Sepúlveda                  | 10309039-3 | Torta                | TOR0098       |
| 416 | 9cf17d      | 2023-08-05        | Francis Figueroa Muñoz                   | 18087443-K | Queque               | QUE0066       |
| 417 | dee9ae      | 2023-11-23        | Maribel Zúñiga                           | 18764909-5 | Torta                | TOR0092       |
| 418 | e5db7c      | 2023-06-18        | Pascual López Arancibia                  | 14999479-7 | Brownie              | BRO0038       |
| 419 | 021e6c      | 2023-09-01        | Elba Silvana Valenzuela Riffo            | 11074696-8 | Rosca                | ROS0065       |
| 420 | 82f0dc      | 2023-06-09        | Florencia Paula Albornoz Quezada         | 14723845-9 | Galleta              | GAL0084       |
| 421 | fc3c65      | 2023-08-31        | Eliana Gómez                             | 19015655-5 | Cupcake              | CUP0080       |
| 422 | ebb18d      | 2023-10-27        | Exequiel Brian Loyola Gómez              | 15523986-K | Magdalena            | MAG0084       |
| 423 | 3a18d8      | 2023-06-14        | Daniela Briones                          | 14292932-6 | Torta                | TOR0022       |
| 424 | 2162f7      | 2023-07-25        | Gustavo Pablo Martínez San Martín        | 14264158-6 | Brownie              | BRO0012       |
| 425 | c16d32      | 2023-06-17        | Pedro Rojas Vásquez                      | 19159660-K | Torta                | TOR0029       |
| 426 | c549be      | 2023-09-02        | Natalia Berta García Lagos               | 14437375-K | Tarta                | TAR0006       |
| 427 | 2faf35      | 2023-11-28        | Carlos Pino Silva                        | 18487152-2 | Galleta              | GAL0079       |
| 428 | abfc76      | 2023-08-06        | Brayan Pinto Cruz                        | 13687335-K | Brownie              | BRO0079       |
| 429 | 7da11a      | 2023-07-09        | Víctor Alex Céspedes Cabrera             | 11278234-5 | Rosca                | ROS0071       |
| 430 | 9a6a62      | 2023-06-14        | Nelson Llanos Vásquez                    | 17866142-7 | Cupcake              | CUP0047       |
| 431 | 5f86bc      | 2023-10-12        | Priscilla Ayala Ahumada                  | 19951865-1 | Queque               | QUE0052       |
| 432 | 53b138      | 2023-09-15        | Pablo Valdés                             | 16390909-3 | Dona                 | DON0095       |
| 433 | 8a70a1      | 2023-08-08        | Luis Cristopher Pérez Jerez              | 11052060-1 | Torta                | TOR0025       |
| 434 | d87700      | 2023-07-16        | Eduardo Tobar                            | 12343657-0 | Magdalena            | MAG0054       |
| 435 | a7239f      | 2023-11-03        | Manuel Marco Miranda Vergara             | 10782755-2 | Magdalena            | MAG0093       |
| 436 | f480aa      | 2023-08-02        | Víctor Ricardo Rosas Figueroa            | 11923108-1 | Magdalena            | MAG0001       |
| 437 | e2c667      | 2023-07-05        | Sebastián Juan Ruiz Sandoval             | 13575260-K | Tarta                | TAR0053       |
| 438 | 35bef2      | 2023-11-18        | Rebeca Karla Rojas Navarro               | 15840899-0 | Brownie              | BRO0040       |
| 439 | 1ebc46      | 2023-08-17        | Santos Rodrigo Villanueva Cofré          | 19002152-8 | Queque               | QUE0004       |
| 440 | 74b113      | 2023-06-24        | Myriam Laura Aguilera Durán              | 18967830-2 | Dona                 | DON0117       |
| 441 | 7e24db      | 2023-10-02        | Luis Rafael Varas Sepúlveda              | 19910543-8 | Dona                 | DON0048       |
| 442 | 940346      | 2023-08-16        | Maximiliano Bustos Fuentes               | 10279475-2 | Queque               | QUE0107       |
| 443 | bd0018      | 2023-08-19        | Luisa Segura Vega                        | 12577131-3 | Queque               | QUE0034       |
| 444 | 941609      | 2023-09-22        | Catalina Vargas Concha                   | 17381718-0 | Dona                 | DON0103       |
| 445 | 456ff5      | 2023-11-02        | José Gerardo Salgado Zúñiga              | 10946061-4 | Galleta              | GAL0102       |
| 446 | 8c323c      | 2023-06-18        | Diego Pablo Rocha Cárcamo                | 13342133-9 | Galleta              | GAL0095       |
| 447 | 0960bd      | 2023-07-19        | Gloria María Garcés Ortega               | 18545458-9 | Tarta                | TAR0050       |
| 448 | 4e53c7      | 2023-09-21        | Martha Pamela Orellana Reyes             | 19721578-K | Brownie              | BRO0023       |
| 449 | 0e5458      | 2023-10-15        | Camilo Toloza                            | 14775036-8 | Magdalena            | MAG0089       |
| 450 | a07cc9      | 2023-11-16        | Magdalena Corvalán                       | 18714888-K | Cupcake              | CUP0016       |
| 451 | fa4772      | 2023-08-09        | Bárbara Isidora Díaz Olivares            | 17474941-7 | Queque               | QUE0065       |
| 452 | 9a66bd      | 2023-11-05        | Sebastián Vicente Toro Cortez            | 14528432-6 | Rosca                | ROS0012       |
| 453 | 7fa525      | 2023-10-07        | Ruth Medina Herrera                      | 10356920-1 | Brownie              | BRO0094       |
| 454 | c7ac72      | 2023-08-06        | María Patricia Ortega Molina             | 13562982-2 | Tarta                | TAR0037       |
| 455 | 2d1f98      | 2023-10-19        | Mauricio Meza Órdenes                    | 15975545-4 | Torta                | TOR0094       |
| 456 | 650b96      | 2023-07-25        | Fernando Benjamín Pérez Morales          | 18853525-3 | Brownie              | BRO0086       |
| 457 | ad669b      | 2023-11-05        | María Farías Jaramillo                   | 16479257-5 | Brownie              | BRO0023       |
| 458 | 701575      | 2023-11-24        | Diego Gustavo Gallegos Arancibia         | 12237869-2 | Brownie              | BRO0050       |
| 459 | 0045fe      | 2023-11-05        | Celia Sara Gatica Torres                 | 17785386-7 | Torta                | TOR0094       |
| 460 | fd3646      | 2023-08-13        | Javiera María Salinas Hernández          | 17228097-3 | Queque               | QUE0008       |
| 461 | 953952      | 2023-08-25        | Paz Adriana Quijada Díaz                 | 10349018-9 | Rosca                | ROS0079       |
| 462 | fddde7      | 2023-09-29        | Roberto Darío Santander Cortés           | 18669242-9 | Torta                | TOR0056       |
| 463 | f14c5d      | 2023-11-09        | Elías Antonio Urzúa Alvarado             | 19953582-6 | Torta                | TOR0046       |
| 464 | 1e13a4      | 2023-06-10        | Maximiliano Ángel Gómez Guerrero         | 12873138-4 | Magdalena            | MAG0053       |
| 465 | 76375a      | 2023-07-10        | Juan Richard Saldaña Espinoza            | 10267532-9 | Queque               | QUE0021       |
| 466 | f9f085      | 2023-09-03        | Nicole Castillo Vallejos                 | 15544044-K | Galleta              | GAL0003       |
| 467 | a866f8      | 2023-07-21        | Rosa Elizabeth Herrera Rojas             | 16043136-6 | Cupcake              | CUP0063       |
| 468 | 266ebe      | 2023-06-22        | Isabel Emilia González Lagos             | 15433080-6 | Magdalena            | MAG0009       |
| 469 | ce787b      | 2023-09-11        | Adriana Neira                            | 19333958-7 | Queque               | QUE0115       |
| 470 | 89a8b7      | 2023-07-16        | Manuel Contreras                         | 16313739-0 | Cupcake              | CUP0055       |
| 471 | 08d724      | 2023-09-09        | Cecilia Leal Paredes                     | 13826923-9 | Galleta              | GAL0089       |
| 472 | 8893f4      | 2023-07-10        | Bianca Yanet Fica Solís                  | 13892497-8 | Cupcake              | CUP0026       |
| 473 | f94294      | 2023-07-08        | Leonel Sergio Figueroa Cofré             | 13637138-5 | Tarta                | TAR0046       |
| 474 | f60e71      | 2023-10-19        | María Doris Arancibia Farías             | 14320333-3 | Cupcake              | CUP0025       |
| 475 | ce5136      | 2023-09-16        | Julio Troncoso Álvarez                   | 18127839-K | Dona                 | DON0024       |
| 476 | 0bff33      | 2023-06-28        | Martina Calderón Espinoza                | 18950594-3 | Brownie              | BRO0004       |
| 477 | 0337ae      | 2023-07-24        | Berta Jeannette Herrera Figueroa         | 12255158-0 | Tarta                | TAR0114       |
| 478 | 32a19a      | 2023-08-01        | Ismael Muñoz                             | 12352843-0 | Tarta                | TAR0063       |
| 479 | ab8aa8      | 2023-08-25        | John Roberto Aravena Hurtado             | 11823335-7 | Queque               | QUE0086       |
| 480 | 52e9b6      | 2023-06-08        | Anahí Reyes                              | 10678640-6 | Torta                | TOR0096       |
| 481 | e9f8e7      | 2023-10-11        | Rosa Pamela Lara Alarcón                 | 12413436-7 | Magdalena            | MAG0046       |
| 482 | 72df0a      | 2023-11-09        | Cristian Diego Sepúlveda Díaz            | 13236511-3 | Dona                 | DON0085       |
| 483 | 8826aa      | 2023-09-12        | Jorge Luis Flores Parraguez              | 16012905-K | Galleta              | GAL0009       |
| 484 | 530701      | 2023-10-10        | Josefa Inés Tobar Zamora                 | 16701960-3 | Rosca                | ROS0072       |
| 485 | b9cd87      | 2023-07-25        | Irma Alvarado                            | 10701104-1 | Magdalena            | MAG0053       |
| 486 | fbcf00      | 2023-10-03        | María Teresa Sánchez Valencia            | 14977234-7 | Dona                 | DON0053       |
| 487 | 20f5dc      | 2023-07-06        | Cristian Cristopher Sandoval Romero      | 16893950-K | Rosca                | ROS0093       |
| 488 | 73ddd4      | 2023-07-04        | Fernanda Gladys García Orellana          | 19231518-4 | Cupcake              | CUP0039       |
| 489 | f30561      | 2023-09-17        | Angelina Alejandra Benítez Bernal        | 10234228-K | Galleta              | GAL0107       |
| 490 | 170d1c      | 2023-07-19        | Francisco Benjamín Viveros Salinas       | 16500516-0 | Torta                | TOR0110       |
| 491 | fcfada      | 2023-08-25        | Miguel Francisco Carrasco Solís          | 15117087-4 | Torta                | TOR0098       |
| 492 | 5dd70f      | 2023-06-18        | Eduardo Tobar                            | 12343657-0 | Dona                 | DON0113       |
| 493 | e6254e      | 2023-11-02        | Rosa Romina Díaz Guerrero                | 13372470-9 | Brownie              | BRO0082       |
| 494 | 62d6bb      | 2023-06-27        | Ángela Arancibia Poblete                 | 15349562-4 | Galleta              | GAL0042       |
| 495 | d734c4      | 2023-10-03        | Cristóbal Galaz                          | 11661696-7 | Galleta              | GAL0089       |
| 496 | 0abdc4      | 2023-06-06        | Isidora Katherine Peña Arriagada         | 16190743-6 | Dona                 | DON0121       |
| 497 | 925233      | 2023-06-17        | Manuel Marco Miranda Vergara             | 10782755-2 | Dona                 | DON0036       |
| 498 | 798ae0      | 2023-10-24        | Magaly Ruiz                              | 13052363-7 | Rosca                | ROS0057       |
| 499 | d8f09d      | 2023-06-19        | Álvaro Aracena Moraga                    | 13637756-3 | Dona                 | DON0122       |
| 500 | da9c7a      | 2023-07-11        | Juan Mauricio Pizarro Sepúlveda          | 11992541-2 | Magdalena            | MAG0085       |
| 501 | f59a69      | 2023-06-19        | Berta Martina Marín Hernández            | 17284393-9 | Cupcake              | CUP0091       |
| 502 | 99426d      | 2023-07-15        | Pedro Rojas Vásquez                      | 19159660-K | Rosca                | ROS0031       |
| 503 | 6d21c6      | 2023-11-03        | Elia Patricia Quezada Moya               | 10968100-9 | Torta                | TOR0054       |
| 504 | 9e9a0b      | 2023-06-25        | Paulina Verónica Muñoz Herrera           | 11928085-6 | Torta                | TOR0082       |
| 505 | 6057d5      | 2023-07-05        | Julieta Lilian Aravena Medina            | 19295002-9 | Brownie              | BRO0076       |
| 506 | c2fe8e      | 2023-09-26        | Roberto Cristian Uribe Pérez             | 10048520-1 | Galleta              | GAL0036       |
| 507 | 805f2e      | 2023-10-21        | Jeannette Sanhueza Norambuena            | 12152763-0 | Galleta              | GAL0084       |
| 508 | e9c549      | 2023-07-12        | María Patricia Donoso-González Barrios   | 19556377-9 | Torta                | TOR0056       |
| 509 | a82092      | 2023-06-23        | María Belén Contreras Palma              | 10607341-2 | Magdalena            | MAG0043       |
| 510 | 43ddb0      | 2023-10-25        | Natacha Ashley Carmona Araya             | 17709684-2 | Cupcake              | CUP0066       |
| 511 | d2ca50      | 2023-06-16        | Antonia Yasna Vidal Morales              | 19094951-5 | Queque               | QUE0025       |
| 512 | 5c65a5      | 2023-07-14        | Diego Luis Urbina Alegría                | 12562177-8 | Torta                | TOR0093       |
| 513 | 80e85a      | 2023-11-22        | Leidy Gómez-Opazo Vega                   | 15021396-5 | Dona                 | DON0097       |
| 514 | 7efd15      | 2023-11-19        | Isabel Gómez                             | 18639001-2 | Cupcake              | CUP0019       |
| 515 | c3b398      | 2023-11-18        | Álvaro Aracena Moraga                    | 13637756-3 | Dona                 | DON0093       |
| 516 | 547af0      | 2023-09-17        | Ángel Juan Flores Olave                  | 13397154-5 | Cupcake              | CUP0052       |
| 517 | 346042      | 2023-07-16        | Natalia Fuentes Ramírez                  | 12349206-2 | Rosca                | ROS0031       |
| 518 | e1be92      | 2023-06-28        | Aníbal Roberto Candia Valenzuela         | 14491143-3 | Brownie              | BRO0046       |
| 519 | 24bc5d      | 2023-08-19        | Anahí Reyes                              | 10678640-6 | Torta                | TOR0062       |
| 520 | 4d1e46      | 2023-08-19        | Tatiana Graciela Fuentealba Burgos       | 18232977-5 | Dona                 | DON0056       |
| 521 | f627e6      | 2023-07-14        | Ricardo Guillermo Gutiérrez Plaza        | 19475696-6 | Galleta              | GAL0107       |
| 522 | 1f05ba      | 2023-09-06        | Verónica Muñoz Muñoz                     | 18581440-4 | Brownie              | BRO0096       |
| 523 | e56cc4      | 2023-08-21        | Christian Aranda                         | 16899941-9 | Dona                 | DON0066       |
| 524 | 336c14      | 2023-09-27        | Leonel Gustavo Vargas Escobar            | 19457804-8 | Torta                | TOR0041       |
| 525 | f911e8      | 2023-10-05        | Alexander José Venegas Ramírez           | 19647307-7 | Torta                | TOR0001       |
| 526 | 45b3a1      | 2023-09-03        | Uberlinda Valentina Guerrero Burgos      | 12075718-8 | Torta                | TOR0102       |
| 527 | 01ee52      | 2023-07-29        | Ana Contreras Díaz                       | 13943457-9 | Tarta                | TAR0006       |
| 528 | 313f26      | 2023-07-05        | Angelina Matus Padilla                   | 15213763-1 | Torta                | TOR0060       |
| 529 | 9ca58c      | 2023-10-30        | Alma Valeska Baez Pastén                 | 18559403-1 | Magdalena            | MAG0089       |
| 530 | cf6983      | 2023-08-03        | Flavio José Marín Riquelme               | 15372024-8 | Dona                 | DON0015       |
| 531 | a07b73      | 2023-10-14        | Elvis Sanhueza                           | 14717092-3 | Magdalena            | MAG0008       |
| 532 | 5ccfa3      | 2023-07-29        | Robert Darío Urra Vidal                  | 11864189-7 | Cupcake              | CUP0004       |
| 533 | cd5e29      | 2023-06-04        | Elizabeth Sáez Montenegro                | 12719807-3 | Tarta                | TAR0077       |
| 534 | be7f7f      | 2023-07-14        | Manuel Olivares                          | 17556367-6 | Galleta              | GAL0087       |
| 535 | c2f3df      | 2023-06-11        | Catalina Verónica Olguín Vergara         | 16162117-2 | Torta                | TOR0093       |
| 536 | 768698      | 2023-08-12        | Claudio Douglas Valenzuela Alarcón       | 15836789-0 | Queque               | QUE0024       |
| 537 | f78fb1      | 2023-10-22        | Nicole Iris Ovalle Miranda               | 16369557-4 | Torta                | TOR0071       |
| 538 | 8a161f      | 2023-09-02        | Martín Dylan Hernández Torres            | 15708709-1 | Queque               | QUE0072       |
| 539 | 9a0bcd      | 2023-09-26        | Rubén Federico Ruiz Villarroel           | 16649520-8 | Queque               | QUE0111       |
| 540 | 0e213d      | 2023-10-09        | Elías Moya Pérez                         | 10169733-9 | Magdalena            | MAG0041       |
| 541 | 7ff866      | 2023-06-08        | Luis Cid Verdugo                         | 18353524-1 | Cupcake              | CUP0063       |
| 542 | bf585c      | 2023-06-21        | Margarita Navarrete Henríquez            | 19322247-3 | Tarta                | TAR0088       |
| 543 | def89e      | 2023-10-15        | Paula Bravo Espinoza                     | 13450584-7 | Magdalena            | MAG0080       |
| 544 | 9e1f8a      | 2023-10-12        | Sergio Héctor Andrade Villagrán          | 11975867-2 | Tarta                | TAR0042       |
| 545 | 1f01bd      | 2023-10-18        | Cristian Martín Gutiérrez Herrera        | 11736474-6 | Torta                | TOR0060       |
| 546 | 17b9e6      | 2023-09-07        | Rosa Karina Urrutia Castro               | 19567912-1 | Tarta                | TAR0077       |
| 547 | bbfa2a      | 2023-10-03        | Manuel Cerda Oyarzún                     | 18990104-7 | Torta                | TOR0107       |
| 548 | 1fb44f      | 2023-08-13        | María Catalina Lizama Pérez              | 18788333-7 | Magdalena            | MAG0037       |
| 549 | e2055b      | 2023-06-05        | Víctor Ricardo Rosas Figueroa            | 11923108-1 | Queque               | QUE0011       |
| 550 | 3e0cbe      | 2023-07-12        | Ester María Reyes Guzmán                 | 10826143-4 | Magdalena            | MAG0036       |
| 551 | af62b9      | 2023-08-20        | Patricio Muñoz                           | 15477758-7 | Rosca                | ROS0060       |
| 552 | 98ad4d      | 2023-06-19        | Emilia Fanny Cáceres Salas               | 12266285-9 | Tarta                | TAR0102       |
| 553 | a2eb70      | 2023-07-30        | Mauricio Guillermo Romero Ulloa          | 11194051-0 | Queque               | QUE0028       |
| 554 | 8144e3      | 2023-10-04        | Víctor Ricardo Calderón Mella            | 17722464-8 | Dona                 | DON0040       |
| 555 | a47769      | 2023-09-19        | Felipe Jara                              | 15382551-6 | Tarta                | TAR0102       |
| 556 | 613048      | 2023-11-09        | Danilo Joaquín Oyarce Urrutia            | 18487901-0 | Queque               | QUE0004       |
| 557 | 711a01      | 2023-10-04        | Marilyn Irene Ramírez Opazo              | 15983726-1 | Cupcake              | CUP0066       |
| 558 | 2c871d      | 2023-11-09        | Luis Uribe Tobar                         | 16209807-5 | Rosca                | ROS0039       |
| 559 | 21aa02      | 2023-07-07        | Juana Javiera Cáceres Gálvez             | 18201231-6 | Dona                 | DON0074       |
| 560 | 310e71      | 2023-06-09        | Mauro Hermosilla Catalán                 | 16326089-5 | Rosca                | ROS0012       |
| 561 | 0bcb9c      | 2023-11-28        | Juan Márquez Cáceres                     | 16920962-2 | Dona                 | DON0106       |
| 562 | 604c0f      | 2023-11-18        | Pascale Núñez                            | 14359694-0 | Magdalena            | MAG0058       |
| 563 | 0f985d      | 2023-10-19        | Ester María Reyes Guzmán                 | 10826143-4 | Brownie              | BRO0013       |
| 564 | fd3171      | 2023-08-16        | Camila Sánchez García                    | 15944745-0 | Cupcake              | CUP0021       |
| 565 | 1023ac      | 2023-06-14        | Valeria Vania González Arancibia         | 13160972-K | Brownie              | BRO0077       |
| 566 | 3dc026      | 2023-11-08        | Juan Marcos Álvarez Valenzuela           | 18611521-K | Queque               | QUE0096       |
| 567 | 37be91      | 2023-08-09        | Marco Carrasco-Briones                   | 11148178-1 | Magdalena            | MAG0015       |
| 568 | e5a342      | 2023-09-03        | Rebeca Karla Rojas Navarro               | 15840899-0 | Torta                | TOR0099       |
| 569 | a30cae      | 2023-11-28        | Delia Ximena Montes López                | 18572344-K | Queque               | QUE0075       |
| 570 | bb46d5      | 2023-07-09        | Alexander Avendaño                       | 10259013-8 | Dona                 | DON0052       |
| 571 | 92e21e      | 2023-10-08        | Héctor Bastián Garrido Salas             | 16211159-K | Galleta              | GAL0002       |
| 572 | b0be04      | 2023-09-10        | Juan Elías Vergara Mendoza               | 19158626-2 | Queque               | QUE0024       |
| 573 | cc6096      | 2023-07-18        | Heriberto Molina-Pérez                   | 11701460-9 | Galleta              | GAL0067       |
| 574 | ad9927      | 2023-07-25        | Joaquín Sáez                             | 11514995-8 | Dona                 | DON0001       |
| 575 | 3306f6      | 2023-08-01        | Álvaro Aracena Moraga                    | 13637756-3 | Cupcake              | CUP0103       |
| 576 | 29a138      | 2023-10-28        | Augusto Arancibia                        | 14742034-8 | Tarta                | TAR0091       |
| 577 | 35f3f3      | 2023-06-22        | Claudio Hernán Arancibia Rojas           | 19903026-1 | Tarta                | TAR0096       |
| 578 | 1c33d9      | 2023-07-03        | Diego Pablo Rocha Cárcamo                | 13342133-9 | Brownie              | BRO0055       |
| 579 | d6c520      | 2023-06-21        | José Emilio González Molina              | 12037505-K | Galleta              | GAL0069       |
| 580 | e72d34      | 2023-07-18        | Dylan Muñoz Matamala                     | 12977677-K | Brownie              | BRO0085       |
| 581 | 92d4f0      | 2023-10-07        | Diego Andrés Morales Calderón            | 17692939-3 | Dona                 | DON0073       |
| 582 | 966d0e      | 2023-07-17        | Alonso Alex Pérez Bórquez                | 11931623-2 | Dona                 | DON0067       |
| 583 | b814bf      | 2023-11-22        | César Vivanco Quintana                   | 16783840-6 | Tarta                | TAR0084       |
| 584 | 2ab87a      | 2023-06-11        | Rossana Díaz Riquelme                    | 11073575-3 | Galleta              | GAL0117       |
| 585 | 095982      | 2023-07-24        | Pedro Benjamín Díaz Torres               | 11287384-0 | Tarta                | TAR0080       |
| 586 | a2e412      | 2023-06-18        | Bernardo Juan Cárdenas Allende           | 12062171-6 | Brownie              | BRO0104       |
| 587 | ee9355      | 2023-06-09        | Natalia Fuentes Ramírez                  | 12349206-2 | Rosca                | ROS0002       |
| 588 | c79899      | 2023-06-11        | María Vargas                             | 18671704-3 | Torta                | TOR0058       |
| 589 | c4e026      | 2023-11-09        | Alicia González                          | 16413797-6 | Torta                | TOR0044       |
| 590 | 4d5843      | 2023-09-19        | Marcelo Flores                           | 12197219-6 | Cupcake              | CUP0090       |
| 591 | 5c43fd      | 2023-06-14        | Heriberto Mauricio Muñoz Barraza         | 10457069-2 | Dona                 | DON0039       |
| 592 | ef6eb3      | 2023-07-06        | Valentina Silvana Norambuena Villarroel  | 14623271-5 | Cupcake              | CUP0052       |
| 593 | 540a32      | 2023-11-07        | Marcelo Rubio Zúñiga                     | 12100935-7 | Cupcake              | CUP0081       |
| 594 | 38cac9      | 2023-10-05        | Berta Martina Marín Hernández            | 17284393-9 | Galleta              | GAL0059       |
| 595 | 55bd23      | 2023-06-02        | Maximiliano Sánchez                      | 15319157-5 | Torta                | TOR0015       |
| 596 | f51a04      | 2023-08-15        | María Varas Luna                         | 16677355-2 | Magdalena            | MAG0057       |
| 597 | eff022      | 2023-10-20        | Fernanda Carolina Rebolledo Arcos        | 18426636-7 | Dona                 | DON0010       |
| 598 | 41aa52      | 2023-07-16        | Karla Camila Zúñiga Saldivia             | 12067317-7 | Brownie              | BRO0056       |
| 599 | c7b8c7      | 2023-09-23        | Paula Teresa Martínez Pereira            | 11793333-9 | Magdalena            | MAG0069       |
| 600 | 2c85b5      | 2023-09-16        | José Rodríguez Aguirre                   | 18269245-3 | Torta                | TOR0073       |
| 601 | 69e143      | 2023-10-08        | Claudio Jaime Díaz Donoso                | 10799922-2 | Galleta              | GAL0002       |
| 602 | 0066a7      | 2023-10-31        | Thomas Ian Aguilera Cáceres              | 16524128-3 | Cupcake              | CUP0100       |
| 603 | 551712      | 2023-09-25        | Valentina Gladys Neira Zurita            | 13650430-5 | Galleta              | GAL0068       |
| 604 | 5b81e9      | 2023-10-22        | Noemí Flor Medina Segovia                | 19966188-0 | Magdalena            | MAG0098       |
| 605 | 7a486e      | 2023-10-31        | Leonel Gustavo Vargas Escobar            | 19457804-8 | Brownie              | BRO0062       |
| 606 | 0fc54a      | 2023-08-06        | Loreto Nadia Cartes García               | 13298614-9 | Dona                 | DON0012       |
| 607 | 63c22c      | 2023-06-14        | Julio José Jiménez Pérez                 | 12232016-K | Magdalena            | MAG0091       |
| 608 | 24490b      | 2023-06-06        | René Sergio Arias Quispe                 | 12516349-1 | Galleta              | GAL0088       |
| 609 | e1ac76      | 2023-06-11        | José Sergio Riveros Parra                | 16458119-3 | Rosca                | ROS0008       |
| 610 | 32f6b7      | 2023-07-23        | Jeannette Paredes                        | 18438148-1 | Tarta                | TAR0054       |
| 611 | 977958      | 2023-08-19        | Catalina Aguayo Cabello                  | 12265200-9 | Cupcake              | CUP0090       |
| 612 | 902209      | 2023-08-16        | Pablo Valdés                             | 16390909-3 | Galleta              | GAL0089       |
| 613 | 2b12e2      | 2023-11-08        | Lorena Arenas Medel                      | 11224973-8 | Galleta              | GAL0110       |
| 614 | 697676      | 2023-08-22        | Jacqueline Ruth Vera Sepúlveda           | 15029190-4 | Tarta                | TAR0011       |
| 615 | 9768db      | 2023-11-05        | Manuel Cerda Oyarzún                     | 18990104-7 | Galleta              | GAL0084       |
| 616 | 22cf5b      | 2023-10-16        | Pía María Valdés Santos                  | 16089285-K | Cupcake              | CUP0058       |
| 617 | b2182f      | 2023-07-25        | María Ceballos                           | 19167633-1 | Cupcake              | CUP0106       |
| 618 | cca1d1      | 2023-11-01        | Cristina Laura Villagrán Díaz            | 15757242-9 | Rosca                | ROS0008       |
| 619 | 81ee51      | 2023-06-17        | Carmen Urbina González                   | 19480909-6 | Rosca                | ROS0002       |
| 620 | f292da      | 2023-07-13        | Víctor Diego González Segovia            | 15442719-5 | Rosca                | ROS0064       |
| 621 | 99a09d      | 2023-08-02        | José Moisés Martínez Araya               | 18348242-4 | Brownie              | BRO0090       |
| 622 | df1ae6      | 2023-09-10        | Roberto Juan Espinoza Valenzuela         | 18290653-1 | Rosca                | ROS0029       |
| 623 | 31c06d      | 2023-09-02        | Máximo Rodrigo Toledo Valenzuela         | 15327763-1 | Queque               | QUE0050       |
| 624 | 3bbc91      | 2023-06-15        | Paula Antonia Guerrero Gallegos          | 10141898-4 | Torta                | TOR0047       |
| 625 | 625ab0      | 2023-11-17        | Irma Sepúlveda                           | 12942554-0 | Brownie              | BRO0105       |
| 626 | 9c6ce5      | 2023-08-10        | Angelina Matus Padilla                   | 15213763-1 | Galleta              | GAL0004       |
| 627 | 6d1b9d      | 2023-11-26        | Yolanda Ibarra-Paredes                   | 11679190-0 | Brownie              | BRO0037       |
| 628 | 776a16      | 2023-06-20        | Sabina Marcela Carvajal Poblete          | 10707307-6 | Queque               | QUE0063       |
| 629 | 524036      | 2023-08-21        | Alondra Bahamondes Villar                | 18738327-1 | Magdalena            | MAG0076       |
| 630 | a26007      | 2023-07-10        | Miriam Ana Zenteno Lizana                | 17098694-0 | Tarta                | TAR0044       |
| 631 | 56ceff      | 2023-07-08        | Cristian Olea Carrasco                   | 12374613-9 | Magdalena            | MAG0068       |
| 632 | cf2adb      | 2023-11-05        | Valentina Ávila Salazar                  | 13679189-7 | Tarta                | TAR0007       |
| 633 | a20e54      | 2023-08-19        | Camila Carolina Espinosa Torres          | 14360083-5 | Dona                 | DON0037       |
| 634 | 77eaf4      | 2023-08-18        | Rosa Escobar Díaz                        | 18569055-3 | Brownie              | BRO0058       |
| 635 | 722c0a      | 2023-09-29        | María Magdalena Marambio Bobadilla       | 13147756-7 | Torta                | TOR0052       |
| 636 | 203aa4      | 2023-11-04        | Angélica Muñoz Cifuentes                 | 13928949-0 | Magdalena            | MAG0069       |
| 637 | 3eb56e      | 2023-09-15        | Paola Bernardita Herrera González        | 15977741-0 | Brownie              | BRO0001       |
| 638 | 8db1cf      | 2023-09-10        | Camila Carolina Duarte Padilla           | 12677680-9 | Torta                | TOR0061       |
| 639 | 2b4280      | 2023-06-03        | Fernanda Gladys García Orellana          | 19231518-4 | Magdalena            | MAG0019       |
| 640 | a4c222      | 2023-10-31        | Bernardo Esteban Montecinos Morales      | 15187721-9 | Galleta              | GAL0059       |
| 641 | faf662      | 2023-10-06        | Ricardo Guillermo Gutiérrez Plaza        | 19475696-6 | Galleta              | GAL0112       |
| 642 | 602e6f      | 2023-10-05        | Francisco Luis Muñoz Olguín              | 15939801-9 | Dona                 | DON0088       |
| 643 | ef102f      | 2023-08-09        | Sebastián Vicente Toro Cortez            | 14528432-6 | Galleta              | GAL0118       |
| 644 | a1b8b5      | 2023-08-01        | Constanza Celis                          | 10863147-2 | Torta                | TOR0061       |
| 645 | 5eb162      | 2023-06-04        | Francisco Ronald Meza Cárdenas           | 15906096-3 | Galleta              | GAL0105       |
| 646 | c9ff2e      | 2023-08-19        | Santino Mauricio Moreno Orellana         | 12056617-8 | Queque               | QUE0108       |
| 647 | ad5617      | 2023-11-24        | María Catherine León Arenas              | 11660995-3 | Rosca                | ROS0013       |
| 648 | d0633b      | 2023-07-31        | Orlando Sergio Vargas Arancibia          | 14959522-8 | Cupcake              | CUP0040       |
| 649 | da9d9b      | 2023-07-18        | Carla Carolina González Castro           | 10112939-7 | Galleta              | GAL0078       |
| 650 | 6f17e1      | 2023-10-30        | Noemí Martina Navarro-Olivares Rivera    | 12364781-4 | Rosca                | ROS0039       |
| 651 | 2b3333      | 2023-11-20        | Ernesto Medina Martínez                  | 18776970-7 | Magdalena            | MAG0099       |
| 652 | 3cd6dd      | 2023-06-12        | José Gastón Vidal Leal                   | 11550449-2 | Queque               | QUE0052       |
| 653 | d0286e      | 2023-09-18        | Patricio Oscar Valdés Carrasco           | 17310189-3 | Dona                 | DON0059       |
| 654 | 98def4      | 2023-06-23        | María Romero Fernández                   | 18782065-3 | Cupcake              | CUP0080       |
| 655 | 93ee24      | 2023-09-16        | Raúl Urbina                              | 15616810-3 | Magdalena            | MAG0064       |
| 656 | f8bc96      | 2023-07-22        | María Troncoso                           | 12409530-2 | Torta                | TOR0018       |
| 657 | 38ab92      | 2023-08-03        | Juan Luis Contreras Jaramillo            | 10589572-0 | Torta                | TOR0056       |
| 658 | 7d1407      | 2023-11-10        | Jaime Becerra                            | 13423715-4 | Magdalena            | MAG0092       |
| 659 | 5224f2      | 2023-10-21        | Catherine Andrea Duarte Vilches          | 19243571-8 | Tarta                | TAR0056       |
| 660 | fd88dd      | 2023-09-07        | Irma Vera                                | 10038028-8 | Dona                 | DON0065       |
| 661 | 85c854      | 2023-10-14        | Isidora Soto Cabello                     | 16586821-0 | Magdalena            | MAG0012       |
| 662 | e45078      | 2023-08-06        | Claudio Aguirre Bahamondes               | 19287148-9 | Brownie              | BRO0071       |
| 663 | 21d669      | 2023-08-24        | Máximo Rodrigo Toledo Valenzuela         | 15327763-1 | Torta                | TOR0031       |
| 664 | 15aafe      | 2023-09-11        | Mario Carlos Contreras Garcés            | 14586235-2 | Galleta              | GAL0096       |
| 665 | 1f5434      | 2023-09-17        | Norma Juana Muñoz Arias                  | 16924569-6 | Dona                 | DON0029       |
| 666 | 7514ce      | 2023-09-26        | Georgina María Campos Cruz               | 15601503-6 | Magdalena            | MAG0006       |
| 667 | d51204      | 2023-09-11        | Victoria Leal Silva                      | 16918077-9 | Brownie              | BRO0064       |
| 668 | 021979      | 2023-06-27        | Gabriela Sepúlveda                       | 17111160-9 | Queque               | QUE0062       |
| 669 | 78eefa      | 2023-11-10        | Ester María Reyes Guzmán                 | 10826143-4 | Torta                | TOR0093       |
| 670 | b93a3a      | 2023-06-11        | José Emilio González Molina              | 12037505-K | Magdalena            | MAG0059       |
| 671 | b08d87      | 2023-10-10        | Jacqueline Catherine Villagrán Ibáñez    | 10870035-2 | Dona                 | DON0019       |
| 672 | 395bce      | 2023-06-30        | Rosa Bustos Delgado                      | 18417279-4 | Galleta              | GAL0106       |
| 673 | 7899d9      | 2023-08-06        | Cristian Martín Gutiérrez Herrera        | 11736474-6 | Dona                 | DON0035       |
| 674 | cca863      | 2023-06-23        | Elías Moya Pérez                         | 10169733-9 | Queque               | QUE0088       |
| 675 | da2ba9      | 2023-10-27        | Natalia Florencia Morales Jiménez        | 10017334-1 | Tarta                | TAR0022       |
| 676 | 53f902      | 2023-08-15        | Claudia Nora Rosas Valderrama            | 15290800-6 | Torta                | TOR0050       |
| 677 | 5ce408      | 2023-06-13        | José Emilio González Molina              | 12037505-K | Magdalena            | MAG0005       |
| 678 | 9fd278      | 2023-09-19        | Alicia Teresa Gómez Medel                | 19692541-1 | Brownie              | BRO0062       |
| 679 | b46604      | 2023-09-29        | Rossana Dina Espinoza Rosales            | 18600249-5 | Dona                 | DON0040       |
| 680 | cddaae      | 2023-10-15        | Camila Catalina Santander Rivera         | 11717542-9 | Tarta                | TAR0034       |
| 681 | 8070eb      | 2023-08-30        | Francisco Patricio Mancilla Quinteros    | 15404882-3 | Galleta              | GAL0018       |
| 682 | c724bc      | 2023-07-13        | Erika Hurtado                            | 13403990-6 | Cupcake              | CUP0078       |
| 683 | cec817      | 2023-10-01        | Karen Yáñez Castillo                     | 12996347-0 | Brownie              | BRO0010       |
| 684 | 9483b7      | 2023-10-29        | Lucas Cisternas Burgos                   | 19858682-6 | Cupcake              | CUP0001       |
| 685 | c0ee29      | 2023-08-12        | José Díaz                                | 16813964-4 | Rosca                | ROS0070       |
| 686 | f70ee8      | 2023-08-19        | Jocelyn Eliana Vera Acevedo              | 15512192-9 | Cupcake              | CUP0068       |
| 687 | 4e1c0a      | 2023-11-27        | Marco Ruiz                               | 11215531-K | Torta                | TOR0049       |
| 688 | 587ea6      | 2023-07-12        | Luz Matilda Carmona Romero               | 11607803-5 | Brownie              | BRO0001       |
| 689 | 946d45      | 2023-09-20        | Camila Mansilla Valderrama               | 13658398-7 | Rosca                | ROS0087       |
| 690 | 0e7907      | 2023-08-14        | María Nancy Roa Rojas                    | 15857889-K | Brownie              | BRO0034       |
| 691 | 52aa4f      | 2023-06-02        | Mónica María Moreno Quiroz               | 11875998-4 | Brownie              | BRO0004       |
| 692 | 317aa5      | 2023-07-15        | Susana Beatriz Aravena Quezada           | 14375730-8 | Torta                | TOR0083       |
| 693 | ec0bc0      | 2023-09-07        | María Cruz-Chamorro Céspedes             | 19369391-1 | Magdalena            | MAG0033       |
| 694 | 2e40e9      | 2023-10-07        | Teresa Solís                             | 11493415-7 | Rosca                | ROS0004       |
| 695 | e03a1d      | 2023-10-16        | Lorena Claudia Yáñez Muñoz               | 14944449-2 | Dona                 | DON0022       |
| 696 | 92907c      | 2023-11-27        | Sebastián Francisco Ávila López          | 13161078-1 | Torta                | TOR0060       |
| 697 | e18f6f      | 2023-06-10        | Ema Jeanette Ávila Vidal                 | 19266252-5 | Torta                | TOR0030       |
| 698 | 6d1e6a      | 2023-08-04        | Mateo Andrés Villa Soto                  | 18205268-9 | Torta                | TOR0046       |
| 699 | 60ea69      | 2023-11-17        | Jael Gajardo Ibarra                      | 12190909-1 | Dona                 | DON0067       |
| 700 | 872bd6      | 2023-06-12        | Juan Luis Domínguez Tapia                | 19204831-9 | Dona                 | DON0100       |
| 701 | dc718b      | 2023-11-05        | José Gastón Vidal Leal                   | 11550449-2 | Brownie              | BRO0073       |
| 702 | 1affe2      | 2023-10-09        | Claudio Martín Machuca Rojas             | 17666644-2 | Galleta              | GAL0052       |
| 703 | 2e474f      | 2023-09-15        | Bastián Pascual Navarro Salas            | 10819523-0 | Queque               | QUE0092       |
| 704 | 1b678b      | 2023-08-04        | Bernardo Manuel Aguilera Henríquez       | 11169838-K | Tarta                | TAR0104       |
| 705 | 52ef9a      | 2023-09-15        | Patricia Segura Ávila                    | 13786441-3 | Galleta              | GAL0099       |
| 706 | 48d24d      | 2023-08-20        | Adrián Vicente Toro Estrada              | 16322904-4 | Brownie              | BRO0090       |
| 707 | 175da4      | 2023-08-18        | Bastián Lucas Retamal Aguilera           | 16936297-4 | Cupcake              | CUP0050       |
| 708 | 19baa5      | 2023-09-16        | Carla Sáez                               | 15012343-3 | Cupcake              | CUP0046       |
| 709 | 18a16f      | 2023-08-17        | Osvaldo Vicente Hernández Uribe          | 10839812-4 | Magdalena            | MAG0079       |
| 710 | 854373      | 2023-06-16        | Hilda Jofré                              | 18513213-3 | Queque               | QUE0021       |
| 711 | 671f8a      | 2023-06-05        | Robert Darío Urra Vidal                  | 11864189-7 | Brownie              | BRO0019       |
| 712 | e8e0a9      | 2023-08-11        | Julia Benítez                            | 11097563-4 | Cupcake              | CUP0016       |
| 713 | c8c859      | 2023-06-30        | Alejandro Alfonso Toledo Concha          | 18108656-2 | Dona                 | DON0064       |
| 714 | 11e3fc      | 2023-11-02        | Francisco Luis Muñoz Olguín              | 15939801-9 | Magdalena            | MAG0043       |
| 715 | c8acb5      | 2023-11-22        | Patricio Hidalgo                         | 18255655-2 | Brownie              | BRO0051       |
| 716 | 30636b      | 2023-10-29        | Gaspar Jorge Sepúlveda Vásquez           | 13934218-8 | Queque               | QUE0002       |
| 717 | d19d3a      | 2023-08-15        | Gustavo Pablo Martínez San Martín        | 14264158-6 | Galleta              | GAL0025       |
| 718 | 3beae4      | 2023-08-29        | Juan Nelson Cifuentes Cabello            | 17655980-6 | Queque               | QUE0124       |
| 719 | f6558f      | 2023-09-02        | Leonel Sergio Figueroa Cofré             | 13637138-5 | Dona                 | DON0112       |
| 720 | 23e8ee      | 2023-08-12        | César Toro Barría                        | 15563165-6 | Galleta              | GAL0003       |
| 721 | 4a77c8      | 2023-06-21        | Julia Valentina Pérez Sobarzo            | 12713512-3 | Tarta                | TAR0014       |
| 722 | a179fd      | 2023-07-06        | Juan Adán Pérez Saavedra                 | 10141662-2 | Galleta              | GAL0087       |
| 723 | 8dddc5      | 2023-10-13        | Moisés Núñez                             | 10918277-2 | Brownie              | BRO0047       |
| 724 | d853d2      | 2023-10-20        | Aníbal Roberto Candia Valenzuela         | 14491143-3 | Torta                | TOR0024       |
| 725 | e4182b      | 2023-07-09        | Emilia Paulina González Parra            | 12592975-3 | Tarta                | TAR0003       |
| 726 | 6ec91e      | 2023-06-08        | Osvaldo Marchant                         | 15879643-K | Queque               | QUE0073       |
| 727 | 5c2fa6      | 2023-09-20        | Carlos José Arévalo Correa               | 18108973-K | Cupcake              | CUP0092       |
| 728 | bf4dc4      | 2023-11-01        | Berta Baeza                              | 15202780-8 | Tarta                | TAR0088       |
| 729 | a90e57      | 2023-10-06        | María Catalina Lizama Pérez              | 18788333-7 | Cupcake              | CUP0090       |
| 730 | bbb74d      | 2023-10-06        | Carolina Gatica García                   | 16664797-0 | Magdalena            | MAG0044       |
| 731 | 4c680d      | 2023-10-02        | María Ceballos                           | 19167633-1 | Dona                 | DON0029       |
| 732 | 420348      | 2023-11-20        | Pedro Valenzuela Rojas                   | 18606312-1 | Galleta              | GAL0039       |
| 733 | 9926ac      | 2023-08-24        | Yonatan Luis Mendoza Silva               | 13285472-1 | Torta                | TOR0016       |
| 734 | 4ae4a2      | 2023-07-21        | René Véliz Vega                          | 15480224-8 | Rosca                | ROS0036       |
| 735 | 088042      | 2023-07-03        | Patricio Juan Piña Torres                | 18167891-0 | Tarta                | TAR0077       |
| 736 | 6f8f60      | 2023-06-25        | Brígida Gutiérrez                        | 10126260-K | Tarta                | TAR0048       |
| 737 | 6bb611      | 2023-10-04        | Liam Jorge Aránguiz Silva                | 13100995-3 | Dona                 | DON0109       |
| 738 | 607c71      | 2023-11-10        | Renata María Cuevas Quezada              | 15536201-8 | Magdalena            | MAG0030       |
| 739 | 27a4cc      | 2023-10-09        | Juan Nelson Cifuentes Cabello            | 17655980-6 | Tarta                | TAR0037       |
| 740 | 885180      | 2023-07-16        | Paolo Reyes Briceño                      | 10517756-4 | Brownie              | BRO0082       |
| 741 | c61e56      | 2023-11-19        | Juan Mamani Parra                        | 19108201-4 | Magdalena            | MAG0082       |
| 742 | cef34b      | 2023-11-19        | Inés Agüero                              | 13188648-1 | Queque               | QUE0073       |
| 743 | 8f70cf      | 2023-07-26        | Viviana Maritza Díaz Carreño             | 14375847-K | Magdalena            | MAG0055       |
| 744 | 40921d      | 2023-08-21        | Javier Silva                             | 19720890-2 | Rosca                | ROS0087       |
| 745 | c07804      | 2023-06-03        | José Dagoberto Garrido Rojas             | 12150809-6 | Galleta              | GAL0081       |
| 746 | e9cae9      | 2023-10-05        | María Patricia Ortega Molina             | 13562982-2 | Brownie              | BRO0033       |
| 747 | a74d9c      | 2023-10-07        | Brayan Pinto Cruz                        | 13687335-K | Galleta              | GAL0040       |
| 748 | 3c93aa      | 2023-09-07        | Susana Reyes Ojeda                       | 18856334-8 | Dona                 | DON0082       |
| 749 | 33748e      | 2023-09-10        | Julia Valentina Pérez Sobarzo            | 12713512-3 | Dona                 | DON0086       |
| 750 | 709346      | 2023-08-24        | María Catherine León Arenas              | 11660995-3 | Magdalena            | MAG0093       |
| 751 | c7d7b6      | 2023-06-09        | María Mariana Rodríguez Garcés           | 16746947-1 | Magdalena            | MAG0091       |
| 752 | 4676bb      | 2023-09-24        | Marta Rosa Araya Aguirre                 | 10476024-6 | Rosca                | ROS0064       |
| 753 | 92013a      | 2023-08-26        | Natacha Ashley Carmona Araya             | 17709684-2 | Rosca                | ROS0089       |
| 754 | b2e5a9      | 2023-09-10        | Maximiliano Bustos Fuentes               | 10279475-2 | Torta                | TOR0077       |
| 755 | e1ed5c      | 2023-10-15        | Luis Aburto                              | 11725463-8 | Magdalena            | MAG0074       |
| 756 | 876d3b      | 2023-10-01        | Alma Álvarez                             | 12661877-5 | Cupcake              | CUP0058       |
| 757 | da9f6e      | 2023-09-10        | Yolanda Ibarra-Paredes                   | 11679190-0 | Rosca                | ROS0064       |
| 758 | 460323      | 2023-10-11        | Jennifer Marta Zambrano Sandoval         | 19394914-1 | Tarta                | TAR0091       |
| 759 | 449482      | 2023-07-21        | Carlos Urzúa Aguayo                      | 18760427-5 | Dona                 | DON0013       |
| 760 | 572237      | 2023-10-16        | Hilda Antonella Ruiz-Reyes Cortés        | 11257850-8 | Galleta              | GAL0070       |
| 761 | 44d589      | 2023-07-14        | Manuel Pedro Pinto Rojas                 | 17590173-1 | Dona                 | DON0057       |
| 762 | ab7f07      | 2023-06-15        | Marisa Violeta Soto Vega                 | 13098802-5 | Cupcake              | CUP0033       |
| 763 | df9cb1      | 2023-10-07        | Valentina Silvana Norambuena Villarroel  | 14623271-5 | Torta                | TOR0074       |
| 764 | 80d40a      | 2023-06-14        | Darío Vargas Sandoval                    | 11179031-6 | Dona                 | DON0044       |
| 765 | bf5714      | 2023-06-09        | Angélica Muñoz Cifuentes                 | 13928949-0 | Brownie              | BRO0081       |
| 766 | aae037      | 2023-11-13        | Andrés Rivera                            | 12686777-5 | Rosca                | ROS0012       |
| 767 | 79cacd      | 2023-08-14        | Hernán Arenas Toro                       | 13730313-0 | Dona                 | DON0103       |
| 768 | d9a134      | 2023-10-19        | Génesis Carrasco Ibarra                  | 12436825-8 | Cupcake              | CUP0060       |
| 769 | 2beb0e      | 2023-10-10        | Ema Jeanette Ávila Vidal                 | 19266252-5 | Dona                 | DON0114       |
| 770 | a8bbb2      | 2023-07-01        | Gisselle Ana Vergara Bahamondes          | 18945471-0 | Rosca                | ROS0063       |
| 771 | 8d69a6      | 2023-06-20        | Lionel Gallegos                          | 14804494-5 | Brownie              | BRO0066       |
| 772 | 500ee1      | 2023-06-24        | Daniel Orellana Hernández                | 13805442-1 | Rosca                | ROS0052       |
| 773 | 5c99c8      | 2023-07-18        | Clara Claudia González Seguel            | 16229934-4 | Dona                 | DON0101       |
| 774 | d0d749      | 2023-08-26        | Paula Astrid Cortés Rivas                | 10970023-0 | Brownie              | BRO0077       |
| 775 | cecb32      | 2023-06-07        | Carla Carolina González Castro           | 10112939-7 | Brownie              | BRO0004       |
| 776 | 7f66e9      | 2023-08-13        | Marcela Ramírez Cuevas                   | 12110296-6 | Cupcake              | CUP0024       |
| 777 | e1bdcc      | 2023-08-23        | Francisco Flores Cáceres                 | 16854712-2 | Torta                | TOR0038       |
| 778 | fb0692      | 2023-10-26        | Antonia Catalina Verdugo Cabello         | 11684977-6 | Cupcake              | CUP0079       |
| 779 | 43a307      | 2023-09-14        | Rosa Bustos Villarroel                   | 13217852-0 | Torta                | TOR0074       |
| 780 | 6ee69f      | 2023-08-14        | Efraín Carlos Jaramillo Navarro          | 13843773-K | Cupcake              | CUP0057       |
| 781 | cb8938      | 2023-08-14        | Joaquín Barrera                          | 15956314-6 | Torta                | TOR0051       |
| 782 | 12a583      | 2023-06-20        | Elsa Cecilia Bravo Celis                 | 16950266-3 | Dona                 | DON0054       |
| 783 | 1b01f4      | 2023-07-21        | Benjamín Faúndez Henríquez               | 15903863-4 | Dona                 | DON0085       |
| 784 | 90480d      | 2023-09-15        | Marta Sandra Suárez Torres               | 14806923-5 | Magdalena            | MAG0041       |
| 785 | d47798      | 2023-07-26        | Jesús Jaime Concha San Martín            | 16195147-7 | Tarta                | TAR0025       |
| 786 | 1927fd      | 2023-06-30        | Guillermo Escobar Mondaca                | 11278135-1 | Brownie              | BRO0031       |
| 787 | 45e4cd      | 2023-06-20        | Loreto Nadia Cartes García               | 13298614-9 | Dona                 | DON0103       |
| 788 | f1d401      | 2023-11-03        | Berta Jeannette Herrera Figueroa         | 12255158-0 | Queque               | QUE0090       |
| 789 | 54e685      | 2023-07-31        | Matías Cáceres                           | 18046189-5 | Dona                 | DON0105       |
| 790 | ebb9c0      | 2023-06-15        | Rubén José Peralta Sepúlveda             | 10710132-7 | Cupcake              | CUP0033       |
| 791 | 108bfd      | 2023-08-30        | Violeta Victoria Barra Troncoso          | 18900459-K | Magdalena            | MAG0063       |
| 792 | 2f8075      | 2023-08-29        | Luz Alison Yáñez González                | 15236162-4 | Galleta              | GAL0062       |
| 793 | 50e327      | 2023-06-29        | María Maritza Arévalo Rodríguez          | 13042641-6 | Cupcake              | CUP0003       |
| 794 | f0952d      | 2023-07-07        | Noah Iván Valenzuela Hermosilla          | 19598915-2 | Queque               | QUE0113       |
| 795 | c885ea      | 2023-07-25        | Patricia Henríquez Silva                 | 17175684-4 | Tarta                | TAR0011       |
| 796 | da86fa      | 2023-09-22        | Antonio Jaime Pizarro Estay              | 16418398-4 | Cupcake              | CUP0028       |
| 797 | c2b814      | 2023-08-26        | Javiera Agustina Calderón Guerrero       | 19641357-0 | Brownie              | BRO0071       |
| 798 | 0b53ad      | 2023-07-15        | Jaime Villagra Cuevas                    | 11066531-9 | Queque               | QUE0110       |
| 799 | db9b07      | 2023-09-08        | Denisse Carrasco Contreras               | 18023169-4 | Dona                 | DON0095       |
| 800 | c5a956      | 2023-11-27        | Rodolfo Aguilar                          | 10430301-1 | Brownie              | BRO0003       |
| 801 | c5a08e      | 2023-11-09        | Camila Sánchez García                    | 15944745-0 | Galleta              | GAL0041       |
| 802 | 931dab      | 2023-06-18        | Ángel Juan Flores Olave                  | 13397154-5 | Magdalena            | MAG0043       |
| 803 | 8d4974      | 2023-10-31        | Paulina Cecilia Muñoz Arévalo            | 13872284-1 | Magdalena            | MAG0069       |
| 804 | 9d991e      | 2023-11-02        | Mario Cerda Pino                         | 14747753-2 | Torta                | TOR0040       |
| 805 | 825716      | 2023-08-21        | Marisa Violeta Soto Vega                 | 13098802-5 | Tarta                | TAR0003       |
| 806 | 4f48c9      | 2023-08-03        | Carlos Durán Olguín                      | 17234779-K | Torta                | TOR0076       |
| 807 | ce76f4      | 2023-08-19        | María Patricia Ortega Molina             | 13562982-2 | Torta                | TOR0004       |
| 808 | af4134      | 2023-07-05        | Carlos Escobar Casanova                  | 17494771-5 | Dona                 | DON0068       |
| 809 | d4a173      | 2023-08-08        | Jonathan Ignacio Lagos Madrid            | 17265039-2 | Cupcake              | CUP0101       |
| 810 | d02c3a      | 2023-06-10        | María Salazar                            | 16847655-K | Queque               | QUE0067       |
| 811 | 97e845      | 2023-07-06        | Felipe Jara                              | 15382551-6 | Rosca                | ROS0082       |
| 812 | 262df2      | 2023-11-22        | Edmundo Javier Arriagada Flores          | 11837686-2 | Magdalena            | MAG0093       |
| 813 | 4807af      | 2023-11-05        | Pedro Hugo Albornoz Cáceres              | 11915933-9 | Galleta              | GAL0071       |
| 814 | 5849a0      | 2023-10-09        | Juan Luis Contreras Jaramillo            | 10589572-0 | Magdalena            | MAG0095       |
| 815 | 0c46b9      | 2023-10-17        | Heriberto Mauricio Muñoz Barraza         | 10457069-2 | Torta                | TOR0072       |
| 816 | e2983b      | 2023-10-01        | Luis Cid Verdugo                         | 18353524-1 | Rosca                | ROS0004       |
| 817 | aa5321      | 2023-10-23        | Ariel Aguilera Toledo                    | 17892265-5 | Dona                 | DON0050       |
| 818 | 3614d0      | 2023-09-19        | María Vera Aguilera                      | 13847164-8 | Torta                | TOR0097       |
| 819 | 1a88a5      | 2023-07-11        | Valentina Marín                          | 14937967-5 | Magdalena            | MAG0064       |
| 820 | 515550      | 2023-08-20        | Valentina Marín                          | 14937967-5 | Rosca                | ROS0026       |
| 821 | d5376b      | 2023-08-28        | Angélica Muñoz Cifuentes                 | 13928949-0 | Galleta              | GAL0040       |
| 822 | be6332      | 2023-11-03        | Felipe Jara                              | 15382551-6 | Cupcake              | CUP0045       |
| 823 | f09ef4      | 2023-07-31        | Natalia Fuentes Ramírez                  | 12349206-2 | Torta                | TOR0065       |
| 824 | 9a7177      | 2023-10-31        | Pascale Roxana Rojas Hidalgo             | 11179458-7 | Tarta                | TAR0001       |
| 825 | d87120      | 2023-09-20        | Antonia Rivera Peralta                   | 16353978-2 | Queque               | QUE0062       |
| 826 | fafb9f      | 2023-07-29        | Olga Emilia Fernández Ramírez            | 11532342-7 | Cupcake              | CUP0045       |
| 827 | 87f278      | 2023-11-03        | Leonel Aravena                           | 12026901-4 | Torta                | TOR0101       |
| 828 | 85a755      | 2023-07-09        | María Nancy Roa Rojas                    | 15857889-K | Galleta              | GAL0080       |
| 829 | 1b787c      | 2023-06-02        | Marco Carrasco-Briones                   | 11148178-1 | Rosca                | ROS0056       |
| 830 | 111627      | 2023-06-29        | Rocío Bernal                             | 10754589-2 | Cupcake              | CUP0021       |
| 831 | 4e4a46      | 2023-06-11        | Nelson Llanos Vásquez                    | 17866142-7 | Dona                 | DON0063       |
| 832 | 775bf0      | 2023-06-01        | Francisca Antonieta Alarcón Navarro      | 16039710-3 | Dona                 | DON0033       |
| 833 | 07e78d      | 2023-09-20        | Luis Garay Acuña                         | 12836581-1 | Galleta              | GAL0092       |
| 834 | 05d3f7      | 2023-10-14        | Gustavo Javier Gatica Silva              | 11392266-K | Brownie              | BRO0072       |
| 835 | b0cea8      | 2023-07-20        | Erika Hurtado                            | 13403990-6 | Rosca                | ROS0089       |
| 836 | e409ea      | 2023-11-15        | Pedro Hugo Albornoz Cáceres              | 11915933-9 | Queque               | QUE0071       |
| 837 | a9ab14      | 2023-11-07        | María Marianela Farías Abarca            | 15525494-9 | Dona                 | DON0054       |
| 838 | 096ac2      | 2023-08-13        | Carolina Gatica García                   | 16664797-0 | Brownie              | BRO0039       |
| 839 | 442f11      | 2023-07-02        | Sandra Ximena Matus Baeza                | 11131094-7 | Cupcake              | CUP0023       |
| 840 | d7ffd4      | 2023-10-15        | José González Fuentes                    | 18046339-1 | Magdalena            | MAG0069       |
| 841 | 4a3e5e      | 2023-10-12        | Marco Carrasco-Briones                   | 11148178-1 | Dona                 | DON0051       |
| 842 | 99bcc3      | 2023-11-16        | Juana Fernanda Huerta Castillo           | 13793953-1 | Tarta                | TAR0034       |
| 843 | cc2482      | 2023-11-09        | Lucy Miranda González                    | 16872897-3 | Torta                | TOR0050       |
| 844 | 468255      | 2023-06-12        | Noemí Martina Navarro-Olivares Rivera    | 12364781-4 | Queque               | QUE0055       |
| 845 | 9457e8      | 2023-11-28        | Gregory Jaime Luna Villegas              | 19337387-8 | Queque               | QUE0073       |
| 846 | 5b6d5e      | 2023-07-11        | Marisol Valentina López Elgueta          | 11930224-3 | Torta                | TOR0074       |
| 847 | c45526      | 2023-09-05        | Rosa Ana Peña Araya                      | 13747054-K | Rosca                | ROS0047       |
| 848 | 19fd43      | 2023-09-13        | Kelly Zúñiga                             | 13719157-6 | Rosca                | ROS0058       |
| 849 | 44f6ca      | 2023-11-12        | Alexander Sánchez                        | 19039823-7 | Queque               | QUE0064       |
| 850 | 121baf      | 2023-06-06        | Priscilla Ayala Ahumada                  | 19951865-1 | Tarta                | TAR0014       |
| 851 | 1455ad      | 2023-08-24        | María Constanza Pino Vargas              | 11669269-4 | Brownie              | BRO0033       |
| 852 | d82fd3      | 2023-10-13        | Juana Fernanda Huerta Castillo           | 13793953-1 | Queque               | QUE0045       |
| 853 | 39b2ee      | 2023-10-11        | Carolina Díaz Muñoz                      | 11848319-4 | Rosca                | ROS0091       |
| 854 | 748e9b      | 2023-11-07        | Alejandro Astudillo Venegas              | 10170046-6 | Queque               | QUE0003       |
| 855 | b1dd69      | 2023-10-20        | Osvaldo Torres                           | 17925241-0 | Galleta              | GAL0089       |
| 856 | 4ea44d      | 2023-10-04        | Francisco Benjamín Viveros Salinas       | 16500516-0 | Brownie              | BRO0092       |
| 857 | a60129      | 2023-08-04        | María Karen Troncoso Erices              | 18251783-2 | Brownie              | BRO0005       |
| 858 | 9109af      | 2023-10-05        | Patricia Eugenia Concha González         | 17490750-0 | Queque               | QUE0090       |
| 859 | 82c44c      | 2023-11-14        | Pablo Valdés                             | 16390909-3 | Tarta                | TAR0017       |
| 860 | b3c6ad      | 2023-06-04        | Pía María Valdés Santos                  | 16089285-K | Brownie              | BRO0003       |
| 861 | 599d50      | 2023-09-28        | Delia Ximena Montes López                | 18572344-K | Torta                | TOR0055       |
| 862 | 8d3c58      | 2023-08-23        | José Moisés Martínez Araya               | 18348242-4 | Queque               | QUE0027       |
| 863 | ede300      | 2023-10-21        | María Julieta Pino Muñoz                 | 12851711-K | Rosca                | ROS0090       |
| 864 | a918f4      | 2023-09-11        | Milton Francisco Labra Álvarez           | 15429108-6 | Cupcake              | CUP0035       |
| 865 | 38b497      | 2023-09-15        | Roberto Darío Santander Cortés           | 18669242-9 | Cupcake              | CUP0052       |
| 866 | 7dd7bb      | 2023-06-03        | Jeannette Claudia Carmona Ortega         | 16965689-2 | Brownie              | BRO0096       |
| 867 | df8761      | 2023-11-04        | Camila Sánchez García                    | 15944745-0 | Queque               | QUE0013       |
| 868 | b371eb      | 2023-11-24        | Jesús Jaime Concha San Martín            | 16195147-7 | Cupcake              | CUP0050       |
| 869 | f1f132      | 2023-07-04        | Mónica Myriam Alvarado Sepúlveda         | 10317350-7 | Magdalena            | MAG0037       |
| 870 | 643ee8      | 2023-09-22        | Williams Bravo                           | 17963646-8 | Dona                 | DON0085       |
| 871 | 0f99fb      | 2023-07-16        | Edward Luis Salinas Espinoza             | 19063956-4 | Queque               | QUE0117       |
| 872 | b1b7ed      | 2023-11-06        | Luis Rojas                               | 18906449-9 | Brownie              | BRO0069       |
| 873 | a9d31a      | 2023-07-31        | Leonel Sergio Figueroa Cofré             | 13637138-5 | Brownie              | BRO0017       |
| 874 | 61c176      | 2023-07-24        | Felipe Juan Plaza Soto                   | 18821221-7 | Tarta                | TAR0063       |
| 875 | 3b5675      | 2023-11-08        | Sandra Letelier Muñoz                    | 19507484-8 | Galleta              | GAL0004       |
| 876 | 4efe03      | 2023-06-10        | Carla Carolina González Castro           | 10112939-7 | Tarta                | TAR0043       |
| 877 | acc3ee      | 2023-10-15        | Evelyn María López Farías                | 12136561-0 | Cupcake              | CUP0096       |
| 878 | 9c45ae      | 2023-10-08        | Osvaldo Torres                           | 17925241-0 | Dona                 | DON0122       |
| 879 | e4049a      | 2023-09-20        | Marta Maira Oñate Rodríguez              | 19170420-K | Queque               | QUE0103       |
| 880 | 050c03      | 2023-10-08        | Manuel Patricio Muñoz Ceballos           | 11859023-1 | Magdalena            | MAG0025       |
| 881 | 344f73      | 2023-07-28        | Verónica Muñoz Muñoz                     | 18581440-4 | Magdalena            | MAG0063       |
| 882 | cc7f38      | 2023-09-10        | Roxana Marcela Vergara Escobar           | 10201801-5 | Dona                 | DON0115       |
| 883 | 60d983      | 2023-11-13        | José Vásquez Campos                      | 18298087-0 | Queque               | QUE0096       |
| 884 | 4432ff      | 2023-09-17        | Sara Avilés                              | 17035051-K | Rosca                | ROS0054       |
| 885 | fce7e4      | 2023-09-01        | Luis Rojas                               | 18906449-9 | Magdalena            | MAG0048       |
| 886 | 8ba707      | 2023-06-16        | María Julieta Pino Muñoz                 | 12851711-K | Cupcake              | CUP0027       |
| 887 | fa8aa3      | 2023-08-28        | Jaime Villagra Cuevas                    | 11066531-9 | Brownie              | BRO0071       |
| 888 | f03738      | 2023-06-06        | Máximo Rodrigo Toledo Valenzuela         | 15327763-1 | Rosca                | ROS0094       |
| 889 | e2e718      | 2023-10-18        | Maximiliano Bustos                       | 12588084-0 | Cupcake              | CUP0067       |
| 890 | fe2535      | 2023-08-05        | Sergio Córdova                           | 12692991-6 | Tarta                | TAR0021       |
| 891 | 74b64a      | 2023-06-08        | Elsa Cecilia Bravo Celis                 | 16950266-3 | Galleta              | GAL0029       |
| 892 | 1f317e      | 2023-06-22        | Alma Álvarez                             | 12661877-5 | Cupcake              | CUP0012       |
| 893 | f208e7      | 2023-10-03        | Hugo Quispe Ceballos                     | 10800929-9 | Torta                | TOR0040       |
| 894 | f88dbb      | 2023-06-04        | Laura Osorio                             | 14952164-4 | Dona                 | DON0005       |
| 895 | 20279b      | 2023-09-20        | Ester María Reyes Guzmán                 | 10826143-4 | Queque               | QUE0032       |
| 896 | 2a3893      | 2023-11-17        | Noemí Martina Navarro-Olivares Rivera    | 12364781-4 | Tarta                | TAR0100       |
| 897 | 2b70ec      | 2023-06-07        | Rocío Bernal                             | 10754589-2 | Brownie              | BRO0006       |
| 898 | 90e3d5      | 2023-11-07        | Valentina Marín                          | 14937967-5 | Torta                | TOR0011       |
| 899 | bd25dc      | 2023-07-06        | Noemí Martina Navarro-Olivares Rivera    | 12364781-4 | Galleta              | GAL0075       |
| 900 | dbde7a      | 2023-08-23        | María Karen Troncoso Erices              | 18251783-2 | Torta                | TOR0093       |
| 901 | 00ccdf      | 2023-07-21        | Magaly Ruiz                              | 13052363-7 | Cupcake              | CUP0055       |
| 902 | fcae83      | 2023-10-27        | Alexander Sánchez                        | 19039823-7 | Cupcake              | CUP0010       |
| 903 | ce4faa      | 2023-11-01        | Augusto Arancibia                        | 14742034-8 | Brownie              | BRO0027       |
| 904 | c9d773      | 2023-07-13        | Ester Josefa Hidalgo González            | 18605721-K | Cupcake              | CUP0025       |
| 905 | 90e4f9      | 2023-09-07        | Ángela Arancibia Poblete                 | 15349562-4 | Dona                 | DON0093       |
| 906 | 256223      | 2023-11-25        | Jeanette Anita Toro Miranda              | 14019034-9 | Torta                | TOR0031       |
| 907 | 8dbb8d      | 2023-07-12        | Alondra Bahamondes Villar                | 18738327-1 | Cupcake              | CUP0006       |
| 908 | 8e7664      | 2023-07-04        | Camila Berríos Ramírez                   | 15115079-7 | Queque               | QUE0053       |
| 909 | 34bfd0      | 2023-08-02        | Catalina Riquelme-Barraza                | 17566730-1 | Torta                | TOR0078       |
| 910 | 5e92fb      | 2023-11-22        | María Consuelo Aguayo Márquez            | 16569601-K | Rosca                | ROS0033       |
| 911 | 2f2e2f      | 2023-09-29        | Daniela Millaray Pizarro González        | 18321458-7 | Dona                 | DON0091       |
| 912 | 93b644      | 2023-07-08        | Sofía Leiva                              | 12109194-8 | Galleta              | GAL0074       |
| 913 | bc1f92      | 2023-07-10        | Karen Nelly González Villarroel          | 14297342-5 | Queque               | QUE0004       |
| 914 | 3b321b      | 2023-09-29        | Carlos Henríquez Morales                 | 17129388-4 | Magdalena            | MAG0101       |
| 915 | 1bedda      | 2023-11-02        | Sandra Letelier Muñoz                    | 19507484-8 | Torta                | TOR0054       |
| 916 | 8d71c2      | 2023-08-07        | Iván José Pinto Lorca                    | 14648456-K | Brownie              | BRO0070       |
| 917 | 1e0c6b      | 2023-08-02        | Iván Espinoza Ruiz                       | 12063406-4 | Galleta              | GAL0029       |
| 918 | 463d04      | 2023-11-22        | Orlando Sergio Vargas Arancibia          | 14959522-8 | Brownie              | BRO0036       |
| 919 | de12ff      | 2023-11-09        | Brígida Gutiérrez                        | 10126260-K | Queque               | QUE0124       |
| 920 | 2a5917      | 2023-09-08        | Bianca Yanet Fica Solís                  | 13892497-8 | Dona                 | DON0120       |
| 921 | 59ae67      | 2023-10-21        | Jacqueline Catherine Villagrán Ibáñez    | 10870035-2 | Dona                 | DON0086       |
| 922 | b0dc55      | 2023-10-17        | Jesús Jaime Concha San Martín            | 16195147-7 | Brownie              | BRO0095       |
| 923 | aa7347      | 2023-09-25        | Francisco Paulo López Tapia              | 19249766-4 | Tarta                | TAR0092       |
| 924 | 453bd4      | 2023-09-16        | Anahí Véliz Carrasco                     | 11258752-9 | Tarta                | TAR0082       |
| 925 | c97352      | 2023-07-29        | Edward Luis Salinas Espinoza             | 19063956-4 | Cupcake              | CUP0090       |
| 926 | 0f0d21      | 2023-10-02        | Roxana Marcela Vergara Escobar           | 10201801-5 | Queque               | QUE0067       |
| 927 | 816723      | 2023-10-12        | Isabel Emilia González Lagos             | 15433080-6 | Tarta                | TAR0035       |
| 928 | f5c2fa      | 2023-10-25        | Alexander Jaramillo Ibarra               | 12315891-3 | Rosca                | ROS0078       |
| 929 | 93ce48      | 2023-09-08        | Matías Matamala Lagos                    | 10642186-7 | Galleta              | GAL0090       |
| 930 | 613b3a      | 2023-09-07        | Iván Bugueño Loyola                      | 18775936-K | Rosca                | ROS0076       |
| 931 | a07c89      | 2023-11-15        | Anahí Sofía Palma Mondaca                | 13271966-8 | Dona                 | DON0023       |
| 932 | ff7dff      | 2023-07-05        | Alexander Silva                          | 15316337-2 | Galleta              | GAL0083       |
| 933 | a441b9      | 2023-09-30        | Maximiliano Sánchez                      | 15319157-5 | Queque               | QUE0021       |
| 934 | ad5524      | 2023-08-06        | Brayan Pinto Cruz                        | 13687335-K | Queque               | QUE0014       |
| 935 | 1e8b3e      | 2023-08-10        | David José Merino Escobar                | 19684032-9 | Magdalena            | MAG0009       |
| 936 | 4bfd20      | 2023-08-26        | Ricardo Guillermo Gutiérrez Plaza        | 19475696-6 | Tarta                | TAR0010       |
| 937 | 892acf      | 2023-08-01        | Vanessa Contreras                        | 16499425-9 | Magdalena            | MAG0070       |
| 938 | 3dfdae      | 2023-06-30        | María Doris Arancibia Farías             | 14320333-3 | Rosca                | ROS0076       |
| 939 | 74217f      | 2023-07-28        | Georgina María Campos Cruz               | 15601503-6 | Queque               | QUE0040       |
| 940 | 5891c4      | 2023-06-19        | Erika Toledo                             | 12335263-8 | Tarta                | TAR0112       |
| 941 | c0e52a      | 2023-11-22        | Eliseo Jiménez Castillo                  | 16455110-K | Brownie              | BRO0001       |
| 942 | d4dbcb      | 2023-07-29        | Elizabeth Camila Arenas Barría           | 10190077-1 | Torta                | TOR0035       |
| 943 | ad4a22      | 2023-10-11        | María Maritza Arévalo Rodríguez          | 13042641-6 | Tarta                | TAR0047       |
| 944 | 551104      | 2023-08-06        | Delia Ximena Montes López                | 18572344-K | Tarta                | TAR0058       |
| 945 | c5f48c      | 2023-07-24        | Ana Torres-Salgado Vargas                | 12562967-9 | Torta                | TOR0086       |
| 946 | abd9f3      | 2023-06-28        | Jeanette Anita Toro Miranda              | 14019034-9 | Cupcake              | CUP0033       |
| 947 | d3aace      | 2023-09-20        | Edison Castillo Cruz                     | 14650302-4 | Cupcake              | CUP0081       |
| 948 | 48d74a      | 2023-08-01        | Karla Camila Zúñiga Saldivia             | 12067317-7 | Dona                 | DON0099       |
| 949 | 5752bc      | 2023-09-30        | Anaís Veas Chávez                        | 12300566-K | Queque               | QUE0101       |
| 950 | bb469b      | 2023-11-07        | Zoila Fernanda Figueroa Sánchez          | 15098937-8 | Tarta                | TAR0036       |
| 951 | 941645      | 2023-06-24        | Héctor Ignacio Varela Montecino          | 16682327-0 | Tarta                | TAR0060       |
| 952 | 657649      | 2023-08-23        | Manuel Oyarce                            | 13549533-8 | Magdalena            | MAG0029       |
| 953 | 25d632      | 2023-09-27        | María Patricia Ortega Molina             | 13562982-2 | Cupcake              | CUP0071       |
| 954 | 3d1ffb      | 2023-06-16        | Francisco Ricardo Silva Catalán          | 16747790-K | Magdalena            | MAG0088       |
| 955 | 6b3b18      | 2023-06-20        | Juan Matías León Romero                  | 18554189-3 | Galleta              | GAL0080       |
| 956 | 56a72e      | 2023-09-19        | Mónica María Moreno Quiroz               | 11875998-4 | Magdalena            | MAG0009       |
| 957 | 177817      | 2023-06-21        | Fiorella Carla Durán Ávila               | 18620251-4 | Dona                 | DON0118       |
| 958 | c0c894      | 2023-09-11        | Javiera Sabina Cortés Román              | 11472126-9 | Queque               | QUE0002       |
| 959 | 7466cd      | 2023-11-04        | Anahí Daniela Valdebenito Santander      | 14786002-3 | Torta                | TOR0068       |
| 960 | 374f15      | 2023-08-01        | Juan Matías León Romero                  | 18554189-3 | Magdalena            | MAG0019       |
| 961 | 5a937b      | 2023-08-27        | Lidia Santibáñez                         | 15879299-6 | Queque               | QUE0113       |
| 962 | 490d9f      | 2023-07-10        | Jacqueline Catherine Villagrán Ibáñez    | 10870035-2 | Galleta              | GAL0117       |
| 963 | 56bcff      | 2023-10-11        | Silvia Makarena Osses Guerrero           | 11368502-2 | Cupcake              | CUP0042       |
| 964 | af5fb3      | 2023-10-16        | Antonia Yasna Vidal Morales              | 19094951-5 | Tarta                | TAR0117       |
| 965 | 4d9296      | 2023-07-06        | Inés Alarcón Gutiérrez                   | 19036614-8 | Tarta                | TAR0048       |
| 966 | 5640a6      | 2023-06-18        | María Reyes Salazar                      | 18592887-1 | Galleta              | GAL0081       |
| 967 | fc880a      | 2023-09-14        | Claudio Abarca-Ferrada                   | 16315349-3 | Cupcake              | CUP0056       |
| 968 | 2c7935      | 2023-09-23        | Margarita Claudia Vargas Barrera         | 12595740-5 | Galleta              | GAL0072       |
| 969 | f8d627      | 2023-10-06        | Bianca Yanet Fica Solís                  | 13892497-8 | Queque               | QUE0043       |
| 970 | 8ea7c5      | 2023-08-09        | Marcelo Rubio Zúñiga                     | 12100935-7 | Queque               | QUE0108       |
| 971 | 303df8      | 2023-06-18        | Anahí Véliz Carrasco                     | 11258752-9 | Rosca                | ROS0085       |
| 972 | f22fa8      | 2023-11-16        | Antonia Rivera Peralta                   | 16353978-2 | Dona                 | DON0112       |
| 973 | feaa5a      | 2023-06-18        | Verónica Muñoz Muñoz                     | 18581440-4 | Rosca                | ROS0014       |
| 974 | 581df8      | 2023-11-19        | Claudio Sebastián Sánchez Calderón       | 12687889-1 | Galleta              | GAL0015       |
| 975 | 836548      | 2023-10-11        | Rosa Romina Díaz Guerrero                | 13372470-9 | Brownie              | BRO0095       |
| 976 | f7674b      | 2023-11-11        | Joaquín Sáez                             | 11514995-8 | Galleta              | GAL0083       |
| 977 | e8d2d3      | 2023-08-26        | Benjamin Henríquez Rojas                 | 11818130-7 | Rosca                | ROS0055       |
| 978 | 7ae0fb      | 2023-08-19        | Georgina Consuelo Cornejo Parada         | 17984033-3 | Cupcake              | CUP0089       |
| 979 | fd0faf      | 2023-08-14        | Jorge Guzmán Moraga                      | 12681819-7 | Dona                 | DON0059       |
| 980 | 3a3e56      | 2023-11-13        | Juan Matías León Romero                  | 18554189-3 | Tarta                | TAR0090       |
| 981 | e9ab4a      | 2023-08-20        | Raúl Urbina                              | 15616810-3 | Cupcake              | CUP0034       |
| 982 | 14075e      | 2023-10-02        | Nolberto Berríos Arévalo                 | 15642175-5 | Rosca                | ROS0081       |
| 983 | 1aefed      | 2023-09-21        | Arturo Poblete Paredes                   | 17935901-9 | Tarta                | TAR0064       |
| 984 | a30773      | 2023-11-25        | Jeannette Sanhueza Norambuena            | 12152763-0 | Galleta              | GAL0085       |
| 985 | 443364      | 2023-08-16        | Macarena Pérez Águila                    | 14918445-4 | Galleta              | GAL0010       |
| 986 | b9c1c8      | 2023-07-07        | Elba Silvana Valenzuela Riffo            | 11074696-8 | Tarta                | TAR0052       |
| 987 | 663e66      | 2023-07-28        | Jeanette Anita Toro Miranda              | 14019034-9 | Dona                 | DON0001       |
| 988 | 784039      | 2023-11-25        | Marcia Erika Morales Arancibia           | 16626844-0 | Dona                 | DON0122       |
| 989 | bde47a      | 2023-09-09        | Berta Martina Marín Hernández            | 17284393-9 | Tarta                | TAR0042       |
| 990 | 993f0a      | 2023-07-29        | María Teresa Sánchez Valencia            | 14977234-7 | Torta                | TOR0009       |
| 991 | d5f28c      | 2023-07-09        | Antonia Figueroa Vergara                 | 13984705-5 | Brownie              | BRO0049       |
| 992 | 090f39      | 2023-06-26        | Pilar Cortés González                    | 18701866-4 | Brownie              | BRO0066       |
| 993 | b3fef7      | 2023-08-05        | Christian Aranda                         | 16899941-9 | Queque               | QUE0102       |
| 994 | 5122bb      | 2023-08-14        | María Doris Arancibia Farías             | 14320333-3 | Torta                | TOR0087       |
| 995 | 2efd4a      | 2023-07-01        | Reinaldo Carlos Muñoz Reyes              | 13563799-8 | Brownie              | BRO0011       |
| 996 | cf9b2d      | 2023-09-15        | María Castillo Ulloa                     | 12782005-0 | Magdalena            | MAG0083       |
| 997 | c447f7      | 2023-10-11        | Viviana Maritza Díaz Carreño             | 14375847-K | Queque               | QUE0111       |
| 998 | 53fcdb      | 2023-06-11        | Hilda Albornoz                           | 14313449-3 | Magdalena            | MAG0029       |
| 999 | cab013      | 2023-06-14        | José Soto                                | 18468389-8 | Galleta              | GAL0038       |